In [6]:
class xoodyak_kun:


    # Inisialisasi dan preprocess data
    def input_message(self, message):
        self.msg = message
        return self.msg

    def preprocess(self):
        message_bit = ''
        for i in range(len(self.msg)):
            binary_msg = bin(ord(self.msg[i]))[2:]
            while len(binary_msg) < 8:
                binary_msg = '0' + binary_msg
            message_bit += (binary_msg)
        return message_bit

    # Commonly Used Functions
    def rot(self, msg, shift_n_bit): #right shift
        msg_list=['' for i in range(len(msg))]
        msg_new=''
        for i in range(len(msg)):
            msg_list[(i+shift_n_bit)%(len(msg))]= msg[i]
        for i in range(len(msg)):
            msg_new+=msg_list[i]
        return msg_new
    
    def xor(self, msg1, msg2):
        msg_xor = ''
        assert len(msg1) == len(msg2), f"length of both message are not same"
        for i in range(len(msg1)):
            if (msg1[i] == '1' and msg2[i] == '0') or ((msg1[i] == '0' and msg2[i] == '1')):
                msg_xor += '1'
            else:
                msg_xor += '0'
        return msg_xor

    def and_bit(self, msg1, msg2):
        msg_and = ''
        assert len(msg1) == len(msg2), f"length of both message are not same"
        for i in range(len(msg1)):
            if (msg1[i] == '1' and msg2[i] == '1'):
                msg_and += '1'
            else:
                msg_and += '0'
        return msg_and

    def or_bit(self, msg1, msg2):
        msg_or = ''
        assert len(msg1) == len(msg2), f"length of both message are not same"
        for i in range(len(msg1)):
            if (msg1[i] == '0' and msg2[i] == '0'):
                msg_or += '0'
            else:
                msg_or += '1'
        return msg_or

    def not_bit(self, msg):
        msg_not = ''
        for i in range(len(msg)):
            if msg[i] == '0':
                msg_not += '1'
            else:
                msg_not += '0'
        return msg_not     

    def hextobit(self, X):
        hexvalues = ['0', '1', '2','3' ,'4' ,'5' ,'6' ,'7' ,'8' ,'9', 'a' ,'b' ,'c' ,'d' ,'e' ,'f']
        bithexvalues = ['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']
        out=''
        for i in range(len(X)):
            for j in range(len(hexvalues)):
                if X[i]==hexvalues[j]:
                    out+=bithexvalues[j]
        return out 

    def bittohex(self, X):
        hexvalues = ['0', '1', '2','3' ,'4' ,'5' ,'6' ,'7' ,'8' ,'9', 'a' ,'b' ,'c' ,'d' ,'e' ,'f']
        bithexvalues = ['0000', '0001', '0010', '0011', '0100', '0101', '0110', '0111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111']
        out=''
        if len(X)%4 == 0:
            for i in range(int(len(X)/4)):
                temp=''
                for k in range(4):
                    temp+=X[4*i+k]
                for j in range(len(bithexvalues)):
                    if temp == bithexvalues[j]:
                        out+=hexvalues[j]
            return out

    def bittomatrix(self, A,  w):
        temp=[['' for j in range(5)]for i in range(5)]
        for i in range(5):
            for j in range(5):
                for k in range(w):
                    if (w-k-1)+w*(4-j+5*(4-i))<len(A):
                        temp[i][j]+= A[(w-k-1)+w*(4-j+5*(4-i))]
        for i in range(5):
            for j in range(5):
                if len(temp[i][j])<w:
                    while len(temp[i][j])<w:
                        temp[i][j]+='0'             
        return temp

    def matrixtobit(self, A, w):
        temp=''
        for i in range(5):
            for j in range(5):
                for k in range(w):
                    temp+=A[4-i][4-j][w-k-1]
        return temp


    # SHA-3 (Keccak) Functions
    def teta(self, A, initial_state):
        C=[initial_state for j in range(5)]
        D=[initial_state for j in range(5)]
        for i in range(5):
            C[i]=self.xor(self.xor(self.xor(self.xor(A[i][0],A[i][1]),A[i][2]),A[i][3]),A[i][4])
        for i in range(5):
            D[i]=self.xor(C[(i-1)%5],self.rot(C[(i+1)%5],1))
        for i in range(5):
            for j in range(5):
                A[i][j]=self.xor(A[i][j],D[i]) 
        return A #theta()[0] is C and theta()[1] is D


    def rhopi(self, A):
        ra=[[0,36,3,41,18],[1,10,44,45,2],[62,6,43,15,61],[28,55,25,21,56],[27,20,39,8,14]] 
        B=[['' for j in range(5)] for i in range(5)]
        for i in range(5):
            for j in range(5):
                B[j][int((2*i+3*j)%5)]=self.rot(A[i][j],ra[i][j])
            
        return B

    def chi(self, A, B1):
        for i in range(5):
            for j in range(5):
                A[0][0]=self.xor(B1[0][0],self.and_bit(self.not_bit(B1[(i+1)%5][j]),B1[(i+2)%5][j]))
        return A

    def iota(self, A, t, RCbit):
        A[0][0]=self.xor(A[0][0],RCbit[t])
        return A

    def permutate(self, A, t, initial_state, RCbit):
        A=self.teta(A, initial_state)
        B1=self.rhopi(A)
        A=self.chi(A, B1)
        A=self.iota(A,t, RCbit)
        return A
        
    def keccak(self):
        l= 6 #exponent of 2 for z direction w = 2^6
        nr= 12 + 2*l #number of rounds
        w= 2**l #z direction
        b= 5*5*(2**l) #state size
        r= 1088 #bit rate
        c=b-r 
        z_size=384
        toy_bits=self.preprocess()
        p_bit=len(toy_bits)%r
        toy_bits+='11001'
        toy_bits+='1'
        for i in range(r-7-(p_bit)): 
            toy_bits+='0'
        toy_bits+='1'
        rounds=0

        initial_state=''
        for i in range(w):
            initial_state+='0'
        State=[[initial_state for j in range(5)] for i in range(5)]
        #rotation constant
        ra=[[0,36,3,41,18],[1,10,44,45,2],[62,6,43,15,61],[28,55,25,21,56],[27,20,39,8,14]] 
        #round constant
        RChex=['0000000000000001','0000000000008082','800000008000808a', '8000000080008000', '0000000000000808B', '0000000080000001', '8000000080008081', '8000000000008009', '000000000000008a', '0000000000000088', '0000000080008009', '000000008000000a', '000000008000808b', '800000000000008b', '8000000000008089', '8000000000008003', '8000000000008002', '8000000000000080', '000000000000800a', '800000008000000a', '8000000080008081', '8000000000008080', '0000000080000001', '8000000080008008']
        RCbit=['' for i in range(nr)]
        for i in range(nr):
            RCbit[i]=self.hextobit(RChex[i])

        while rounds*r<len(toy_bits): #absorb
            State = self.matrixtobit(State, w)
            tempState=''
            for i in range(r):
                tempState+=self.xor(State[i],toy_bits[i+r*rounds])
            for i in range(r, b):
                tempState+=State[i]
            State=self.bittomatrix(tempState, w)
            for t in range(nr):    
                State=self.permutate(State, t, initial_state, RCbit)
            rounds+=1

        #squeeze
        for t in range(nr):
            State = self.permutate(State, t, initial_state, RCbit)
        squeeze=''
        for i in range(z_size):
            squeeze+=self.matrixtobit(State, w)[i]
        return self.bittohex(squeeze), squeeze
                










    # Xoodoo Permutation
    def theta(self, Arr, X, Z, Y):

        P = [[0 for x in range(X)] for z in range(Z)]
        E = [[0 for x in range(X)] for z in range(Z)]
        for x in range(0,X) :
            for z in range(0,Z) :
                for y in range(0,Y) :
                    P[z][x] += Arr[y][z][x]
                    P[z][x] = P[z][x]%2
                        #P[x][0][z] <- P[x][0][z] XOR A[x][y][z]

        for x in range(0,X) :
            for z in range(0,Z) :
                if 5<=z<14 and x<1:
                    E[z][x] = P[z-5][x+3] + P[z+18][x+3]
                if 5<=z<14 and x>=1 :
                    E[z][x] = P[z-5][x-1] + P[z+18][x-1]
                if z<5 and x>=1 :
                    E[z][x] = P[z+27][x-1] + P[z+18][x-1]
                if z<5 and x<1 :
                    E[z][x] = P[z+27][x+3] + P[z+18][x+3]
                if z>=14 and x<1:
                    E[z][x] = P[z-5][x+3] + P[z-14][x+3]
                else:
                    E[z][x] = P[z-5][x-1] + P[z-14][x-1]
                E[z][x] =  E[z][x]%2
                #E[x][y][z] <- P[x-1][y][z-5] XOR P[x-1][y][z-14]
  
        for y in range(0,Y) :
            for z in range(0,Z) :
                for x in range(0,X) :
                    Arr[y][z][x] =+ E[z][x]
                    Arr[y][z][x] = Arr[y][z][x]%2
                    #A[x][y][z] <- A[x][y][z] + E[x][y][z]

        return Arr
    
    def Ci(self, s):
        Arres = [0 for z in range(len(s))]
        Arrbit = [c for c in s]
        j=0
        for z in range(len(s)) :
            Arres[z] = int(Arrbit[j])
            j += 1
        return Arres

    def split_str(self, s):
        return [c for c in s]

    def get_constant_xoodoo(self):
        C = [0 for i in range(12)]
        C[0] = self.Ci('00011010000000000000000000000000')
        C[1] = self.Ci('00011100000000000000000000000000')
        C[2] = self.Ci('00000011110000000000000000000000')
        C[3] = self.Ci('00001011000000000000000000000000')
        C[4] = self.Ci('00000100100000000000000000000000')
        C[5] = self.Ci('00101000000000000000000000000000')
        C[6] = self.Ci('00000110000000000000000000000000')
        C[7] = self.Ci('00110100000000000000000000000000')
        C[8] = self.Ci('00000001110000000000000000000000')
        C[9] = self.Ci('00001111000000000000000000000000')
        C[10] = self.Ci('00000101100000000000000000000000')
        C[11] = self.Ci('01001000000000000000000000000000')
        return C

    def rhowest(self, Arr, X, Y, Z, C, round) :
        newArr = [[[0 for x in range(X)] for z in range(Z)] for y in range(Y)]
        for x in range(0,X) :
            for z in range(0,Z) :
                newArr[0][z][x] += C[round][z] 
                if x<1:
                    newArr[1][z][x] = Arr[1][z][x+3]
                else:
                    newArr[1][z][x] = Arr[1][z][x-1]
                if z<11:
                    newArr[2][z][x] = Arr[2][z+21][x]
                else:
                    newArr[2][z][x] = Arr[2][z-11][x]
                        #A[x][0][z] <- A[x][0][z] + Ci
                        #A[x][1][z] <- A[x-1][1][z]
                        #A[x][2][z] <- A[x][1][z-11]

        for x in range(0,X) :
            for z in range(0,Z) :
                for y in range(0,Y) :
                    newArr[y][z][x] = newArr[y][z][x]%2
        return newArr

    def Chi(self, Arr, X, Y, Z) :
        B = [[[0 for x in range(X)] for z in range(Z)] for y in range(Y)]
        notArr = [[[0 for x in range(X)] for z in range(Z)] for y in range(Y)]
        for x in range(0,X) :
            for z in range(0,Z) :
                for y in range(0,Y) :  
                    notArr[y][z][x] = Arr[y][z][x]
                    notArr[y][z][x] = notArr[y][z][x]

        for x in range(0,X) :
            for z in range(0,Z) :  
                if notArr[1][z][x] and Arr[2][z][x] :
                    B[0][z][x] = 1
                else :
                    B[0][z][x] = 0
                if notArr[2][z][x] and Arr[0][z][x] :
                    B[1][z][x] = 1
                else :
                    B[1][z][x] = 0
                if notArr[0][z][x] and Arr[1][z][x] :
                    B[2][z][x] = 1
                else :
                    B[2][z][x] = 0
                            #B[x][0][z] <- not(A[x][1][z]) AND A[x][2][z]
                            #B[x][1][z] <- not(A[x][2][z]) AND A[x][0][z]
                            #B[x][2][z] <- not(A[x][0][z]) AND A[x][1][z]

        for x in range(0,X) :
            for z in range(0,Z) :
                for y in range(0,Y) :  
                    Arr[y][z][x] += B[y][z][x]
                    Arr[y][z][x] = Arr[y][z][x]%2

        return Arr

    def rhoeast(self, Arr, X, Y, Z) :
        newArr = [[[0 for x in range(X)] for z in range(Z)] for y in range(0,Y)]
        for z in range(0,Z) :
            for x in range(0,X) :
                if x<1:
                    newArr[1][z][x] = Arr[1][z][x+3]
                else:
                    newArr[1][z][x] = Arr[1][z][x-1]

                if z<8 and x<2:
                    newArr[2][z][x] = Arr[2][z+24][x+2]
                elif z<8 and x>=2:
                    newArr[2][z][x] = Arr[2][z+24][x-2]
                elif z>=8 and x<2:
                    newArr[2][z][x] = Arr[2][z-8][x+2]
                else:
                    newArr[2][z][x] = Arr[2][z-8][x-2]
                newArr[0][z][x] = Arr[0][z][x]
                #A[x][1][z] <- A[x][1][z-1]
                        #A[x][2][z] <- A[x-2][1][z-8]
        return newArr

    def xoodoo_permutation(self, X=4, Y=3, Z=32):

        Xoodoo=[[[0 for x in range(X)] for z in range(Z)] for y in range(Y)]
        msg = self.keccak()[1]
        C = self.get_constant_xoodoo()
        Arrbit = self.split_str(msg)

        for y in range(0) :
            for x in range(0,X) :
                for z in range(0,Z) :
                    Xoodoo[y][z][x] = 0

        k=0
        for y in range(1,Y) :
            for x in range(0,X) :
                for z in range(0,Z) :
                    Xoodoo[y][z][x] = int(Arrbit[k])
                    k += 1

        for i in range(12): # round for 12 times
            Xoodoo = self.theta(Xoodoo,X,Z,Y)
            Xoodoo = self.rhowest(Xoodoo,X,Y,Z,C,i)
            Xoodoo = self.Chi(Xoodoo,X,Y,Z)
            Xoodoo = self.rhoeast(Xoodoo,X,Y,Z)

        feedback=''

        for y in range(0,Y) :
            for x in range(0,X) :
                for z in range(0,Z) :
                    feedback += str(Xoodoo[y][z][x])

        return self.bittohex(feedback)
    

In [7]:
message1 = xoodyak_kun() # SHA-384 (Keccak)

In [8]:
text_2 = """
Lompat ke isi
Buka/tutup bilah samping

WikipediaEnsiklopedia Bebas
Telusuri Wikipedia
Buat akun baru

Perkakas pribadi

Ikuti Wikipedia bahasa Indonesia di F icon.svg Facebook, Twitter-logo.svg Twitter, Instagram simple icon.svg Instagram, dan Telegram logo.svg Telegram
Tutup
Daftar isi sembunyikan
Awal
Silsilah Jayakatwang
Penguasa Gelanggelang
Pemberontakan Jayakatwang
Kekalahan Jayakatwang
Lihat pula
Referensi
Jayakatwang

Halaman
Pembicaraan
Baca
Perubahan tertunda
Sunting
Sunting sumber
Lihat riwayat
Dari Wikipedia bahasa Indonesia, ensiklopedia bebas
Jayakatwang
Penguasa Gelanggelang
Berkuasa	1271 - 1293
Kelahiran	Daha, Kediri
Jawa Timur
Kematian	1293
Hujung Galuh, Surabaya, Jawa Timur
Wangsa	Isyana
Ayah	Sastrajaya (putra Jayashaba putra Kertajaya)
Pasangan	Nararya Turuk Bali
Anak	Ardharaja
Agama	Hindu-Buddha
Jayakatwang adalah bupati Gelanggelang (kini termasuk wilayah Madiun) yang pada tahun 1292 memberontak dan meruntuhkan kerajaan Singhasari, untuk membangkitkan kembali kerajaan leluhurnya, yaitu Kadiri, tetapi hanya bertahan selama setahun sebelum dihancurkan oleh pasukan gabungan Mongol dan Majapahit.

Silsilah Jayakatwang
Jayakatwang juga sering kali disebut dengan nama Sanjaya, Aji Katong, atau Jayakatyeng. Dalam berita Tiongkok ia disebut Ha-ji-ka-tang.

Nagarakretagama dan Kidung Harsawijaya menyebutkan Jayakatwang adalah keturunan Kertajaya raja terakhir Kadiri. Dikisahkan pada tahun 1222 Ken Arok mengalahkan Kertajaya. Sejak itu Kadiri menjadi bawahan Singhasari di mana sebagai bupatinya adalah Jayasabha putra Kertajaya. Tahun 1258 Jayasabha digantikan putranya yang bernama Sastrajaya. Pada tahun 1271 Sastrajaya digantikan putranya, yaitu Jayakatwang.

Ayah Jayakatwang, Sastrajaya, menikah dengan saudara perempuan Wisnuwardhana, karena dalam prasasti Mula Malurung Jayakatwang disebut sebagai keponakan Seminingrat (nama lain Wisnuwardhana). Prasasti itu juga menyebutkan nama istri Jayakatwang adalah Nararya Turuk Bali putri Seminingrat. Dari prasasti Kudadu diketahui Jayakatwang memiliki putra bernama Ardharaja, yang menjadi menantu Kertanagara. Jadi hubungan antara Jayakatwang dengan Kertanagara adalah sepupu, sekaligus ipar, sekaligus besan.

Penguasa Gelanggelang
Nagarakretagama, Pararaton, Kidung Harsawijaya, dan Kidung Panji Wijayakrama menyebut Jayakatwang adalah raja bawahan di Kadiri yang memberontak terhadap Kertanagara di Singhasari. Naskah prasasti Kudadu dan prasasti Penanggungan menyebut Jayakatwang pada saat memberontak masih menjabat sebagai bupati Gelang-Gelang. Setelah Singhasari runtuh, baru kemudian ia menjadi raja di Kadiri.

Sempat muncul pendapat bahwa Gelang-Gelang merupakan nama lain dari Kadiri. Namun gagasan tersebut digugurkan oleh naskah prasasti Mula Malurung (1255). Dalam prasasti itu dinyatakan dengan tegas kalau Gelang-Gelang dan Kadiri adalah dua wilayah yang berbeda. Prasasti itu menyebutkan kalau saat itu Kadiri diperintah Kertanagara sebagai yuwaraja (raja muda), sedangkan Gelang-Gelang diperintah oleh Hurukbali dan Jayakatwang.

Lagi pula lokasi Kadiri berada di daerah Kediri, sedangkan Gelang-Gelang ada di daerah Madiun. Kedua kota tersebut terpaut jarak puluhan kilometer.

Pemberontakan Jayakatwang
Pararaton dan Kidung Harsawijaya menceritakan Jayakatwang menyimpan dendam karena leluhurnya Kertajaya Kadiri dikalahkan Ken Arok pendiri Singhasari. Suatu hari ia menerima kedatangan Wirondaya putra Aria Wiraraja yang menyampaikan surat dari ayahnya sebagai balasan "formal" terhadap permintaan pertimbangan yang diajukan Jayakatwang sebelumnya, mengingat Aria Wiraraja adalah dianggap sesepuh Jayakatwang. Dimana isi pertanyaan surat sebelumnnya mungkinkah Jayakatwang bisa melakukan balas dendam terhadap Kertanegara akibat kekuasaan Kadiri yang merupakan leluhur Jayakatwang telah ditaklukkan Singhasari leluhur dari Kertanegara, Atas pertanyaan ini Aria Wiraraja menyarankan supaya Jayakatwang jika telah terpikirkan secara matang segera melakukan penyerangan karena saat itu Singhasari sedang dalam keadaan kosong, ditinggal sebagian besar pasukannya ke luar Jawa. Adapun Aria Wiraraja adalah mantan pejabat Singhasari yang dimutasi ke Sumenep karena dianggap sebagai penentang politik Kertanagara. Yang pada akhirnya di kemudian hari Aria Wiraraja menyayangkan dan sangat menyesali terhadap apa yang dilakukannya dengan Jayakatwang.

Jayakatwang melaksanakan saran Aria Wiraraja. Ia mengirim pasukan kecil yang dipimpin Jaran Guyang menyerbu Singhasari dari utara. Mendengar hal itu, Kertanagara segera mengirim pasukan untuk menghadapi yang dipimpin oleh menantunya, bernama Raden Wijaya. Pasukan Jaran Guyang berhasil dikalahkan. Namun sesungguhnya pasukan kecil ini hanya bersifat pancingan supaya pertahanan kota Singhasari kosong.

Pasukan kedua Jayakatwang menyerang Singhasari dari arah selatan dipimpin oleh Patih Mahisa Mundarang. Dalam serangan tak terduga ini, Kertanagara tewas di dalam istananya.

Menurut prasasti Kudadu, Ardharaja putra Jayakatwang yang tinggal di Singhasari bersama istrinya, ikut serta dalam pasukan Raden Wijaya. Tentu saja ia berada dalam posisi sulit karena harus menghadapi pasukan ayahnya sendiri. Ketika mengetahui kekalahan Singhasari, Ardaraja berbalik meninggalkan Raden Wijaya dan memilih bergabung dengan pasukan Gelang-Gelang.

Kekalahan Jayakatwang
Peristiwa kehancuran Singhasari terjadi tahun 1292. Jayakatwang lalu menjadi raja, dengan Kadiri sebagai pusat pemerintahannya. Atas saran Aria Wiraraja, Jayakatwang memberikan pengampunan kepada Raden Wijaya yang datang menyerahkan diri. Raden Wijaya kemudian diberi alas Trik (Hutan Tarik, Sidoarjo) untuk dibuka menjadi kawasan wisata perburuan.

Sesungguhnya Aria Wiraraja telah berbalik melawan Jayakatwang. Saat itu Wiraraja ganti membantu Raden Wijaya untuk merebut kembali takhta peninggalan mertuanya. Pada tahun 1293 pasukan Mongol datang untuk menghukum Kertanagara yang telah berani menyakiti utusan Kubilai Khan tahun 1289. Pasukan Mongol tersebut diterima Raden Wijaya di desanya yang bernama Majapahit. Raden Wijaya yang mengaku sebagai ahli waris Kertanagara bersedia menyerahkan diri kepada Kubilai Khan asalkan terlebih dahulu dibantu mengalahkan Jayakatwang.

Berita Tiongkok menyebutkan perang terjadi pada tanggal 20 Maret 1293. Gabungan pasukan Mongol dan Majapahit menggempur kota Kadiri sejak pagi hari. Sekitar 5000 orang Kadiri tewas menjadi korban. Akhirnya pada sore harinya, Jayakatwang menyerah dan ditawan di atas kapal Mongol.

Dikisahkan kemudian pasukan Mongol ganti diserang balik oleh pihak Majapahit untuk diusir keluar dari tanah Jawa. Sebelum meninggalkan Jawa, pihak Mongol sempat menghukum mati Jayakatwang dan Ardharaja di atas kapal mereka.

Menurut kitab Pararaton dan Kidung Panji Wijayakrama, Jayakatwang yang telah menyerah lalu ditawan di benteng pertahanan Mongol di Hujung Galuh. Menurut Pararaton dan Kidung Harsawijaya, ia meninggal di dalam tahanan penjara Hujung Galuh setelah menyelesaikan sebuah karya sastra berjudul Kidung Wukir Polaman.

Lihat pula
Invasi Yuan-Mongol ke Jawa
Referensi
Slamet Muljana. 1979. Nagarakretagama dan Tafsir Sejarahnya. Jakarta: Bhratara.
Slamet Muljana. 2005. Menuju Puncak Kemegahan(terbitan ulang 1965). Yogyakarta: LKIS.
Poesponegoro, M.D., Notosusanto, N. (editor utama). Sejarah Nasional Indonesia. Edisi ke-4. Jilid II. Jakarta: Balai Pustaka, 1990.
Didahului oleh:
Kertajaya	Raja Kadiri
1292-1293	Diteruskan oleh:
Akhir riwayat Kadiri serta wangsa Isyana digantikan Majapahit
Kategori: Raja KadiriKerajaan MajapahitTokoh Jawa TimurTokoh JawaKerajaan KadiriKerajaan Singhasari
Halaman ini terakhir diubah pada 10 November 2022, pukul 07.22.
Teks tersedia di bawah Lisensi Creative Commons Atribusi-BerbagiSerupa; ketentuan tambahan mungkin berlaku. Lihat Ketentuan Penggunaan untuk lebih jelasnya.
Kebijakan privasiTentang WikipediaPenyangkalanTampilan selulerPengembangStatistikPernyataan kukiWikimedia FoundationPowered by MediaWiki

Lompat ke isi
Buka/tutup bilah samping

WikipediaEnsiklopedia Bebas
Telusuri Wikipedia
Buat akun baru

Perkakas pribadi

Ikuti Wikipedia bahasa Indonesia di F icon.svg Facebook, Twitter-logo.svg Twitter, Instagram simple icon.svg Instagram, dan Telegram logo.svg Telegram
Tutup
Daftar isi sembunyikan
Awal
Nama Kerajaan
Pendirian Kerajaan
Silsilah Wangsa Rajasa
Toggle Silsilah Wangsa Rajasa subsection
Versi Pararaton
Versi Kakawin Nagarakretagama
Pemerintahan bersama
Masa Kejayaan
Keruntuhan
Hubungan dengan Majapahit
Warisan Budaya
Toggle Warisan Budaya subsection
Arca
Candi
Prasasti
Kutipan
Referensi
Pranala luar
Kerajaan Singasari

Halaman
Pembicaraan
Baca
Sunting
Sunting sumber
Lihat riwayat
Dari Wikipedia bahasa Indonesia, ensiklopedia bebas
(Dialihkan dari Kerajaan Singhasari)
Untuk kegunaan lain, lihat Tumapel.
Kerajaan Singhasari
1222–1292
Perkembangan Kerajaan Singhasari pada masa pemerintahan Kertanagara.
Perkembangan Kerajaan Singhasari pada masa pemerintahan Kertanagara.
Ibu kota	Kutaraja yang lalu berganti nama Singhasari
Bahasa yang umum digunakan	Jawa Kuno, Sanskerta
Agama	Siwa-Buddha (Hindu dan Buddha), Kapitayan, Animisme, Tantra Bhairawa
Pemerintahan	Monarki
Sri/Raja	 
• 1222–1227
Ken Arok
• 1227–1248
Anusapati
• 1248–1268
Wisnuwardhana
• 1268–1292
Kertanegara
Sejarah	 
• Awal berdiri oleh Pemberontakan Ken Arok
1222
• Ekspedisi Pamalayu
1275–1286
• Ekspedisi Pabali
1282–1284
• Runtuh oleh pemberontakan Jayakatwang dari Gelanggelang
1292
Mata uang	Koin emas dan perak
Didahului oleh	Digantikan oleh
	Kerajaan Kadiri
Kerajaan Majapahit	
Sekarang bagian dari	 Indonesia
 Malaysia

 Singapura

Arca Prajnaparamita ditemukan dekat candi Singhasari dipercaya sebagai arca perwujudan Ken Dedes (koleksi Museum Nasional Indonesia). Keindahan arca ini mencerminkan kehalusan seni budaya Singhasari.
Kerajaan Singasari (bahasa Jawa: ꦏꦫꦠꦺꦴꦤ꧀ꦱꦶꦔꦱꦫꦶ, translit. Karaton Singhasari) atau Kerajaan Tumapel, merupakan sebuah kerajaan bercorak Hindu-Buddha yang didirikan oleh Ken Arok atau disebut juga sebagai Ken Angrok pada tahun 1222. Sejarah kerajaan ini terkait erat dengan sosok Ken Angrok (1222–1247) yang adalah pendiri Wangsa Rajasa sekaligus kerajaan Tumapel. Lokasi dari kerajaan ini sekarang diperkirakan berada di daerah Kecamatan Singasari, Kabupaten Malang, Provinsi Jawa Timur.

Nama Kerajaan
Berdasarkan keterangan dalam Prasasti Kudadu, nama resmi Kerajaan Singhasari adalah Kerajaan Tumapel. Nama Tumapel juga muncul dalam berita Tiongkok dari Dinasti Yuan dengan ejaan Tu-ma-pan. Kakawin Nagarakretagama memperjelas jika sesungguhnya ibu kota Tumapel bernama Kutaraja ketika pertama kali didirikan tahun 1222.[1][2]

Pada 1253, Wisnuwardhana mengganti nama ibu kota kerajaan menjadi Singhasari. Nama Singhasari yang merupakan nama ibu kota kemudian justru lebih terkenal daripada nama Tumapel. Inilah yang membuat Tumapel juga lebih dikenal dengan nama Kerajaan Singasari.

Pendirian Kerajaan
Pararaton menyebut Tumapel awalnya hanya sebuah daerah bawahan Kerajaan Kadiri. Adapun yang menjabat sebagai akuwu (setara camat) Tumapel saat itu adalah Tunggul Ametung. Dia mati dibunuh dengan cara tipu muslihat oleh pengawalnya sendiri, yaitu Ken Angrok, yang kemudian mengangkat dirinya menjadi raja pertama Tumapel dengan gelar Sri Ranggah Rajasa Bhatara Sang Amurwabhumi.

Ken Angrok lantas menikahi janda Tunggul Ametung yang saat itu sedang mengandung, yaitu Ken Dedes. Anak Ken Dedes dari Tunggul Ametung ini nantinya diberi nama Anusapati. Selain beristrikan Ken Dedes, Ken Angrok mempunyai satu istri lagi bernama Ken Umang yang kelak melahirkan anak laki-laki bernama Tohjaya.

Ketika berkuasa, Ken Angrok berniat melepaskan Tumapel dari kekuasaan Kadiri. Pada 1221, terjadi perseteruan antara Kertajaya, raja Kerajaan Panjalu, dengan kaum brahmana. Para brahmana lantas menggabungkan diri dengan Ken Angrok. Perang melawan Kadiri lantas meletus di Desa Ganter pada 1222 yang dimenangkan oleh pihak Tumapel.

Nagarakretagama juga menyebut tahun yang sama untuk pendirian Tumapel, tetapi tidak menyebutkan adanya nama Ken Angrok. Dalam naskah itu, pendiri Tumapel bernama Ranggah Rajasa Sang Girinathaputra yang berhasil mengalahkan Kertajaya, raja Kadiri.

Pada 1253, Wisnuwardhana kemudian mengangkat putranya yang bernama Kertanagara sebagai yuwaraja (putra mahkota) dan mengganti nama ibu kota kerajaan menjadi Singhasari. Nama Singhasari yang merupakan nama ibu kota kemudian justru lebih terkenal daripada nama Tumapel. Inilah yang membuat Tumapel juga dikenal dengan nama Kerajaan Singhasari.

Penemuan Prasasti Mula Malurung di sisi lain memberikan pandangan yang berbeda dengan versi Pararaton, yang selama ini dikenal mengenai sejarah Tumapel. Prasasti yang dikeluarkan Kertanagara tahun 1255 atas perintah Wisnuwardhana itu menyebutkan jika Tumapel didirikan oleh Rajasa yang dijuluki "Batara Syiwa", setelah menaklukkan Kerajaan Kadiri. Nama ini kemungkinan adalah gelar anumerta dari Ranggah Rajasa, karena dalam Nagarakretagama arwah pendiri Tumapel itu dipuja sebagai Syiwa. Selain itu, Pararaton juga menyebutkan bahwa Ken Angrok lebih dulu menggunakan julukan Batara Syiwa sebelum maju dalam perang melawan Kadiri.

Prasasti itu juga menyatakan jika kerajaan kemudian terpecah menjadi dua sepeninggal Ken Angrok, yaitu Tumapel yang dipimpin oleh Anusapati dan Kadiri yang dipimpin oleh Mahesa Wong Ateleng alias Batara Parameswara. Parameswara digantikan oleh Guningbhaya, kemudian Tohjaya. Sementara itu, Anusapati digantikan oleh Seminingrat yang bergelar Wisnuwardhana. Prasasti itu juga menyebutkan bahwa Tumapel dan Kadiri dipersatukan kembali oleh Seminingrat. Kadiri kemudian menjadi kerajaan bawahan yang dipimpin oleh putranya, yaitu Kertanagara.

Lebih lanjut, prasasti ini menyatakan Tohjaya sebagai raja Kadiri, bukan raja Tumapel. Hal ini memperkuat kebenaran berita dalam Nagarakretagama yang tidak menyebut Tohjaya sebagai raja di Tumapel. Selain itu, pemberitaan dalam Nagarakretagama yang menyebut Kertanagara naik takhta tahun 1254 juga dapat diperdebatkan. Kemungkinannya adalah Kertanagara menjadi raja muda di Kadiri terlebih dahulu, kemudian barulah pada 1268 dia bertakhta di Singasari.

Silsilah Wangsa Rajasa

Silsilah Wangsa Rajasa dari sumber prasasti dan naskah kepujanggaan.

Silsilah Wangsa Rajasa, keluarga penguasa Singhasari dan Majapahit. Penguasa ditandai dengan blok warna dalam gambar ini.[3]
Ada dua versi dalam mengidentifikasi sejarah Tumapel atau Singhasari, yaitu Pararaton dan Kakawin Nagarakretagama. Perbedaan ini meliputi daftar Wangsa Rajasa yang berkuasa dan angka tahunnya. Wangsa Rajasa sendiri adalah keluarga yang berkuasa di Kerajaan Singhasari dan Majapahit pada kurun abad ke-13 sampai ke-15. Wangsa ini didirikan oleh Ken Angrok pada awal abad ke-13 berdasarkan gelar yang didapatkannya, yaitu "Rajasa". Keluarga kerajaan ini menjadi penguasa Singhasari dan berlanjut hingga Kerajaan Majapahit.

Versi Pararaton
Dikisahkan dalam Pararaton, Anusapati yang merupakan putra Tunggul Ametung dan Ken Dedes ingin membalas dendam terhadap Ken Arok yang telah membunuh ayahnya. Pada 1247, Ken Arok mati di tangan Anusapati yang kemudian berkuasa di Tumapel. Namun, pada 1249 Anusapati tewas dihabisi oleh Tohjaya yang tidak lain adalah anak Ken Arok dari Ken Umang.

Tohjaya naik singgasana sebagai raja Tumapel setelah Anusapati tiada, tetapi takhtanya hanya berlangsung singkat. Pada 1250, pemerintahannya digulingkan oleh pasukan khusus yang dihimpun oleh Ranggawuni atau yang nantinya dikenal sebagai Wisnuwardhana. Wisnuwardhana adalah anak dari Anusapati yang melanjutkan lingkaran dendam dalam takhta Kerajaan Singasari. Wisnuwardhana lantas dinobatkan sebagai raja selanjutnya hingga mewariskan kekuasaan kepada putranya yang bernama Kertanagara.

Berikut daftar raja Tumapel menurut versi Pararaton.

Sri Ranggah Rajasa Bhatara Sang Amurwabhumi (1222–1247);
Anusapati (1247–1249);
Tohjaya (1249–1250);
Ranggawuni alias Wisnuwardhana (1250–1272);
Kertanagara (1272–1292).
Versi Kakawin Nagarakretagama
Sementara itu, Nagarakretagama tidak menyebut sosok Tunggul Ametung, Ken Angrok, Ken Dedes, Ken Umang, dan Tohjaya maupun pembunuhan di antara penguasa Tumapel. Hal ini dapat dimaklumi karena kitab tersebut berisi pujian untuk Hayam Wuruk, raja Majapahit. Peristiwa berdarah yang menimpa leluhurnya itu dianggap sebagai aib. Namun demikian, dapat diketahui hanya Wisnuwardhana dan Kertanagara saja yang didapati menerbitkan prasasti sebagai bukti kesejarahan mereka.

Menurut Nagarakretagama, penguasa Tumapel yang mengalahkan Kadiri adalah Sri Ranggah Rajasa Sang Girinathaputra. Rangga Rajasa memiliki putra bernama Anusapati, yang kemudian bertakhta di Tumapel dengan gelar Batara Anusapati. Anusapati digantikan oleh putranya yang bernama Wisnuwardhana pada 1248 dan memerintah hingga 1254. Selanjutnya, raja terakhir Tumapel adalah Kertanagara, putra Wisnuwardhana, yang memimpin hingga meninggal pada 1292. Kematiannya sekaligus mengakhiri riwayat kerajaan ini lantaran terjadinya pemberontakan dari dalam.

Berikut daftar raja Tumapel menurut versi Nagarakretagama.

Sri Ranggah Rajasa Sang Girinathaputra (1222–1227);
Anusapati (1227–1248);
Wisnuwardhana (1248–1254);
Kertanagara (1254–1292).
Diagram silsilah di samping ini adalah urutan penguasa dari Wangsa Rajasa yang bersumber dari Pararaton maupun prasasti dan naskah kepujanggaan.

Pemerintahan bersama
Pararaton dan Nagarakretagama menyebutkan adanya pemerintahan bersama antara Wisnuwardhana dan Narasingamurti. Dalam Pararaton disebutkan nama asli Narasingamurti adalah Mahisa Campaka.

Apabila kisah kudeta berdarah dalam Pararaton benar-benar terjadi, maka dapat dipahami maksud dari pemerintahan bersama ini adalah suatu upaya penggabungan atau rekonsiliasi antara Tumapel dan Kadiri yang awalnya terpecah. Wisnuwardhana penguasa Tumapel yang merupakan cucu Tunggul Ametung - Ken Dedes, sedangkan Narasingamurti penguasa Kadiri adalah cucu Ken Arok - Ken Dedes.

Masa Kejayaan
Kertanagara adalah raja terakhir dan raja terbesar dalam sejarah Tumapel(1272 - 1292). Ia adalah raja pertama yang mengalihkan wawasannya ke luar Jawa.

Pada tahun 1275 ia mengirim pasukan Ekspedisi Pamalayu untuk menjadikan Sumatra sebagai benteng pertahanan dalam menghadapi ekspansi bangsa Mongol. Saat itu penguasa Sumatra adalah Kerajaan Melayu. Kerajaan ini akhirnya dianggap telah ditundukkan, dengan dikirimkannya bukti arca Amoghapasa yang dari Kertanagara, sebagai tanda persahabatan kedua negara.[4]

Pada tahun 1284, Kertanagara juga mengadakan ekspedisi menaklukkan Bali.

Pada tahun 1289 Kaisar Kubilai Khan mengirim utusan ke Tumapel meminta agar Jawa mengakui kedaulatan Mongol. Namun permintaan itu ditolak tegas oleh Kertanagara. Nagarakretagama menyebutkan daerah-daerah bawahan Tumapel di luar Jawa pada masa Kertanagara antara lain, Kerajaan Melayu, Bali, Pahang, Gurun, dan Bakulapura.[5]

... 2. Samankana nikaɳ digantara padanabhaya mark i jöɳ nareçwara, ikaɳ sa- (110b) kahawat/ pahaɳ sakahawat malayu pada manunkul adara, muwah sakahawat gurun sakahawat/ bakulapura manaçrayomark, ndatan linen i sunda len/ madura pan satanah i yawa bhakti tan salah. ...


... 2. Begitulah dari empat penjuru orang lari berlindung dibawah Baginda. Seluruh Pahang, segenap Melayu tunduk menekur dihadapan beliau. Seluruh Gurun, segenap Bakulapura lari mencari perlindungan. Sunda Madura tak perlu dikatakan, sebab sudah terang setanah Jawa. ...
— (Kakawin Nagarakretagama, Pupuh 42).

Wilayah kerajaan Tumapel juga meliputi Mojokerto jauh sebelum Majapahit berdiri. Kekuasaan Tumapel di Mojokerto salah satunya dibuktikan dengan Prasasti Gondang. Prasasti Gondang adalah sebuah prasasti in-situ (masih ditempat asli) peninggalan Kerajaan Tumapel yang baru ditemukan pada tahun 2017 silam di tengah persawahan di Dusun Rejoso, Desa Gondang, Kecamatan Gondang, Kabupaten Mojokerto, Jawa Timur. Prasasti tersebut ditemui oleh warga setempat dan terdapat bacaan dalam bahasa Jawa Kuno yang bertuliskan tahun 1197 saka atau 1275 masehi.[6] Berdasarkan angka tahunnya, prasasti ini dibuat pada masa kekuasaan Raja Kertanegara. Prasasti ini menandakan wilayah yang masuk dalam kekuasaan Singasari yaitu Gresik, Surabaya, Sidoarjo, sampai Mojokerto sebelum berdirinya Kerajaan Majapahit.

Keruntuhan

Candi Singhasari dibangun sebagai tempat pemuliaan Kertanegara, raja terakhir Singhasari.
Kerajaan Singhasari yang sibuk mengirimkan angkatan perangnya ke luar Jawa, akhirnya membuat pertahanan di dalam kerajaan menjadi lemah.

Pada tahun 1292 terjadi pemberontakan Jayakatwang bupati Gelanggelang, yang merupakan sepupu, ipar, dan sekaligus besan dari Kertanagara sendiri, karena ingin membalas dendam terhadap Wangsa Rajasa yang telah merebut kekuasaan, serta membunuh keluarga dan leluhurnya. Pemberontakan ini menyebabkan kematian Kertanegara dan runtuhnya kerajaan Tumapel.

Setelah runtuhnya Tumapel, Jayakatwang mengangkat dirinya menjadi raja dan membangun Kerajaan Kediri dengan ibukota di Daha. Riwayat Kerajaan Tumapel pun berakhir.

Hubungan dengan Majapahit
Pararaton, Nagarakretagama, dan prasasti Kudadu mengisahkan Raden Wijaya cucu Narasingamurti yang menjadi menantu Kertanagara lolos dari maut. Berkat bantuan Aria Wiraraja (penentang politik Kertanagara), ia kemudian diampuni oleh Jayakatwang dan diberi hak mendirikan desa Majapahit.

Pada tahun 1293 datang pasukan Mongol yang dipimpin Ike Mese untuk menaklukkan Jawa. Mereka dimanfaatkan Raden Wijaya untuk mengalahkan Jayakatwang di Kerajaan Kadiri. Setelah Kadiri runtuh, Raden Wijaya dengan siasat cerdik ganti mengusir tentara Mongol keluar dari tanah Jawa.

Raden Wijaya kemudian mendirikan Kerajaan Majapahit sebagai kelanjutan Tumapel, dan menyatakan dirinya sebagai anggota Wangsa Rajasa, yaitu dinasti yang didirikan oleh Ken Arok.

Warisan Budaya
Arca
Arca Amoghapasa
Arca Anusapati
Arca Dwarapala
Arca Joko Dolog
Arca Ken Dedes
Arca Wisnu Wardhana
Candi
Candi Jago
Candi Jawi
Candi Kangenan
Candi Katang Lumbang
Candi Kidal
Candi Singasari
Candi Sumberawan
Prasasti
Prasasti Maribong
Prasasti Mula Malurung
Prasasti Padang Roco
Prasasti Wurare
Kutipan
 Komandoko, Gamal (2010). Ensiklopedia Pelajar dan Umum. Pustaka Widyatama. ISBN 9789796103713.
 Anshoriy,Ch, HM Nasruddin (2008-01-01). Neo Patriotisme ; Etika Kekuasaan dalam Kebudayaan Jawa. Lkis Pelangi Aksara. ISBN 9789791283670.
 Bullough, Nigel (1995). Historic East Java: Remains in Stone. Jakarta: ADLine Communications. hlm. 116–117.
 Reichle, Natasha (2007). Violence and Serenity: Late Buddhist Sculpture from Indonesia (dalam bahasa Inggris). University of Hawaii Press. hlm. 120. doi:10.1515/9780824865474. ISBN 978-0-8248-6547-4. § The Sumatran Image of Amoghapāśa. [...]. It is known from the Nāgarakṛtāgama that eleven years earlier Kṛtanagara had sent a military force to Malāyu. Kṛtanagara was victorious, and, according to the text, “[t]he whole territories of Pahang and Malāyu bowed humbly before him.”
 http://www.spaetmittelalter.uni-hamburg.de/java-history/JavaNK/Java1365.Nagara-Kertagama.Canto.38.3-49.html
 Budianto, Enggran Eko (05 Jun 2020). "Prasasti Gondang, Bukti Kekuasaan Kerajaan Singasari di Mojokerto". detikcom.
Referensi
Poesponegoro & Notosusanto (ed.). 1990. Sejarah Nasional Indonesia Jilid II. Jakarta: Balai Pustaka
Purwadi. 2007. Sejarah Raja-Raja Jawa. Yogyakarta: Media Ilmu
R.M. Mangkudimedja. 1979. Serat Pararaton Jilid 2. Jakarta: Departemen Pendidikan dan Kebudayaan, Proyek Penerbitan Buku Sastra Indonesia dan Daerah
Slamet Muljana. 2005. Menuju Puncak Kemegahan (terbitan ulang 1965). Yogyakarta: LKIS
Slamet Muljana. 1979. Nagarakretagama dan Tafsir Sejarahnya. Jakarta: Bhratara
Vlekke, Bernard H.M. Nusantara. Jakarta:KPG (Kepustakaan Populer Gramedia)
Pranala luar
Kakawin Nagarakretagama di Wikisource
Didahului oleh:
Kadiri	Kerajaan Hindu-Budha
1222–1292	Diteruskan oleh:
Majapahit
lbs
Kerajaan di Jawa
Sebelum 600 M
(Hindu-Buddha pra-Mataram)	
TarumanagaraKalinggaSunda
600-1500 (Hindu-Buddha)	
Mataram Kuno SyailendraIsyanaKahuripanJanggalaKadiriSingasariMajapahitPajajaran
1500-sekarang (Islam)	
Cirebon KasepuhanKanomanKacirebonanKaprabonanDemakKalinyamatBantenPajangMataram Islam SurakartaYogyakartaMangkunagaranPakualaman
Kategori: Kerajaan di NusantaraKerajaan di JawaKerajaan di Jawa TimurKabupaten Malang
Halaman ini terakhir diubah pada 15 Desember 2022, pukul 14.22.
Teks tersedia di bawah Lisensi Creative Commons Atribusi-BerbagiSerupa; ketentuan tambahan mungkin berlaku. Lihat Ketentuan Penggunaan untuk lebih jelasnya.
Kebijakan privasiTentang WikipediaPenyangkalanTampilan selulerPengembangStatistikPernyataan kukiWikimedia FoundationPowered by MediaWiki

Lompat ke isi
Buka/tutup bilah samping

WikipediaEnsiklopedia Bebas
Telusuri Wikipedia
Buat akun baru

Perkakas pribadi

Ikon Tantangan EUforia Wiki4Peace.svgLogo WMID Menurun SVG.svg  Ayo ikut EUforia Wiki4Peace dan dapatkan hadiahnya! Tantangan berlangsung 1 – 31 Desember 2022. 
Tutup
Daftar isi move to sidebar sembunyikan
Awal
Asal usul
Toggle Asal usul subsection
Pembagian wilayah kerajaan Airlangga
Nama Panjalu
Nama Kadiri
Latar belakang
Toggle Latar belakang subsection
Runtuhnya Kerajaan Medang
Perkembangan kerajaan
Toggle Perkembangan kerajaan subsection
Catatan Kronik Tiongkok
Hubungan dengan kekuatan regional
Keruntuhan
Daftar raja-raja
Warisan Budaya
Toggle Warisan Budaya subsection
Candi
Karya Sastra
Prasasti
Situs
Lihat pula
Daftar pustaka
Kerajaan Kadiri

Halaman
Pembicaraan
Baca
Sunting
Sunting sumber
Lihat riwayat
Dari Wikipedia bahasa Indonesia, ensiklopedia bebas

Artikel ini sudah memiliki daftar referensi, bacaan terkait, atau pranala luar, tetapi sumbernya belum jelas karena belum menyertakan kutipan pada kalimat. Mohon tingkatkan kualitas artikel ini dengan memasukkan rujukan yang lebih mendetail bila perlu. (Pelajari cara dan kapan saatnya untuk menghapus pesan templat ini)
Karājāan Pañjalu
Kaḍhiri
1019–1222
Kerajaan Janggala dan Panjalu, kemudian bersatu menjadi Kerajaan Kadiri
Kerajaan Janggala dan Panjalu, kemudian bersatu menjadi Kerajaan Kadiri
Ibu kota	
Watan Mas
(1019-1031)
Kahuripan
(1032-1042)
Dahanapura
(1042-1222)
Bahasa yang umum digunakan	Jawa Kuno
Agama	Kapitayan Hinduisme, Buddhisme, Animisme
Pemerintahan	Monarki
Sri/Maharaja	 
• 1019-1042
Airlangga
• 1042-1051
Sri Samarawijaya
• 1051-1112
Sri Jitendrakara
• 1112-1135
Sri Bameswara
• 1135-1159
Jayabaya
• 1159-1171
Sri Sarweswara
• 1171-1181
Sri Aryeswara
• 1181-1182
Sri Gandra
• 1182-1194
Kamesywara
• 1194-1222
Kertajaya
Sejarah	 
• Didirikan
1019
• Dibagi oleh Airlangga
1042
• Janggala ditaklukkan Jayabaya
1135
• Kakawin Bhāratayuddha selesai ditulis
1157
• Runtuh oleh Pemberontakan Ken Arok
1222
Mata uang	Koin emas dan perak
Didahului oleh	Digantikan oleh
	Kerajaan Medang
Kerajaan Tumapel	
Sekarang bagian dari	 Indonesia
 Timor Leste
Bagian dari seri mengenai
Sejarah Indonesia
Nicolaes Visscher Indiae Orientalis.jpg
Prasejarah
Kerajaan Hindu-Buddha
Kerajaan Islam
Kerajaan Kristen
Kolonialisme Eropa
Kemunculan Indonesia
Kemerdekaan
Menurut topik
Garis waktu
Flag of Indonesia.svg Portal Indonesia
lbs
Artikel ini membahas tentang Kerajaan Kediri (Sejarah Nusantara). Lihat pula Kota Kediri dan Kabupaten Kediri. Untuk kegunaan lain, lihat Kediri (disambiguasi).
Kerajaan Kadiri atau Kediri (Aksara Jawa :ꦥꦁꦗꦭꦸ ꦏꦣꦶꦫꦶ ) disebut juga sebagai Panjalu, adalah sebuah kerajaan Hindu-Buddha yang terdapat di Jawa Timur, Indonesia, antara tahun 1019-1222 M. Dan merupakan salah satu kerajaan hasil pembelahan yang juga didirikan Airlangga. Kerajaan ini dipimpin oleh wangsa Isyana dan berpusat di Dahanapura, adalah nama kota kuno di masa lalu yang sekarang menjadi bagian dari Kota Kediri.

Asal usul

Arca Dwarapāla (penjaga gapura) Totok Kerot, Kediri.
Sesungguhnya kota Daha sudah ada sebelum peristiwa pembelahan kerajaan oleh Airlangga. Daha merupakan singkatan dari Dahanapura, yang berarti kota api. Nama ini terdapat dalam prasasti Pamwatan yang dikeluarkan Airlangga tahun 1042. Hal ini sesuai dengan berita dalam Serat Calon Arang, bahwa saat akhir pemerintahan Airlangga, pusat kerajaan sudah tidak lagi berada di Kahuripan, melainkan telah berpindah ke Dahanapura dan menyebut Airlangga sebagai raja Daha atau (kerajaan Daha).

... 15. Sigra datang pwa sirêng sagara Rupěk, mantas ta sira ngkana, Sang Yogîswara Mpu Baradah. Tan lingěn pwa sirêng (h)ěnu lampah Sang Mahamuni ambramaga. Sigra datang ta sirêng nagarêng Daha, panggih ta sirâtmajanira Sang Maharaja Erlanggya sědang tinangkil...


... 15. Segera tiba di Sagara Rupek, beliau menyeberang di sana, Sang Pendeta Baradah. Tidak diceritakan perjalanan Sang Pendeta di jalan sangat cepat jalannya. Beliau segera tiba di Kerajaan Daha, bertemu dengan putranya Sang Maharaja Erlangga yang sedang dihadap...
— (Lontar Calon Arang).

Pembagian wilayah kerajaan Airlangga
Pada akhir November 1042, Airlangga terpaksa membelah wilayah kerajaannya karena kedua putranya saling bersaing memperebutkan takhta. Putra yang bernama Sri Samarawijaya mendapatkan kerajaan barat bernama Panjalu yang berpusat di kota baru, yaitu Daha. Sedangkan putra yang bernama Mapanji Garasakan mendapatkan kerajaan timur bernama Janggala yang berpusat di kota lama, yaitu Kahuripan.

Menurut Nagarakretagama, sebelum dibelah menjadi dua, nama kerajaan yang dipimpin Airlangga sudah bernama Panjalu, yang berpusat di Daha. Jadi, kerajaan Janggala lahir sebagai pecahan dari Panjalu. Adapun Kahuripan adalah nama kota lama yang sudah ditinggalkan oleh Airlangga dan kemudian menjadi ibu kota Janggala.[1]

... 1. Nahan tatwanikaɳ kamal/ widita deniɳ sampradaya sthiti, mwaɳ çri pañjalunatha riɳ daha te- (122a) wekniɳ yawabhumy/ apalih, çri airlanghya sirandani ryyasihiran/ panak/ ri saɳ rwa prabhu, ...


... 1. Demikian sejarah Kamal menurut tutur yang dipercaya, Dan Sri Nata Panjalu di Daha, waktu bumi Jawa dibelah, Karena cinta raja Airlangga kepada dua puteranya, ...
— (Kakawin Nagarakretagama, Pupuh 68).

Setelah turun takhta, Airlangga menjalani hidup sebagai pertapa sampai meninggal sekitar tahun 1049. Menurut Serat Calon Arang ia kemudian bergelar Resi Erlangga Jatiningrat, sedangkan menurut Babad Tanah Jawi ia bergelar Resi Gentayu. Namun yang paling dapat dipercaya adalah prasasti Gandhakuti (1042) yang menyebut gelar kependetaan Airlangga adalah Resi Aji Paduka Mpungku Sang Pinaka Catraning Bhuwana.

Nama Panjalu
Pada mulanya, nama Pañjalu pembacaan yang tepat sesuai aksara Pangjalu memang lebih sering dipakai daripada nama Kadiri. Hal ini dapat dijumpai dalam prasasti-prasasti yang diterbitkan oleh raja-raja Panjalu. Bahkan, nama Panjalu juga dikenal sebagai Pu-chia-lung dalam kronik Tiongkok berjudul Ling wai tai ta (1178).


Arca Wisnu, berasal dari Kediri, abad ke-12 dan ke-13.
Terdapat tiga jenis tanah yang akan digunakan oleh masyarakat di masa lalu saat akan membangun sebuah tempat atau pemukiman, pertama adalah tanah Anupa sebagai tanah subur serta dekat dengan sumber mata air berbagai macam biji-bijian jika ditanam akan tumbuh dengan baik. Kedua adalah tanah Sadarana, tanah yang di sebagian wilayahnya subur dan sebagian yang lainnya kurang subur, selanjutnya yang ketiga adalah Janggala yang merupakan tanah yang kurang subur atau hutan belantara.

Pangjalu berasal dari kata Jalu yang memiliki arti Jantan atau Pria, selanjutnya diberi unsur kata Pang yang adalah Pe, merupakan tambahan sehingga menjadi kalimat Pe-jantan dalam konteks kewilayahan istilah pejantan tersebut bermakna wilayah yang subur serta berdikari atau mandiri.

Istilah Kadiri merupakan sinonim kata atau persamaan dari Pangjalu yang bermakna kemandirian. Kasus tersebut mirip dengan nama Majapahit dengan Wilwatikta, dimana wilwa adalah buah maja dan tikta adalah pahit.

Nama Kadiri
Nama "Kadiri" atau "Kediri" juga berasal dari kata bahasa Sansekerta, khadri, yang berarti pacé atau mengkudu (Morinda citrifolia). Batang kulit kayu pohon ini menghasilkan zat perwarna ungu kecokelatan yang digunakan dalam pembuatan batik, sementara buahnya dipercaya memiliki khasiat pengobatan.

Asal-usul kata yang dipandang lebih tepat adalah berasal dari kata "kadiri" dalam Bahasa Jawa Kuno yang berarti bisa berdiri sendiri, mandiri, berdiri tegak, berkepribadian, atau berswasembada.

Latar belakang
Runtuhnya Kerajaan Medang
Raja Kerajaan Medang yang terakhir bernama Dharmawangsa Teguh, saingan berat Kedatuan Sriwijaya. Pada tahun 1016 M, Raja Wurawari dari Lwaram sekitar Cepu, Blora (sekutu Sriwijaya) menyerang Istana Wwatan, ibu kota Kerajaan Medang, yang tengah mengadakan pesta pernikahan Airlangga dengan putri dari raja Dharmawangsa Teguh, Dharmawangsa Teguh sendiri tewas dalam serangan tersebut sedangkan keponakannya yang bernama Airlangga beserta dengan putri Dharmawangsa berhasil lolos.

Airlangga adalah putra pasangan Mahendradatta (saudari Dharmawangsa Teguh) dengan Udayana raja dari Kerajaan Bedahulu, Bali. ia lolos bersama putri Dharmawangsa ditemani pembantunya yang bernama Mpu Narotama. Sejak saat itu Airlangga menjalani kehidupan sebagai pertapa di hutan pegunungan (Vana giri) sekarang Wonogiri, kemudian menuju Sendang Made, Kudu, Jombang.

Pada saat pelarian dan dalam masa pertapaannya, setelah tiga tahun hidup didalam hutan pada tahun 1019, Airlangga didatangi utusan rakyat beserta para senopati yang masih setia, meminta agar dirinya mendirikan dan membangkitkan kembali sisa-sisa kejayaan kerajaan Medang. Atas dukungan dari para pendeta dan kaum brahmana ia kemudian membangun sebuah Ibukota baru yang bernama Watan Mas.

“	15. Kemudian dalam tahun penting yaitu 932 tahun saka , tanggal 13 paro terang , bulan magha, pada hari kamis menghadaplah para abdi dan para brahmana terpandang kepada raja di raja Erlangga , menunduk hormat di sertai harapan tulus.mereka dengan penuh ketulusan mengajukan permohonan kepadanya: "perintahlah negara ini sampai batas-batas yang paling jauh ! ...
— (Prasasti Pucangan).	”
Ibu kota baru bernama Watan Mas terletak di dekat sekitar Gunung Penanggungan. Pada mulanya wilayah kerajaan yang diperintah Airlangga hanya meliputi daerah Gunung Penanggungan dan sekitarnya, karena banyak daerah-daerah bawahan Kerajaan Medang yang membebaskan diri setelah keruntuhannya. Baru setelah Kedatuan Sriwijaya dikalahkan Rajendra Coladewa, raja Colamandala dari Kerajaan Chola, India di tahun 1023, Airlangga bisa dengan leluasa membangun kembali kejayaan wangsa Isyana.

Sejak tahun 1025, peperangan demi peperangan dijalani Airlangga. Satu demi satu kerajaan-kerajaan di Jawa Timur dapat ditaklukkannya. Namun pada tahun 1031 Airlangga kehilangan kota Watan Mas karena diserang oleh raja wanita yang kuat bagai raksasa. Raja wanita itu adalah Ratu Dyah Tulodong, yang merupakan salah satu raja Kerajaan Lodoyong (sekarang wilayah Tulungagung, Jawa Timur). Dyah Tulodong digambarkan sebagai ratu yang memiliki kekuatan luar biasa. Salah satu peristiwa sejarah penting adalah pertempuran antara bala tentara Raja Airlangga yang berhasil dikalahkan oleh Dyah Tulodong. Pertempuran tersebut terjadi lantaran Dyah Tulodong berusaha membendung ekspansi Airlangga yang waktu itu sudah menguasai wilayah-wilayah di sekitar kerajaan Lodoyong. Bahkan di beberapa riwayat, diceritakan pasukan khusus yang dibawa Ratu Dyah Tulodong merupakan prajurit-prajurit wanita pilihan, pasukan ini bahkan berhasil memukul mundur pasukan Airlangga dari pusat kota kerajaannya "Watan Mas" di dekat Gunung Penanggungan hingga ke Patakan (Sambeng, Lamongan, Jawa Timur).

Tetapi satu tahun kemudian di penghujung tahun 1032, dari arah utara, pasukan Airlangga bergerak ke selatan menuju Lodoyong. Dyah Tulodong berhasil dikalahkan oleh Airlangga lewat pertempuran sengit. Tidak lama kemudian Raja Wurawari pun dapat dihancurkannya, sekaligus membalaskan dendam keluarga Airlangga dan wangsa Isyana. Sejak saat itu wilayah kerajaan Airlangga mencakup hampir seluruh Jawa Timur.

Tahun 1032, Airlangga kemudian membangun ibu kota baru bernama Kahuripan yang berpusat di daerah Kabupaten Sidoarjo sekarang.

Pada tahun 1042, berdasarkan prasasti Pamwatan dan Serat Calon Arang, di akhir masa pemerintahannya, Airlangga kemudian memindahkan ibukotanya ke Daha.

Perkembangan kerajaan

Arca Buddha Vajrasattva zaman Kadiri, abad X/XI, koleksi Museum für Indische Kunst, Berlin-Dahlem, Jerman.

Candi Penataran merupakan candi yang berumur empat abad karena dibangun dan dikembangkan oleh beberapa kerajaan sekaligus, mulai dari Kerajaan Kediri hingga Majapahit.

Wayang Kulit wayang panji brajanata―pangeran Kerajaan Kadiri.
Masa-masa awal Kerajaan Kadiri tidak banyak diketahui. Prasasti Turun Hyang (1044) yang diterbitkan Kerajaan Janggala hanya memberitakan adanya perang saudara antara kedua kerajaan sepeninggal Airlangga, sejarah Kerajaan Kadiri mulai diketahui dengan adanya Prasasti Banjaran dan Prasasti Mataji. Setelah Raja Sri Jitendrakara diketahui terdapat raja bernama Sri Bameswara berdasarkan Prasasti Karanggayam. Selanjutnya dalam Prasasti Hantang raja yang memerintah sudah berganti Sri Jayabhaya. Panjalu di bawah pemerintahan Sri Jayabhaya berhasil menaklukkan Kerajaan Janggala dengan semboyannya yang terkenal dalam prasasti Ngantang (1135), yaitu Panjalu Jayati, yang berarti "Kadiri Menang".

Pada masa pemerintahan Sri Jayabhaya inilah, Kerajaan Kadiri mengalami masa kejayaannya. Wilayah kerajaan ini meliputi seluruh Jawa dan beberapa pulau di Nusantara, bahkan sampai mengalahkan pengaruh Kedatuan Sriwijaya di Sumatra. Hal ini diperkuat kronik Tiongkok berjudul Ling wai tai ta karya Chou Ku-fei tahun 1178, bahwa pada masa itu negeri paling kaya selain Tiongkok secara berurutan adalah Arab, Jawa, dan Sumatra. Saat itu yang berkuasa di Arab adalah Bani Abbasiyah, di Jawa ada Panjalu, sedangkan Sumatra dikuasai Sriwijaya.[2] Pada Prasasti Talan tahun 1136, raja Jayabhaya menganugerahkan desa Talan sebagai sima karena telah menyimpan prasasti ripta (lontar) dari masa leluhurnya wangsa Isyana yaitu Airlangga lontar tersebut disalin ke prasasti batu dan diberi tambahan anugerah lain oleh Raja Jayabhaya. Prabu Jayabhaya sendiri mengklaim bahwa Raja Airlangga adalah nenek moyangnya.

Penemuan Situs Tondowongso awal tahun 2007 diyakini sebagai peninggalan Kerajaan Panjalu juga bersama dengan Situs Adan-Adan yang memiliki bermacam temuan benda-benda bersejarah seperti batuan fondasi candi, makara, sistem pertirtaan (pengairan) diduga embung, pecahan keramik dan beberapa arca peninggalan era Kerajaan Panjalu dan Tumapel yang terletak di Desa Adan-adan, Kecamatan Gurah, Kabupaten Kediri, Jawa Timur, diharapkan dapat membantu memberikan lebih banyak informasi tentang kerajaan tersebut.

Catatan Kronik Tiongkok

Wayang Kulit boneka Dewi Ragil Kuning―putri Kerajaan Kadiri.
Menurut sumber berita Tiongkok, pekerjaan utama orang Panjalu berkisar pada pertanian (bercocok tanam padi), peternakan (sapi, babi hutan, unggas), dan perdagangan rempah-rempah. Daha, ibu kota Kerajaan Panjalu, terletak di pedalaman, dekat lembah sungai Brantas yang subur. Dari masa pemerintahan raja sebelumnya Airlangga, Panjalu mewarisi sistem irigasi, termasuk bendungan Waringin Sapta. Perekonomian Panjalu sebagian dimonetisasi, dengan koin perak yang dikeluarkan oleh istana.

Pada periode-periode selanjutnya, perekonomian Kadiri tumbuh dengan lebih bertumpu pada perdagangan, khususnya perdagangan rempah-rempah. Hal ini dihasilkan dari pengembangan angkatan laut Panjalu, memberi mereka kesempatan untuk mengontrol jalur perdagangan rempah-rempah ke pulau-pulau timur. Panjalu mengumpulkan rempah-rempah dari anak sungai di Kalimantan bagian selatan dan Kepulauan Maluku. Orang India dan Asia Tenggara kemudian mengangkut rempah-rempah ke pasar Mediterania dan Tiongkok melalui Rute Rempah-rempah yang menghubungkan rantai pelabuhan dari Samudra Hindia ke Cina selatan.

Pertanian, peternakan, dan perdagangan berkembang pesat dan mendapat perhatian penuh dari pemerintah. Dia melaporkan bahwa peternakan ulat sutera untuk memproduksi pakaian sutra dan katun telah diadopsi oleh orang Jawa pada waktu itu. Tidak ada hukuman fisik (penjara atau penyiksaan) bagi para penjahat. Sebaliknya, orang yang melakukan perbuatan melawan hukum terpaksa membayar denda berupa emas, kecuali pencuri dan perampok yang dieksekusi mati.

Dalam adat perkawinan, keluarga mempelai wanita menerima mas kawin berupa emas dari mempelai pria. Alih-alih mengembangkan pengobatan medis, masyarakat Panjalu mengandalkan doa kepada Buddha. Pada bulan ke-5 tahun ini, festival air dirayakan dengan orang-orang yang bepergian dengan perahu di sepanjang sungai untuk merayakannya. Pada bulan ke-10, festival lain diadakan di pegunungan. Orang-orang akan berkumpul di sana untuk bersenang-senang dan memainkan musik dengan instrumen seperti seruling, gendang, dan gambang kayu (bentuk gamelan kuno).

Hubungan dengan kekuatan regional

Sriwijaya dan Panjalu sekitar abad ke 12 hingga awal abad ke-13
Kerajaan Panjalu yang berkuasa di Jawa bersama dengan Kedatuan Sriwijaya yang berbasis di Sumatera sepanjang abad ke 12 hingga ke-13, tampaknya telah mempertahankan hubungan perdagangan dengan Tiongkok dan sampai batas tertentu dengan India. Catatan Cina mengidentifikasi kerajaan ini sebagai Tsao-wa atau Chao-wa (Jawa), sejumlah catatan Tiongkok menandakan bahwa penjelajah dan pedagang Cina sering mengunjungi kerajaan ini. Hubungan dengan India adalah hubungan budaya, karena sejumlah Rakawi (penyair atau sarjana) Jawa menulis literatur yang diilhami oleh mitologi, kepercayaan, dan epos Hindu seperti Mahabharata dan Ramayana.

Pada abad ke-11, hegemoni Sriwijaya di kepulauan Indonesia mulai menurun, ditandai dengan invasi Rajendra Chola dari Kerajaan Chola ke Semenanjung Malaya dan Sumatera. Melemahnya hegemoni Sriwijaya telah memungkinkan terbentuknya kerajaan-kerajaan regional, seperti Panjalu, yang berbasis pertanian daripada perdagangan. Belakangan Kerajaan Kadiri berhasil menguasai jalur perdagangan rempah-rempah ke Maluku.

Menurut berita Cina, dan kitab Ling-wai-tai-ta diterangkan bahwa dalam kehidupan sehari-hari orang-orang memakai kain sampai di bawah lutut. Rambutnya diurai. Rumah-rumah mereka bersih dan teratur, lantainya ubin yang berwarna kuning dan hijau. Raja mengenakan pakaian sutra, sepatu kulit dan perhiasan emas berukir. Rambutnya disanggul tinggi-tinggi di atas kepala. Setiap hari, dia akan menerima pejabat negara, dan menjalankan kerajaannya, di atas takhta persegi. Setelah pertemuan, pejabat negara akan membungkuk tiga kali kepada raja. Jika raja bepergian ke luar istana, raja naik gajah atau kereta yang diiringi oleh 500 sampai 700 tentara dan pejabat, sementara rakyatnya, orang-orang Panjalu, bersujud saat raja lewat. masih menurut Zhou Ku-fei bahwa Kerajaan Panjalu kekuasaannya sangat luas dan kaya raya, menurutnya di dunia saat itu ada 3 kerajaan kaya; Kekhalifahan Abbasiyah yang berkuasa di Arab, Kerajaan Panjalu yang menguasai Bagian Timur Nusantara dan Sriwijaya yang menguasai bagian barat Nusantara.

Chau Ju-Kua, seorang pegawai resmi Dinasti Song menuliskan dalam bukunya Zhu-fan-zhi, menggambarkan bahwa di kepulauan Asia Tenggara ada dua kerajaan yang kuat dan kaya: Sriwijaya dan Jawa (Panjalu). Di Jawa ia menemukan bahwa orang-orang menganut dua agama: Buddha dan agama Brahmana (Hindu). Orang Jawa adalah pemberani dan pemarah, mereka berani untuk melawan. Waktu luangnya dipergunakan untuk mengadu binatang, hiburan favoritnya adalah sabung ayam dan adu babi. Mata uangnya dibuat dari campuran tembaga, perak, dan timah. Buku Chu-fan-chi menyebut bahwa maharaja jawa mempunyai wilayah jajahan: Pai-hua-yuan (Pacitan), Ma-tung (Medang), Ta-pen (Tumapel, Malang), Hi-ning (Dieng), Jung-ya-lu (Hujung Galuh, sekarang Surabaya), Tung-ki (Jenggi, Papua Barat), Ta-kang (Sumba), Huang-ma-chu (Papua), Ma-li (Bali), Kulun (Gurun, mungkin Gorong atau Sorong di Papua Barat atau Nusa Tenggara), Tan-jung-wu-lo (Tanjungpura di Borneo), Ti-wu (Timor), Pingya-i (Banggai di Sulawesi), dan Wu-nu-ku (Maluku)

Mengenai Sriwijaya, Chou-Ju-Kua melaporkan bahwa Kien-pi (Kampe, di Sumatera bagian utara) dengan pemberontakan bersenjatanya telah membebaskan diri dari pengaruh Sriwijaya, dan menobatkan raja mereka sendiri. Nasib yang sama menimpa beberapa koloni Sriwijaya di Semenanjung Malaya yang membebaskan diri dari dominasi Sriwijaya. Namun Sriwijaya masih negara terkuat dan terkaya di bagian barat Nusantara. Koloni Sriwijaya adalah: Pong-fong (Pahang), Tong-ya-nong (Trengganu), Ling-ya-ssi-kia (Langkasuka), Kilan-tan (Kelantan), Fo-lo-an, Ji-lo-t' ing (Jelutong), Ts'ien-mai (?), Pa-t'a (Paka), Tan-ma-ling (Tambralinga, Ligor atau Nakhon Si Thammarat), Kia-lo-hi (Grahi, bahasa Melayu bagian utara semenanjung), Pa-lin-fong (Palembang), Sin-t'o (Sunda), Lan-wu-li (Lamuri di Aceh), dan Si-lan. Menurut sumber ini, pada awal abad ke-13 Sriwijaya masih menguasai Sumatera, Semenanjung Malaya, dan Jawa bagian barat (Sunda).

Mengenai Sunda, buku itu merinci bahwa pelabuhan Sunda (Sunda Kelapa) sangat bagus dan letaknya strategis, dan lada dari Sunda termasuk yang kualitas terbaik. Orang-orang bekerja di pertanian; rumah mereka dibangun di atas tiang kayu (rumah panggung). Namun negara itu penuh dengan perampok dan pencuri.

Keruntuhan

Ganesha dan pecahan candi di Karesidenan Kediri tahun 1866-1867
Kerajaan Panjalu-Kadiri runtuh pada masa pemerintahan Kertajaya, dan dikisahkan dalam Pararaton dan Nagarakretagama.

Artikel utama: Pemberontakan Ken Arok
Pada tahun 1222 Kertajaya sedang berselisih melawan kaum brahmana. Kemudian para Brahmana meminta perlindungan kepada Ken Arok akuwu Tumapel. Kebetulan Ken Arok juga bercita-cita memerdekakan Tumapel yang merupakan daerah bawahan Kadiri.

Puncak peperangan antara Kadiri dan Tumapel terjadi dekat Desa Ganter (Genter), di wilayah timur Kadiri. Pasukan Ken Arok berhasil menghancurkan pasukan Kertajaya. Dengan demikian, berakhirlah masa Kerajaan Kadiri, yang sejak saat itu kemudian menjadi bawahan Tumapel atau Singhasari.

Setelah Ken Arok mengalahkan Kertajaya, Kadiri menjadi suatu wilayah di bawah kekuasaan Singhasari. Ken Arok mengangkat Jayasabha, putra Kertajaya sebagai bupati Kadiri. Tahun 1258 Jayasabha digantikan putranya yang bernama Sastrajaya. Pada tahun 1271 Sastrajaya digantikan putranya, yaitu Jayakatwang.

Artikel utama: Pemberontakan Jayakatwang
Pada tahun 1292, Jayakatwang memberontak terhadap Singhasari yang dipimpin oleh Kertanegara, karena dendam masa lalu dimana leluhurnya Kertajaya dikalahkan oleh Ken Arok. Setelah berhasil membunuh Kertanegara, Jayakatwang membangun kembali Kerajaan Kadiri, namun hanya bertahan satu tahun dikarenakan serangan gabungan yang dilancarkan oleh pasukan Mongol dan pasukan menantu Kertanegara, Raden Wijaya.

Daftar raja-raja
Penguasa Panjalu-Kadiri

Masa pemerintahan	Sri/Maharaja	Prasasti dan berita
1019-1042	Sri Maharaja Rakai Halu Sri Lokeswara Dharmawangsa Airlangga Anantawiramottunggadewa
(Airlangga)	Kakawin Nagarakretagama (1365), Serat Calon Arang.
1042-1051	Sri Samarawijaya Dharmasuparnawahana Teguh Uttunggadewa
(Sri Samarawijaya)	Disebutkan dalam prasasti Pucangan (1041). Adalah raja kerajaan Kadiri setelah peristiwa pembagian kerajaan oleh Airlangga kepada kedua putranya.
1051-1112	Sri Jitendra Kara Wuryyawiryya Parakrama Bhakta
(Sri Jitendrakara)	Disebutkan dalam prasasti Mataji (1051).
1112-1135	Sri Maharaja Rakai Sirikan Sri Bameswara Sakalabhuwana Tustikarana Sarwaniwariwirya Parakrama Digjaya Uttunggadewa
(Sri Bameswara)	prasasti Pandlegan I (1117), prasasti Panumbangan, prasasti Geneng (1128), prasasti Tangkilan (1130), prasasti Besole (1132), prasasti Pagiliran (1134), prasasti Bameswara (1135), prasasti Karanggayam.
1135-1159	Sri Maharaja Sang Mapanji Jayabhaya Sri Warmeswara Madhusudana Awataranindita Suhtrisingha Parakrama Uttunggadewa
(Jayabaya)	Disebutkan dalam prasasti Hantang (1135), Jepun (1144) dan Talan (1136). Janggala ditaklukkan dan bersatu kembali dengan Panjalu.
1159-1171	Sri Maharaja Rakai Sirikan Sri Sarweswara Janardanawatara Wijaya Agrajasama Singhadani Waryawirya Parakrama Digjaya Uttunggadewa
(Sri Sarweswara)	Disebutkan dalam prasasti Kahyunan dan Padlegan II (1159).
1171-1181	Sri Maharaja Rake Hino Sri Aryeswara Madhusudanawatara Arijamuka Sakalabhuwanaritiniwiryya Parakrama Uttunggadewa
(Sri Aryeswara)	Disebutkan dalam prasasti Waleri dan Angin (1171).
1181-1182	Sri Maharaja Koncaryadipa Handabhuwanapadalaka Parakrama Anindita Digjaya Uttunggadewa Sri Gandra
(Sri Gandra)	Disebutkan dalam prasasti Jaring.
1182-1194	Sri Maharaja Rake Sirikan Sri Kameswara Sakalabhuwanatustikarana Sarwaniwaryyawiryya Parakrama Digjaya Uttunggadewa
(Kameswara)	Disebutkan dalam prasasti Semanding (1182) dan prasasti Ceker (1185).
1194-1222	Paduka Sri Maharaja Sri Sarweswara Triwikramawatara Anindita Srenggalancana Digjaya Uttunggadewa
(Kertajaya)	Disebutkan dalam prasasti Sapu Angin (1190), prasasti Galunggung (1194), prasasti Kamulan (1194), prasasti Palah (1197), prasasti Biri, prasasti Lawadan (1205), Kakawin Nagarakretagama (1365), Gugur tahun 1144 Saka (1222).
Pemberontakan Jayakatwang dari Gelang-gelang atau Gegelang yang menghidupkan kembali dinasti kedua Kadiri yang berumur pendek.
1292-1293	Sri Jayakatwang
(Jayakatwang)	Disebutkan dalam Kakawin Nagarakretagama (1365).
Warisan Budaya
Candi

candi Gunung Gangsir dibangun pada masa pemerintahan wangsa Isyana dan direnovasi pada periode Majapahit selanjutnya.
Candi Gununggangsir, terletak di Beji, Kabupaten Pasuruan.
Candi Penataran, Candi termegah dan terluas di Jawa Timur ini terletak di lereng barat daya Gunung Kelud, di sebelah utara Blitar, diperkirakan dibangun pada masa Raja Srengga dari Kerajaan Kadiri
Candi Watutulis di daerah Watutulis, Prambon, Sidoarjo.
Karya Sastra
Kakawin Bharatayuddha ditulis oleh Mpu Sedah dan diselesaikan oleh Mpu Panuluh.
Kakawin Lubdhaka dan Kakawin Wrettasañcaya adalah karya Mpu Tanakung.
Kakawin Hariwangsa dan Kakawin Gatotkachasraya ditulis oleh Mpu Panuluh.
Kakawin Smaradahana ditulis oleh Mpu Dharmaja.
Kakawin Sumanasantaka ditulis oleh Mpu Monaguna.
Kakawin Kresnayana ditulis oleh Mpu Triguna.
Prasasti
Prasasti Cane (1021 M).
Prasasti Baru (1030 M).
Prasasti Terep (1032 M).
Prasasti Kusambyan (1037 M), Kudu, Jombang.
Prasasti Kamalagyan (1037 M), Tropodo, Kabupaten Sidoarjo
Prasasti Pucangan (1042 M), terletak di lereng barat gunung Penanggungan, Kabupaten Mojokerto.
Prasasti Pasar Legi (1043 M), Ngimbang, Lamongan.
Prasasti Lemah Abang, Ngimbang, Lamongan.
Prasasti Purwokerto, Ngimbang, Lamongan.
Prasasti Drujugurit, Ngimbang, Lamongan.
Prasasti Wotan, Ngimbang, Lamongan.
Prasasti Sendang Gede, Ngimbang, Lamongan.
Prasasti Sendangrejo, Ngimbang, Lamongan.
Prasasti Sumbersari, Sambeng, Lamongan.
Prasasti Sumbersari II, Sambeng, Lamongan.
Prasasti lawan, Sambeng, Lamongan.
Situs
Gua Selomangleng, terletak di Mojoroto, Kediri.
Petirtaan Belahan, terletak di lereng timur Gunung Penanggungan, Kabupaten Pasuruan.
Sendang Made, terletak di Kudu, Jombang.
Lihat pula
Kerajaan Jenggala
Kerajaan Medang
Wangsa Isyana
Daftar pustaka
H.J.de Graaf dan T.H. Pigeaud. 2001. Kerajaan Islam Pertama di Jawa. Terj. Jakarta: Pustaka Utama Grafiti
Slamet Muljana. 1979. Nagarakretagama dan Tafsir Sejarahnya. Jakarta: Bhratara
Poesponegoro & Notosusanto (ed.). 1990. Sejarah Nasional Indonesia Jilid II. Jakarta: Balai Pustaka.
lbs
Kerajaan di Jawa
Didahului oleh:
Medang	Kerajaan Hindu-Buddha
1019 - 1222	Diteruskan oleh:
Tumapel
 http://www.spaetmittelalter.uni-hamburg.de/java-history/JavaNK/Java1365.Nagara-Kertagama.Canto.63-69.html
 https://storymaps.arcgis.com/stories/39bce63e4e0642d3abce6c24db470760
Kategori: Kerajaan KadiriKerajaan di NusantaraKerajaan di Jawa Timur
Halaman ini terakhir diubah pada 4 Desember 2022, pukul 08.57.
Teks tersedia di bawah Lisensi Creative Commons Atribusi-BerbagiSerupa; ketentuan tambahan mungkin berlaku. Lihat Ketentuan Penggunaan untuk lebih jelasnya.
Kebijakan privasiTentang WikipediaPenyangkalanTampilan selulerPengembangStatistikPernyataan kukiWikimedia FoundationPowered by MediaWiki

Lompat ke isi
Buka/tutup bilah samping

WikipediaEnsiklopedia Bebas
Telusuri Wikipedia
Buat akun baru

Perkakas pribadi

Anda juga bisa ikut ambil peran dalam penyebaran pengetahuan bebas. Mari bergabung dengan sukarelawan Wikipedia bahasa Indonesia!
Tutup
Daftar isi sembunyikan
Awal
Historiografi
Sejarah
Toggle Sejarah subsection
Militer
Pelayaran
Kebudayaan
Ekonomi
Struktur pemerintahan
Toggle Struktur pemerintahan subsection
Luas wilayah
Toggle Luas wilayah subsection
Hubungan diplomatik
Daftar pejabat
Toggle Daftar pejabat subsection
Daftar istilah
Toggle Daftar istilah subsection
Warisan budaya
Toggle Warisan budaya subsection
Pusat inspirasi
Toggle Pusat inspirasi subsection
Catatan
Kutipan
Referensi
Lihat pula
Pranala luar
Majapahit

Halaman
Pembicaraan
Baca
Sunting
Sunting sumber
Lihat riwayat
Dari Wikipedia bahasa Indonesia, ensiklopedia bebas
"Majapahit" beralih ke halaman ini. Untuk kegunaan lain, lihat Majapahit (disambiguasi).
Kemaharajaan Majapahit
ꦏꦫꦠꦺꦴꦤ꧀ꦩꦗꦥꦲꦶꦠ꧀ (Jawa)
ᬧ᭄ᬭᬚᬫᬚᬧᬳᬶᬢ᭄ (Bali)
ᮊᮛᮏᮃᮔ᮪ᮙᮏᮕᮠᮤᮒ᮪ (Sunda)
滿者伯夷王國 (Mandarin)
1293–1527
Bendera Majapahit
Warna kebesaran[a]
Lambang Majapahit
Lambang
Majapahit Empire id.svg
Status	Kemaharajaan
Ibu kota	Majapahit, sekarang di situs Trowulan, Jawa Timur
Bahasa yang umum digunakan	Jawa Kuno (utama), Kawi (alternatif), Sanskerta
Agama	Siwa-Buddha (Hindu dan Buddha), Kapitayan, Animisme
Pemerintahan	Monarki
Sri Maharaja	 
• 1293–1309
Raden Wijaya
• 1309–1328
Jayanagara
• 1328–1350
Tribhuwana Wijayatunggadewi
• 1350–1389
Hayam Wuruk
• 1389–1429
Wikramawardhana
• 1429–1447
Suhita
• 1447–1451
Kertawijaya
• 1451–1453
Rajasawardhana
• 1456–1466
Girishawardhana
• 1466–1468
Suraprabhawa
• 1474–1498
Girindrawardhana
Sejarah	 
• Penobatan Raden Wijaya
10 November 1293
• Sumpah Palapa
1334
• Perang Bubat
1357
• Perang Paregreg
1404–1406
• Dibawah kekuasaan Bhre Kertabhumi
1468–1474
• Invasi Kesultanan Demak
1518– 1527
Mata uang	Koin emas, koin perak, koin kepeng, koin gobog
Didahului oleh	Digantikan oleh
	Kerajaan Tumapel
	Kerajaan Melayu
	Kerajaan Kutai
	Kerajaan Singapura
	Kesultanan Samudera Pasai
	Kerajaan Bali
	Kerajaan Tanjungpura
Kesultanan Demak	
Kerajaan Blambangan	
Kesultanan Malaka	
Kesultanan Pagaruyung	
Kesultanan Makassar	
Kesultanan Bima	
Kesultanan Ternate	
Kesultanan Tidore	
Kekaisaran Brunei	
Kesultanan Kutai	
Kesultanan Aceh	
Sekarang bagian dari	 Indonesia
 Malaysia
 Singapura
 Brunei Darussalam
 Timor Leste
 Filipina
 Kamboja
 Merah putih adalah warna kebesaran yang digunakan Majapahit. Bagaimana warna itu digunakan oleh Majapahit masih dipertentangkan, lihat artikel terkait untuk keterangannya.

Artikel ini mengandung huruf dari Aksara Jawa. Tanpa dukungan perenderan yang baik, Anda mungkin akan melihat tanda tanya, kotak, atau simbol lain, bukan karakter aksara Jawa.

Artikel ini mengandung aksara Bali. Tanpa dukungan perenderan yang baik, Anda mungkin akan melihat tanda tanya, kotak, atau simbol lain, bukan karakter aksara Bali.
Bagian dari seri mengenai
Sejarah Indonesia
Nicolaes Visscher Indiae Orientalis.jpg
Prasejarah
Kerajaan Hindu-Buddha
Kerajaan Islam
Kerajaan Kristen
Kolonialisme Eropa
Kemunculan Indonesia
Kemerdekaan
Menurut topik
Garis waktu
Flag of Indonesia.svg Portal Indonesia
lbs
Bagian dari seri artikel mengenai
Sejarah Malaysia
Kemerdekaan Malaya dan proklamasi penggabungan Borneo Utara dan Serawak untuk membentuk Malaysia.
Prasejarah Malaysia
Kerajaan Awal
Kebangkitan Negara-Negara Muslim
Era Kolonial
Perang Dunia II
Era Formatif
Era Barisan Nasional
Era Pakatan-Perikatan
Insiden
Per Topik
Flag of Malaysia.svgPortal Malaysia
lbs
Majapahit (bahasa Jawa: ꦩꦗꦥꦲꦶꦠ꧀; pengucapan bahasa Jawa: [madʒapaɪt]; Sanskerta: Wilwatikta)[Catatan 1] adalah sebuah kemaharajaan yang berpusat di Jawa Timur, Indonesia, yang pernah berdiri sekitar tahun 1293–1527 M. Kemaharajaan ini didirikan oleh Raden Wijaya menantu Kertanagara, raja di raja Singhasari terakhir, dan mencapai puncak kejayaannya menjadi kemaharajaan raya yang menguasai wilayah yang luas di Nusantara pada masa kekuasaan raja Hayam Wuruk, yang berkuasa dari tahun 1350–1389.

Kemaharajaan Majapahit adalah kemaharajaan Hindu-Buddha terakhir yang menguasai Nusantara dan dianggap sebagai monarki terbesar dalam sejarah Indonesia.[2] Menurut Negarakertagama, kekuasaannya terbentang dari Jawa, Sumatra, Semenanjung Malaya, Kalimantan, Filipina (Kepulauan Sulu, Manila (Saludung), Sulawesi, Papua, dan lainnya.[3]

Historiografi
Sejarah mengenai kemaharajaan Majapahit masih menjadi salah satu subjek penelitian yang menarik untuk dibahas dan ditelusuri lebih jauh lagi.[4][5] Sumber utama yang digunakan oleh para sejarawan diantaranya adalah Pararaton ('Kitab Raja-raja') dalam bahasa Kawi dan Nagarakretagama dalam bahasa Jawa Kuno.[6] Pararaton menceritakan Ken Arok (pendiri Kerajaan Singhasari) namun juga memuat beberapa bagian pendek mengenai terbentuknya Majapahit. Sementara itu, Nagarakertagama adalah puisi Jawa Kuno yang ditulis pada masa keemasan Majapahit di bawah pemerintahan Hayam Wuruk. Kakawin Nagarakretagama pada tahun 2008 diakui sebagai bagian dalam Warisan Ingatan Dunia (Memory of the World Programme) oleh UNESCO.[7] Selain itu, terdapat beberapa prasasti dalam bahasa Jawa Kuno maupun catatan sejarah dari Tiongkok dan negara-negara lain.[8]

Beberapa sarjana seperti C.C. Berg menganggap sebagian naskah tersebut bukan catatan masa lalu, tetapi memiliki arti supernatural dalam hal dapat mengetahui masa depan.[9] Namun, banyak pula sarjana yang beranggapan bahwa garis besar sumber-sumber tersebut dapat diterima karena sejalan dengan catatan sejarah dari Tiongkok, khususnya daftar penguasa dan keadaan kerajaan yang cukup meyakinkan.[5] Pada tahun 2010, sekelompok pengusaha Jepang dipimpin Takajo Yoshiaki membiayai pembuatan kapal Majapahit atau Spirit of Majapahit yang akan berlayar ke Asia. Menurut Takajo, hal ini dilakukan untuk mengenang kerjasama Majapahit dan Kerajaan Jepang melawan Kerajaan China (Mongol) dalam perang di Samudera Pasifik.[10] Menurut Guru Besar Arkeologi Asia Tenggara National University of Singapore John N. Miksic, jangkauan kekuasaan Majapahit meliputi Sumatra dan Singapura bahkan Thailand yang dibuktikan dengan pengaruh kebudayaan, corak bangunan, candi, patung dan seni.[11] Bahkan ada perguruan silat bernama Kali Majapahit yang populer di Filipina dengan anggotanya dari Asia dan Amerika. Silat Kali Majapahit ini mengklaim berakar dari Kemaharajaan Majapahit kuno yang disebut menguasai Filipina, Singapura, Malaysia dan Selatan Thailand.[12]

Sejarah
Pendirian

Arca Harihara (paduan Siwa dan Wisnu) perwujudan Kertarajasa dari Candi Simping, Blitar, kini koleksi Museum Nasional.
Sebelum berdirinya Majapahit, Singhasari telah menjadi kerajaan paling kuat di Jawa. Hal ini menjadi perhatian Kubilai Khan, penguasa Dinasti Yuan di Tiongkok. Ia mengirim utusan yang bernama Meng Chi[13] ke Singhasari yang menuntut upeti. Kertanagara, penguasa kerajaan Singhasari yang terakhir menolak untuk membayar upeti dan mempermalukan utusan tersebut dengan merusak wajahnya dan memotong telinganya.[13][14] Kubilai Khan marah dan lalu memberangkatkan ekspedisi besar ke Jawa tahun 1293.

Ketika itu, Jayakatwang, adipati Kediri, sudah menggulingkan dan membunuh Kertanegara. Atas saran Aria Wiraraja, Jayakatwang memberikan pengampunan kepada Raden Wijaya, menantu Kertanegara, yang datang menyerahkan diri. Kemudian, Wiraraja mengirim utusan ke Daha, yang membawa surat berisi pernyataan, Raden Wijaya menyerah dan ingin mengabdi kepada Jayakatwang.[15] Jawaban dari surat di atas disambut dengan senang hati.[15] Raden Wijaya kemudian diberi hutan Tarik. Ia membuka hutan itu dan membangun desa baru. Desa itu dinamai Majapahit, yang namanya diambil dari buah maja, dan rasa "pahit" dari buah tersebut. Ketika pasukan Mongol tiba, Wijaya bersekutu dengan pasukan Mongol untuk bertempur melawan Jayakatwang. Setelah berhasil menjatuhkan Jayakatwang, Raden Wijaya berbalik menyerang sekutu Mongolnya sehingga memaksa mereka menarik pulang kembali pasukannya secara kalang-kabut karena mereka berada di negeri asing.[16][17] Saat itu juga merupakan kesempatan terakhir mereka untuk menangkap angin muson agar dapat pulang, atau mereka terpaksa harus menunggu enam bulan lagi di pulau yang asing.

Tanggal pasti yang digunakan sebagai tanggal kelahiran kerajaan Majapahit adalah hari penobatan Raden Wijaya sebagai raja, yaitu tanggal 15 bulan Kartika tahun 1215 saka yang bertepatan dengan tanggal 10 November 1293. Ia dinobatkan dengan nama resmi Kertarajasa Jayawardhana. Kerajaan ini menghadapi masalah. Beberapa orang tepercaya Kertarajasa, termasuk Ranggalawe, Sora, dan Nambi memberontak melawannya, meskipun pemberontakan tersebut tidak berhasil. Pemberontakan Ranggalawe ini didukung oleh Panji Mahajaya, Ra Arya Sidi, Ra Jaran Waha, Ra Lintang, Ra Tosan, Ra Gelatik, dan Ra Tati. Semua ini tersebut disebutkan dalam Pararaton.[18] Slamet Muljana menduga bahwa mahapatih Halayudha lah yang melakukan konspirasi untuk menjatuhkan semua orang tepercaya raja, agar ia dapat mencapai posisi tertinggi dalam pemerintahan. Namun setelah kematian pemberontak terakhir (Kuti), Halayudha ditangkap dan dipenjara, dan lalu dihukum mati.[17] Wijaya meninggal dunia pada tahun 1309.

Putra dan penerus Wijaya adalah Jayanegara. Pararaton menyebutnya Kala Gemet, yang berarti "penjahat lemah". Kira-kira pada suatu waktu dalam kurun pemerintahan Jayanegara, seorang pendeta Italia, Odorico da Pordenone mengunjungi keraton Majapahit di Jawa. Pada tahun 1328, Jayanegara dibunuh oleh tabibnya, Tanca. Ibu tirinya yaitu Gayatri Rajapatni seharusnya menggantikannya, akan tetapi Rajapatni memilih mengundurkan diri dari istana dan menjadi bhiksuni. Rajapatni menunjuk anak perempuannya Tribhuwana Wijayatunggadewi untuk menjadi ratu Majapahit. Pada tahun 1336, Tribhuwana menunjuk Gajah Mada sebagai Mahapatih, pada saat pelantikannya Gajah Mada mengucapkan Sumpah Palapa yang menunjukkan rencananya untuk melebarkan kekuasaan Majapahit dan membangun sebuah kemaharajaan. Selama kekuasaan Tribhuwana, kerajaan Majapahit berkembang menjadi lebih besar dan terkenal di kepulauan Nusantara. Tribhuwana berkuasa di Majapahit sampai kematian ibunya pada tahun 1350. Ia diteruskan oleh putranya, Hayam Wuruk.

Puncak Kejayaan

Perkembangan Kemaharajaan Majapahit, bermula di Trowulan, Majapahit, Jawa Timur, pada abad ke-13, kemudian mengembangkan pengaruhnya atas kepulauan Nusantara, hingga surut dan runtuh pada awal abad ke-16.
Pada masa pemerintahan Raja Hayam Wuruk (1350–1389) dengan Mahapatih Gajah Mada (?–1364), Majapahit mencapai puncak kejayaannya.

Menurut Kakawin Nagarakretagama pupuh XIII–XV, daerah kekuasaan Majapahit meliputi Sumatra, semenanjung Malaya, Kalimantan, Sulawesi, kepulauan Nusa Tenggara, Maluku, Papua, Tumasik (Singapura) dan sebagian kepulauan Filipina.[19] Sumber ini menunjukkan batas terluas sekaligus puncak kejayaan Kemaharajaan Majapahit.

Selain melancarkan serangan dan ekspedisi militer, Majapahit juga menempuh jalan diplomasi dan menjalin persekutuan. Kemungkinan karena didorong alasan politik, Hayam Wuruk berhasrat mempersunting Citraresmi (Pitaloka), putri Kerajaan Sunda sebagai permaisurinya.[20]

Pihak Sunda menganggap lamaran ini sebagai perjanjian persekutuan. Pada 1357 rombongan raja Sunda beserta keluarga dan pengawalnya bertolak ke Majapahit mengantarkan sang putri untuk dinikahkan dengan Hayam Wuruk. Akan tetapi Gajah Mada melihat hal ini sebagai peluang untuk memaksa kerajaan Sunda takluk di bawah Majapahit. Pertarungan antara keluarga kerajaan Sunda dengan tentara Majapahit di lapangan Bubat tidak terelakkan. Meski dengan gagah berani memberikan perlawanan, keluarga kerajaan Sunda kewalahan dan akhirnya dikalahkan. Hampir seluruh rombongan keluarga kerajaan Sunda dapat dibinasakan secara kejam.[21] Tradisi menyebutkan bahwa sang putri yang kecewa, dengan hati remuk redam melakukan "bela pati", bunuh diri untuk membela kehormatan negaranya.[22]

Kisah Pasunda Bubat menjadi tema utama dalam naskah Kidung Sunda yang disusun pada zaman kemudian di Bali dan juga naskah Carita Parahiyangan. Kisah ini disinggung dalam Pararaton tetapi sama sekali tidak disebutkan dalam Nagarakretagama.

Pada tahun 1377,[Catatan 2] beberapa tahun setelah kematian Gajah Mada, Majapahit melancarkan serangan laut untuk menumpas pemberontakan di Palembang.[2]

Kemunduran
Sesudah mencapai puncaknya pada abad ke-14, kekuasaan Majapahit berangsur-angsur melemah. Setelah wafatnya Hayam Wuruk pada tahun 1389, Majapahit memasuki masa kemunduran akibat konflik perebutan takhta. Kematian Hayam Wuruk dan adanya konflik perebutan takhta menyebabkan daerah-daerah Majapahit di bagian utara Sumatra dan Semenanjung Malaya memerdekakan diri, di mana semenanjung Malaya menjadi daerah kekuasaan Kerajaan Ayutthaya hingga nantinya muncul Kesultanan Melaka yang didukung oleh Dinasti Ming.[23]

Pewaris Hayam Wuruk adalah putri mahkota Kusumawardhani, yang menikahi sepupunya sendiri, pangeran Wikramawardhana. Hayam Wuruk juga memiliki seorang putra dari selirnya Wirabhumi yang juga menuntut haknya atas takhta.[5] Perang saudara yang disebut Perang Regreg diperkirakan terjadi pada tahun 1405–1406, antara Wirabhumi melawan Wikramawardhana. Perang ini akhirnya dimenangi Wikramawardhana, sementara Wirabhumi ditangkap dan kemudian dihukum mati. Tampaknya perang saudara ini melemahkan kendali Majapahit atas wilayah-wilayah taklukannya di daerah-daerah lain.

Pada kurun pemerintahan Wikramawardhana, serangkaian ekspedisi laut Dinasti Ming yang dipimpin oleh laksamana Cheng Ho, seorang jenderal muslim China, tiba di Jawa beberapa kali antara kurun waktu 1405 sampai 1433. Sejak tahun 1430 ekspedisi Cheng Ho ini telah menciptakan komunitas muslim China dan Arab di beberapa kota pelabuhan pantai utara Jawa, seperti di Semarang, Demak, Tuban, dan Ampel; maka Islam pun mulai memiliki pijakan di pantai utara Jawa.[24]

Ketika Majapahit didirikan, pedagang Muslim dan para penyebar agama sudah mulai memasuki Nusantara. Pada akhir abad ke-14 dan awal abad ke-15, pengaruh Majapahit di seluruh Nusantara mulai berkurang. Pada saat bersamaan, sebuah kerajaan perdagangan baru yang berdasarkan Islam, yaitu Kesultanan Malaka, mulai muncul di bagian barat Nusantara.[25] Di bagian barat kemaharajaan yang mulai runtuh ini, Majapahit tak kuasa lagi membendung kebangkitan Kesultanan Malaka yang pada pertengahan abad ke-15 mulai menguasai Selat Malaka dan melebarkan kekuasaannya ke Sumatra. Sementara itu beberapa jajahan dan daerah taklukan Majapahit di daerah lainnya di Nusantara, satu per satu mulai melepaskan diri.

Pada masa pemerintahan Wikramawardhana, daerah kekuasaan Majapahit di pulau Sumatra hanya tinggal Indragiri, Jambi dan Palembang, sebagaimana ditulis pada catatan Yingyai Shenglan ciptaan Ma Huan, salah satu penerjemah laksamana Cheng Ho. Dan setelah kematian Wikramawardhana dan masa pemerintahan penerusnya, daerah Indragiri diberikan kepada Mansur Syah dari Malaka sebagai hadiah pernikahannya dengan putri Majapahit, yang semakin mengurangi kendali Majapahit di Sumatra.

Wikramawardhana memerintah hingga tahun 1426, dan diteruskan oleh putrinya, Ratu Suhita, yang memerintah pada tahun 1426 sampai 1447. Ia adalah putri kedua Wikramawardhana dari seorang selir yang juga putri kedua Bhre Wirabhumi. Pada 1447, Suhita mangkat dan pemerintahan dilanjutkan oleh Kertawijaya, adik laki-lakinya. Ia memerintah hingga tahun 1451. Setelah Kertawijaya wafat, Bhre Pamotan menjadi raja dengan gelar Rajasawardhana dan memerintah di Kahuripan. Ia wafat pada tahun 1453 M.

Terjadi jeda waktu tiga tahun tanpa raja akibat krisis pewarisan tahta antara putra Rajasawardhana dengan Girisawardhana, adik Rajasawardhana, putra Kertawijaya. Girishawardhana menang dan naik takhta pada 1456. Ia kemudian wafat pada 1466 dan digantikan oleh Suraprabhawa (Singhawikramawardhana), adiknya, anak bungsu Kertawijaya.

Kemudian pada tahun 1468, Bhre Kertabhumi putra bungsu Rajasawardhana memberontak terhadap Singhawikramawardhana.[8] Setelah mengalami kekalahan dalam perebutan kekuasaan dengan Bhre Kertabumi, Singhawikramawardhana melarikan diri ke pedalaman di daerah Keling, Daha (bekas ibu kota Kerajaan Kediri). Setelah Singhawikramawardhana meninggal, ia digantikan oleh putranya Ranawijaya.

Pada 1474, Ranawijaya mengalahkan Kertabhumi dengan memanfaatkan ketidakpuasan umat Hindu dan Budha atas kebijakan Bhre Kertabumi serta mempersatukan kembali Majapahit menjadi satu kerajaan. Hal ini diperkuat oleh prasasti Trailokyapuri (Jiyu) dan Petak, Ranawijaya mengaku bahwa pada tahun 1474, ia telah mengalahkan Kertabhumi [26] Ranawijaya kemudian memindahkan ibukota Majapahit ke Daha dan memerintah pada kurun waktu 1474 hingga 1498 dengan gelar Girindrawardhana hingga ia digantikan oleh Patih Udara. Akibat konflik dinasti ini, Majapahit menjadi lemah dan mulai bangkitnya kekuatan kerajaan Demak.

Keruntuhan
Kekalahan Bhre Kertabhumi dari Ranawijaya pada tahun 1474, memicu perang antara Kerajaan Majapahit dengan Demak, karena Demak sudah menjadi penguasa pesisir Jawa yang dominan, dan mereka mengambil alih daerah Jambi dan Palembang dari kekuasaan Majapahit[27](hlm.154-155) yang telah terpukul dan berfokus di pedalaman pulau Jawa.

Konon, waktu berakhirnya Kemaharajaan Majapahit berkisar pada kurun waktu tahun 1478 (tahun 1400 saka,[Catatan 3] berakhirnya abad dianggap sebagai waktu lazim pergantian dinasti dan berakhirnya suatu pemerintahan) hingga tahun 1527.[28]:36 Tetapi dalam tradisi Jawa yang sebenarnya digambarkan oleh candrasengkala atau kronogram tersebut adalah wafatnya Bhre Kertabhumi pada tahun 1478.[26]

Sebenarnya perang Majapahit-Demak ini sudah mulai mereda ketika Patih Udara menggantikan Girindrawardhana dan mengakui kekuasan Demak, tetapi peperangan berkecamuk kembali ketika Patih Udara meminta bantuan Portugis untuk mengalahkan Demak. Sehingga pada tahun 1527, Demak melakukan serangan ke Daha yang mengakhiri sejarah Majapahit.[28]:54-55

Dengan jatuhnya ibukota Daha yang dihancurkan oleh Demak pada tahun 1527, pada awal abad ke-16 kekuatan kerajaan Demak akhirnya mengalahkan sisa-sisa Majapahit dan menjadi akhir dari Kerajaan Majapahit.[29] Catatan sejarah dari Tiongkok, Portugis (Tomé Pires), dan Italia (Antonio Pigafetta) mengindikasikan bahwa telah terjadi perpindahan kekuasaan Majapahit dari tangan penguasa Hindu ke tangan Pati Unus, penguasa dari Kesultanan Demak, antara tahun 1518 dan 1521 M.[26] Sisa-sisa keluarga Majapahit keturunan Girindrawardhana kemudian melarikan diri ke daerah Panarukan, Blambangan (sekarang daerah Kabupaten Banyuwangi). Sejumlah besar abdi istana, seniman, pendeta, dan anggota keluarga kerajaan mengungsi kepulau Bali.

Demak memastikan posisinya sebagai kekuatan regional dan menjadi kerajaan Islam pertama yang berdiri di tanah Jawa. Saat itu setelah keruntuhan Majapahit, sisa kerajaan Hindu yang masih bertahan di Jawa hanya tinggal kerajaan Pasuruan, Panarukan, Blambangan di ujung timur,[30]:7 serta Kerajaan Sunda yang beribu kota di Pajajaran di bagian barat. Perlahan-lahan Islam mulai menyebar seiring mundurnya masyarakat Hindu ke pegunungan dan ke Bali. Beberapa kantung masyarakat Hindu Tengger hingga kini masih bertahan di pegunungan Tengger, kawasan Bromo dan Semeru.

Militer
Pada zaman Majapahit terjadi perkembangan, pelestarian, dan penyebaran teknik pembuatan keris. Teknik pembuatan keris mengalami penghalusan dan pemilihan bahan menjadi semakin selektif. Keris pra-Majapahit dikenal berat namun semenjak masa ini dan seterusnya, bilah keris yang ringan tetapi kuat menjadi petunjuk kualitas sebuah keris. Penggunaan keris sebagai tanda kebesaran kalangan aristokrat juga berkembang pada masa ini dan meluas ke berbagai penjuru Nusantara, terutama di bagian barat.

Tentara Majapahit dibagi menjadi 2 jenis utama, yaitu prajurit (pasukan profesional) dan pasukan wajib militer yang diambil dari petani. Senjata utamanya adalah tombak.[31]:84–85, 130 Kavaleri ada dalam jumlah terbatas, mereka digunakan untuk pengintaian dan patroli, mungkin dipersenjatai dengan tombak.[31]:90, 94 Setelah serangan Mongol, penggunaan kuda semakin meluas terutama untuk perang.[32]:12–13 Kereta perang digunakan untuk mengangkut para prajurit ke medan perang.[31]:96 Gajah perang digunakan terutama untuk transportasi, atau sebagai tunggangan untuk bangsawan dan tentara berpangkat lebih tinggi.[31]:101



Senjata mesiu yang digunakan oleh Majapahit:
Cetbang berjenis meriam tangan, ditemukan di sungai Brantas, Jombang.
Sebuah cetbang berlaras ganda di atas pedati meriam (gun carriage), dengan garpu putar, sekitar tahun 1522. Mulut meriam berbentuk Nāga Jawa.
Selain keris, berkembang pula teknik pembuatan dan penggunaan tombak dan meriam kapal sederhana yang disebut cetbang. Majapahit di bawah Mahapatih (perdana menteri) Gajah Mada memanfaatkan teknologi senjata bubuk mesiu yang diperoleh dari dinasti Yuan untuk digunakan dalam armada laut.[33]:57 Cetbang awal (disebut cetbang bergaya timur) bentuknya mirip meriam dan meriam tangan Cina. Cetbang bergaya timur kebanyakan dibuat dari bahan perunggu dan merupakan meriam isian depan. Ia menembakkan proyektil berupa panah, namun peluru bulat dan proyektil co-viative[Catatan 4] juga dapat digunakan. Panah ini dapat berujung pejal tanpa peledak, maupun disertai bahan peledak dan pembakar di belakang ujungnya. Di bagian dekat belakang, terdapat kamar atau bilik bakar, yang merujuk kepada bagian yang menggelembung dekat belakang meriam, di mana mesiu ditempatkan. Cetbang ini dipasang pada dudukan tetap, ataupun sebagai meriam tangan yang diletakkan di ujung galah. Ada bagian mirip tabung di bagian belakang meriam. Pada cetbang jenis meriam tangan, tabung ini digunakan sebagai tempat untuk menancapkan galah.[34]:94

Karena dekatnya hubungan maritim Nusantara dengan wilayah India Barat, setelah tahun 1460 jenis senjata bubuk mesiu baru masuk ke Nusantara melalui perantara orang Arab. Senjata ini sepertinya adalah meriam dan bedil tradisi Turki Usmani, misalnya prangi, yang merupakan meriam putar isian belakang.[34]:94-95 Ia menghasilkan cetbang jenis baru, disebut "cetbang bergaya barat". Ia dapat dipasang sebagai meriam tetap atau meriam putar, yang kecil dapat dengan mudah dipasang di kapal-kapal kecil. Meriam ini dipergunakan sebagai senjata anti personil, bukan anti kapal. Pada zaman ini, bahkan sampai abad ke-17, prajurit angkatan laut Nusantara bertempur di panggung yang biasa disebut balai. Ditembakan pada kumpulan prajurit dengan peluru scattershot (peluru sebar atau peluru gotri, dapat berupa grapeshot, case shot, atau paku dan batu), cetbang sangat efektif untuk pertempuran jenis ini.[35]:241[36]:162

Majapahit memiliki pasukan elit yang disebut Bhayangkara. Tugas utama pasukan ini adalah untuk melindung raja dan kaum bangsawan, namun mereka juga dapat diterjunkan ke pertempuran jika diperlukan. Hikayat Banjar mencatat perlengkapan Bhayangkara di istana Majapahit:

Maka kaluar dangan parhiasannya orang barbaju-rantai ampat puluh sarta padangnya barkupiah taranggos sakhlat merah, orang mambawa astenggar [senapan sundut] ampat puluh, orang mambawa parisai sarta padangnya ampat puluh, orang mambawa dadap [sejenis perisai][Catatan 5] sarta sodoknya [senjata mirip tombak dengan mata lebar][Catatan 6] sapuluh, orang mambawa panah sarta anaknya sapuluh, yang mambawa tumbak parampukan[Catatan 7] barsulam amas ampat puluh, yang mambawa tameng Bali bartulis air mas ampat puluh.
— Hikayat Banjar, 6.3[37]:Baris 1209–1214[38]

Bagian yang dipotong dari peta Laut Cina di atlas Miller, menunjukkan jong bertiang enam dan tiga.
Bagian yang dipotong dari peta Laut Cina di atlas Miller, menunjukkan jong bertiang enam dan tiga.

 
Relief yang terpisah, mungkin dari candi Penataran, menampilkan zirah sisik.
Relief yang terpisah, mungkin dari candi Penataran, menampilkan zirah sisik.

 
Arquebus Jiaozhi ini mirip dengan arquebus Jawa.
Arquebus Jiaozhi ini mirip dengan arquebus Jawa.

 
Patung dewa memegang sebuah kuiras, dari Nganjuk, Jawa Timur, pada masa sebelumnya (abad ke-10 sampai ke-11).
Patung dewa memegang sebuah kuiras, dari Nganjuk, Jawa Timur, pada masa sebelumnya (abad ke-10 sampai ke-11).

 
Berbagai macam keris dan senjata galah (tombak) dari Jawa.
Berbagai macam keris dan senjata galah (tombak) dari Jawa.

 
Bodhisatwa Manjusri memegang pedang, dari Candi Jago, 1343.
Bodhisatwa Manjusri memegang pedang, dari Candi Jago, 1343.

Pasukan militer di berbagai bagian Asia Tenggara menggunakan pakaian pelindung ringan. Seperti umumnya di Asia Tenggara, sebagian besar pasukan Jawa terdiri dari rakyat jelata yang dimobilisasi sementara dari petani yang dipimpin oleh prajurit dan kasta bangsawan. Prajurit yang lebih kaya menggunakan baju pelindung yang disebut kawaca.[Catatan 8][31]:78 Menurut Irawan Djoko Nugroho, baju pelindung ini mungkin berbentuk seperti tabung panjang dan terbuat dari tembaga yang dicetak.[39] Sebaliknya, infanteri biasa mengenakan zirah sisik yang disebut siping-siping.[31]:75, 78, 79 Jenis baju zirah lain yang digunakan di Jawa era Majapahit adalah waju rante (zirah rantai) dan karambalangan (lapisan logam yang dikenakan di depan dada).[40][41][42][43]:111-113 Dalam Kidung Sunda pupuh 2 bait 85 dijelaskan bahwa mantri-mantri (menteri atau perwira) Gajah Mada mengenakan baju besi dalam bentuk zirah rantai atau plastron dengan hiasan emas dan mengenakan pakaian kuning,[44]:103 sedangkan dalam Kidung Sundayana pupuh 1 bait 95 disebutkan bahwa Gajah Mada mengenakan karambalangan berhias timbul dari emas, bersenjata tombak berlapis emas, dan perisai penuh dengan hiasan dari intan berlian.[41][42]

Majapahit juga mengawali penggunaan senjata api di Nusantara. Meskipun pengetahuan membuat senjata berbasis serbuk mesiu di Nusantara sudah dikenal setelah serangan Mongol ke Jawa, dan pendahulu senjata api, yaitu meriam galah (bedil tombak), dicatat digunakan oleh Jawa pada tahun 1413,[35]:245[45] pengetahuan membuat senjata api sejati datang jauh kemudian, setelah pertengahan abad ke-15. Ia dibawa oleh negara-negara Islam di Asia Barat, kemungkinan besar oleh orang Arab. Tahun pengenalan yang tepat tidak diketahui, tetapi dapat dengan aman disimpulkan tidak lebih awal dari tahun 1460.[46]:23

Catatan Tome Pires tahun 1513 menyebutkan pasukan tentara Gusti Pati (Patih Udara), wakil raja Batara Vojyaya (mungkin Brawijaya atau Ranawijaya), berjumlah 200.000 orang, 2.000 diantaranya adalah prajurit berkuda dan 4.000 adalah musketir.[27]:175-176 Duarte Barbosa sekitar tahun 1514 mengatakan bahwa penduduk Jawa sangat ahli dalam membuat artileri dan merupakan penembak artileri yang baik. Mereka membuat banyak meriam 1 pon (cetbang atau rentaka), senapan lontak panjang, spingarde (arquebus), schioppi (meriam tangan), api Yunani, gun (bedil besar atau meriam), dan senjata api atau kembang api lainnya.[47]:198[48]:224 Setiap tempat disana dianggap sangat baik dalam mencetak/mengecor artileri, dan juga dalam ilmu penggunaanya.[47]:198[49]:254




Relief di candi Penataran.
Kavaleri sejati pertama, unit terorganisir dari penunggang kuda yang kooperatif, mungkin telah muncul di Jawa selama abad ke-12 M.[50] Naskah Jawa kuno kakawin Bhomāntaka menyebutkan kisah kuda Jawa awal dan sejarah menunggang kuda.[51]:436 Naskah tersebut mungkin mencerminkan konflik (secara alegoris) antara kavaleri Jawa yang baru jadi dan infanteri elit mapan yang membentuk inti dari pasukan Jawa sampai abad ke-12.[52]:113 Pada abad ke-14 M, Jawa menjadi peternak kuda yang penting dan pulau ini bahkan terdaftar di antara pemasok kuda ke Cina.[53]:208 Selama masa Majapahit, jumlah kuda dan kualitas kuda keturunan Jawa terus berkembang sehingga pada tahun 1513 masehi Tomé Pires memuji kuda-kuda yang sangat dihiasi dari bangsawan Jawa, dilengkapi dengan sanggurdi bertatahkan emas dan pelana yang dihiasi dengan mewah yang "tidak ditemukan di tempat lain di dunia".[27]:174-175[54]:196–197 Kuda poni Sumbawa tampaknya berasal dari kuda domestikasi Jawa yang diperkenalkan oleh Majapahit sejak abad ke-14 M.[55]:52–53

Majapahit memiliki pasukan angkatan laut yang berbeda dengan satuan pasukan darat, yang disebut wwang jaladhi. Pasukan laut mendapat perlakuan istimewa dalam hal fasilitas. Personel angkatan laut Majapahit Majapahit berjumlah besar, sebagaimana dicatat Nagarakretagama pupuh 16 bait 5:[56][57]:17, 148, 281[58]

irika tang anyabhumi sakhahemban ing Yawapuri, (Kemudian Anyabhumi [tanah-tanah lain] di mana saja semuanya disatukan di kerajaan Jawa,)

amateh i sajna sang nrpati khapwa satya ring ulah, (mematuhi setiap perintah dari sang raja. Semuanya setia dalam sikap,)

pituwi sing ajñalanghyana dinon wiśirnna sahana, (kendati ada para pelanggar perjanjian, mereka diserang oleh tentara yang dikirim ke luar negeri dan dihancurkan semuanya,)

tekap ikang watek jaladhi mantry aneka suyaśa. (dengan pekerjaan itu, kelompok mantri jaladhi [perwira angkatan laut] berjumlah banyak, agung.)





Prajurit dan perlengkapannya sebagaimana digambarkan di candi Penataran.
Untuk angkatan laut, armada Majapahit menggunakan jong secara besar-besaran sebagai kekuatan lautnya. Tidak diketahui secara pasti berapa jumlah total jong yang dimiliki Majapahit, tetapi jumlah terbesar yang pernah digunakan dalam satu ekspedisi adalah berjumlah 400 buah, tepatnya saat Majapahit menyerang Pasai.[59] Setiap kapal berukuran panjang keseluruhan sekitar 28.99–88.56 meter, berat mati (deadweight) sekitar 100–2000 ton dan dapat membawa 50–1000 orang. Sebuah jong dari tahun 1420 hampir saja menyeberangi samudera Atlantik.[60] Jenis jong besar sembilan tingkat yang tercatat di Kidung Panji Wijayakrama-Rangga Lawe (sekitar 1334) disebut jong sasangawangunan, ia membawa 1000 prajurit dengan layar merah.[61]:91 Jong yang umum digunakan oleh Majapahit rata-ratanya dapat membawa 600–700 orang, berbobot mati 1200–1400 ton, dengan panjang keseluruhan sekitar 76,18–79,81 m.[62]:60-62 Sebelum tragedi Bubat tahun 1357, raja Sunda dan keluarganya datang di Majapahit setelah berlayar di laut Jawa dalam armada dengan 200 kapal besar dan 2000 kapal yang lebih kecil.[44]:16-17, 76-77 Kapal yang dinaiki keluarga kerajaan adalah sebuah jong hibrida Cina-Asia tenggara bertingkat sembilan (Bahasa Jawa kuno: Jong sasanga wangunan ring Tatarnagari tiniru). Kapal hibrida ini mencampurkan teknik China dalam pembuatannya, yaitu menggunakan paku besi selain menggunakan pasak kayu dan juga pembuatan sekat kedap air (watertight bulkhead), dan penambahan kemudi sentral.[63]:270[64]:272-276 Jenis kapal lain yang digunakan Majapahit adalah malangbang, kelulus, jongkong, cerucuh, tongkang, dan pelang.[59][65][66] Pada abad ke-16 lancaran dan penjajap juga digunakan.[27]:195[67]:282 Penggambaran angkatan laut Majapahit pada masa modern sering kali menggambarkan kapal-kapal bercadik, namun pada kenyataannya kapal ini berasal dari abad ke-8 yaitu kapal Borobudur, yang digunakan dinasti Sailendra. Penelitian oleh Irawan Djoko Nugroho menyimpulkan bahwa jenis kapal utama yang digunakan oleh Majapahit tidak menggunakan cadik, dan menggunakan ukiran Borobudur sebagai dasar rekonstruksi kapal Majapahit adalah salah.[68][69]

Pelayaran
Selama era Majapahit penjelajahan orang-orang Nusantara mencapai prestasi terbesarnya. Ludovico di Varthema (1470–1517), dalam bukunya Itinerario de Ludouico de Varthema Bolognese menyatakan bahwa orang Jawa Selatan berlayar ke "negeri jauh di selatan" hingga mereka tiba di sebuah pulau di mana satu hari hanya berlangsung selama empat jam dan "lebih dingin daripada di bagian dunia mana pun". Penelitian modern telah menentukan bahwa tempat tersebut terletak setidaknya 900 mil laut (1666 km) selatan dari titik paling selatan Tasmania.[70]:248-251

Orang Jawa, seperti suku-suku Austronesia lainnya, menggunakan sistem navigasi yang mantap: Orientasi di laut dilakukan menggunakan berbagai tanda alam yang berbeda-beda, dan dengan memakai suatu teknik perbintangan sangat khas yang dinamakan star path navigation. Pada dasarnya, para navigator menentukan haluan kapal ke pulau-pulau yang dikenali dengan menggunakan posisi terbitnya dan terbenamnya bintang-bintang tertentu di atas cakrawala.[71]:10 Pada zaman Majapahit, kompas dan magnet telah digunakan, selain itu kartografi (ilmu pemetaan) telah berkembang. Pada tahun 1293 Raden Wijaya memberikan sebuah peta dan catatan sensus penduduk pada pasukan Mongol dinasti Yuan, menunjukkan bahwa pembuatan peta telah menjadi bagian formal dari urusan pemerintahan di Jawa.[72]:53 Penggunaan peta yang penuh garis-garis memanjang dan melintang, garis rhumb, dan garis rute langsung yang dilalui kapal dicatat oleh orang Eropa, sampai-sampai orang Portugis menilai peta Jawa merupakan peta terbaik pada awal tahun 1500-an.[70][73]

Ketika Afonso de Albuquerque menaklukkan Malaka (1511), orang Portugis mendapatkan sebuah peta dari seorang mualim Jawa, yang juga menampilkan bagian dari benua Amerika. Mengenai peta itu, Albuquerque berkata:[74]

... peta besar seorang mualim Jawa, yang berisi Tanjung Harapan, Portugal dan tanah Brazil, Laut Merah dan Laut Persia, Kepulauan Cengkih, navigasi orang Cina dan Gom, dengan garis rhumb dan rute langsung yang bisa ditempuh oleh kapal, dan dataran gigir (hinterland), dan bagaimana kerajaan berbatasan satu sama lain. Bagiku, Tuan, ini adalah hal terbaik yang pernah saya lihat, dan Yang Mulia akan sangat senang melihatnya memiliki nama-nama dalam tulisan Jawa, tetapi saya punya saya orang Jawa yang bisa membaca dan menulis, saya mengirimkan karya ini kepada Yang Mulia, yang ditelusuri Francisco Rodrigues dari yang lain, di mana Yang Mulia dapat benar-benar melihat di mana orang Cina dan Gore (Jepang) datang, dan tentu saja kapal Anda harus pergi ke Kepulauan Cengkih, dan di mana tambang emas ada, dan pulau Jawa dan Banda.
– Surat Albuquerque untuk raja Manuel I dari Portugal, 1 April 1512.

Kebudayaan

Gapura Bajang Ratu, gerbang masuk salah satu kompleks bangunan penting di ibu kota Majapahit. Bangunan ini masih tegak berdiri di Trowulan.
"Dari semua bangunan, tidak ada tiang yang luput dari ukiran halus dan warna indah" [Dalam lingkungan dikelilingi tembok] "terdapat pendopo anggun beratap ijuk, indah bagai pemandangan dalam lukisan... Kelopak bunga katangga gugur tertiup angin dan bertaburan di atas atap. Atap itu bagaikan rambut gadis yang berhiaskan bunga, menyenangkan hati siapa saja yang memandangnya".

— Gambaran ibu kota Majapahit kutipan dari Nagarakertagama.

Nagarakretagama menyebutkan budaya keraton yang adiluhung dan anggun, dengan cita rasa seni dan sastra yang halus, serta sistem ritual keagamaan yang rumit. Peristiwa utama dalam kalender tata negara digelar tiap hari pertama bulan Caitra (Maret–April) ketika semua utusan dari semua wilayah taklukan Majapahit datang ke istana untuk membayar upeti atau pajak. Kawasan Majapahit secara sederhana terbagi dalam tiga jenis: keraton termasuk kawasan ibu kota dan sekitarnya; wilayah-wilayah di Jawa Timur dan Bali yang secara langsung dikepalai oleh pejabat yang ditunjuk langsung oleh raja; serta wilayah-wilayah taklukan di kepulauan Nusantara yang menikmati otonomi luas.[75]

Ibu kota Majapahit di Trowulan adalah kota besar dan terkenal dengan perayaan besar keagamaan yang diselenggarakan setiap tahun. Agama Buddha, Siwa, dan Waisnawa (pemuja Wisnu) dipeluk oleh penduduk Majapahit, dan raja dianggap sekaligus titisan Buddha, Siwa, maupun Wisnu. Nagarakertagama sama sekali tidak menyinggung tentang Islam, akan tetapi sangat mungkin terdapat beberapa pegawai atau abdi istana muslim saat itu.[2]

Walaupun batu bata telah digunakan dalam candi pada masa sebelumnya, arsitek Majapahitlah yang paling ahli menggunakannya.[76] Candi-candi Majapahit berkualitas baik secara geometris dengan memanfaatkan getah tumbuhan merambat dan gula merah sebagai perekat batu bata. Contoh candi Majapahit yang masih dapat ditemui sekarang adalah Candi Tikus dan Gapura Bajang Ratu di Trowulan, Mojokerto. Beberapa elemen arsitektur berasal dari masa Majapahit, antara lain gerbang terbelah candi bentar, gapura paduraksa (kori agung) beratap tinggi, dan pendopo berdasar struktur bata. Gaya bangunan seperti ini masih dapat ditemukan dalam arsitektur Jawa dan Bali.

"Di sekitar itu (pulau Sumatra) ada pulau besar, bernama Jawa, yang memiliki ukuran 3000 mil. Dan rajanya memiliki bawahan tujuh raja yang bermahkota.[Catatan 9] Sekarang pulau ini sangat padat penduduknya, dan merupakan yang terbaik kedua dari semua pulau yang ada. Karena di dalamnya tumbuh kapur barus, kemukus, kapulaga, buah pala, dan banyak rempah-rempah berharga lainnya. Ia juga memiliki persediaan makanan yang baik kecuali anggur.
Raja pulau ini (Jawa) memiliki istana yang benar-benar mengagumkan. Karena itu sangat besar, dan memiliki tangga yang sangat besar, lebar dan tinggi, dan anak tangganya dari emas dan perak secara bergantian. Demikian juga jalan istana dipasangi satu ubin dari emas dan yang lain dari perak, dan dindingnya di bagian dalam dilapisi dengan lapisan emas, di mana ada pahatan ksatria yang semuanya terbuat dari emas, yang memiliki lingkaran emas besar di sekitar kepala mereka, seperti yang kami berikan untuk sosok orang-orang suci. Dan lingkaran ini semua dikelilingi dengan batu mulia. Terlebih lagi, langit-langitnya terbuat dari emas murni, dan singkatnya, istana ini lebih kaya dan lebih indah daripada istana lain yang ada pada hari ini di dunia.
Sekarang Khan Agung Cathay (Cina dinasti Yuan) sudah sering berperang dengan raja ini; tetapi selalu dapat dikalahkan."

— Gambaran Majapahit menurut Mattiussi (Pendeta Odorico da Pordenone).[77]:89[78]



Relief candi Panataran, menggambarkan tembok, gerbang, menara, dan warga.
Catatan yang berasal dari sumber Italia mengenai Jawa pada era Majapahit didapatkan dari catatan perjalanan Mattiussi, seorang pendeta Ordo Fransiskan dalam bukunya: "Perjalanan Pendeta Odorico da Pordenone". Ia mengunjungi beberapa tempat di Nusantara: Sumatra, Jawa, dan Banjarmasin di Kalimantan. Ia dikirim Paus untuk menjalankan misi Katolik di Asia Tengah. Pada 1318 ia berangkat dari Padua, menyeberangi Laut Hitam dan menembus Persia, terus hingga mencapai Kolkata, Madras, dan Srilanka. Lalu menuju kepulauan Nikobar hingga mencapai Sumatra, lalu mengunjungi Jawa dan Banjarmasin. Ia kembali ke Italia melalui jalan darat lewat Vietnam, China, terus mengikuti Jalur Sutra menuju Eropa pada 1330.


Raja Jawa (Majapahit) dan 7 raja bawahannya, seperti yang dibayangkan dalam manuskrip Inggris abad ke-15 yang berisi catatan pendeta Odoric.
Di buku ini ia menyebut kunjungannya di Jawa tanpa menjelaskan lebih rinci nama tempat yang ia kunjungi. Disebutkan raja Jawa menguasai tujuh raja bawahan. Disebutkan juga di pulau ini terdapat banyak cengkih, kemukus, pala, dan berbagai rempah-rempah lainnya. Ia menyebutkan istana raja Jawa sangat mewah dan mengagumkan, penuh bersepuh emas dan perak. Ia juga menyebutkan raja Mongol beberapa kali berusaha menyerang Jawa, tetapi selalu gagal dan berhasil diusir kembali. Kerajaan Jawa yang disebutkan di sini tak lain adalah Majapahit yang dikunjungi pada suatu waktu dalam kurun 1318–1330 pada masa pemerintahan Jayanegara.

Diplomat Portugis Tomé Pires, yang mengunjungi Nusantara pada 1512, mencatat kebudayaan Jawa pada akhir zaman Majapahit setelah kunjungannya ke Jawa antara Maret–Juni 1513.[27]:xxv Kisah Pires menceritakan tentang para tuan dan bangsawan di Jawa. Mereka digambarkan sebagai:

... tinggi dan tampan, dengan dekorasi mewah, dan mereka memiliki banyak kuda yang sangat dihiasi. Mereka menggunakan keris, pedang, dan tombak dari berbagai jenis, semuanya bertatahkan emas. Mereka adalah pemburu dan penunggang kuda yang hebat—kuda itu memiliki sanggurdi semua bertatahkan emas dan pelana yang juga bertatahkan, yang tidak dapat ditemukan di tempat lain di dunia. Penguasa Jawa begitu mulia dan agung sehingga tidak ada bangsa yang bisa dibandingkan dengan mereka di wilayah yang luas di bagian ini. Kepala mereka dicukur—setengah dicukur—sebagai tanda keindahan, dan mereka selalu mengusap rambut mereka dari dahi ke atas tidak seperti yang dilakukan orang Eropa. Penguasa Jawa dipuja seperti dewa, dengan rasa hormat yang tinggi dan penghormatan yang dalam.
Para bangsawan pergi berburu atau mencari kesenangan dengan gaya yang agung. Mereka menghabiskan seluruh waktu mereka dalam kesenangan, pengiring memiliki begitu banyak tombak dengan gagang emas dan perak, begitu kaya tatahannya, dengan begitu banyak anjing jenis harrier, greyhound dan anjing lainnya; dan mereka memiliki begitu banyak gambar yang dilukis dengan pemandangan dan pemandangan berburu. Pakaian mereka dihiasi dengan emas, keris, pedang, pisau, kelewang mereka semua bertatahkan emas; mereka memiliki sejumlah selir, kuda jennet, gajah, lembu untuk menarik kereta dari kayu yang dicat dan bersepuh emas. Para bangsawan pergi dengan kereta kemenangan, dan jika mereka pergi melalui laut mereka pergi dengan kelulus yang dicat dan dihiasi; ada apartemen indah untuk wanita mereka, tempat lain untuk para bangsawan yang menemaninya.[27]:174–175 dan 200

Ekonomi

Celengan zaman Majapahit, abad 14–15 Masehi Trowulan, Jawa Timur. (Koleksi Museum Gajah, Jakarta)
Majapahit merupakan negara agraris dan sekaligus negara perdagangan.[28]:37 Pajak dan denda dibayarkan dalam uang tunai. Ekonomi Jawa telah sebagian mengenal mata uang sejak abad ke-8 pada masa kerajaan Medang yang menggunakan butiran dan keping uang emas dan perak. Sekitar tahun 1300, pada masa pemerintahan raja pertama Majapahit, sebuah perubahan moneter penting terjadi: keping uang dalam negeri diganti dengan uang "kepeng" yaitu keping uang tembaga impor dari China. Pada November 2008 sekitar 10.388 keping koin China kuno seberat sekitar 40 kilogram digali dari halaman belakang seorang penduduk di Sidoarjo. Badan Pelestarian Peninggalan Purbakala (BP3) Jawa Timur memastikan bahwa koin tersebut berasal dari era Majapahit.[79] Alasan penggunaan uang logam atau koin asing ini tidak disebutkan dalam catatan sejarah, akan tetapi kebanyakan ahli menduga bahwa dengan semakin kompleksnya ekonomi Jawa, maka diperlukan uang pecahan kecil atau uang receh dalam sistem mata uang Majapahit agar dapat digunakan dalam aktivitas ekonomi sehari-hari di pasar Majapahit. Peran ini tidak cocok dan tidak dapat dipenuhi oleh uang emas dan perak yang mahal.[80]

Daoyi Zhi, yang ditulis sekitar 1339 M, menyebutkan tentang kekayaan dan kemakmuran Jawa pada masa itu:

"Ladang-ladang di Jawa kaya dan tanahnya rata dan berair baik, maka dari itu gandum dan beras berlimpah, dua kali lipat di negara lain. Orang-orang tidak mencuri, dan apa yang dijatuhkan di jalan tidak diambil. Pepatah umum: "Jawa yang makmur" berarti negara ini. Pria dan wanita menutup kepala mereka dan mengenakan pakaian panjang."[81]:124

Beberapa gambaran mengenai skala ekonomi dalam negeri Jawa saat itu dikumpulkan dari berbagai data dan prasasti. Prasasti Canggu yang berangka tahun 1358 menyebutkan sebanyak 78 titik perlintasan berupa tempat perahu penyeberangan di dalam negeri (mandala Jawa).[75] Prasasti dari masa Majapahit menyebutkan berbagai macam pekerjaan dan spesialisasi karier, mulai dari pengrajin emas dan perak, hingga penjual minuman, dan jagal atau tukang daging. Meskipun banyak di antara pekerjaan-pekerjaan ini sudah ada sejak zaman sebelumnya, namun proporsi populasi yang mencari pendapatan dan bermata pencarian di luar pertanian semakin meningkat pada era Majapahit.

Menurut catatan Wang Ta-Yuan, pedagang Tiongkok, komoditas ekspor Jawa pada saat itu ialah lada, garam, kain, dan burung Kakaktua, sedangkan komoditas impornya adalah mutiara, emas, perak, sutra, barang keramik, dan barang dari besi. Mata uangnya dibuat dari campuran perak, timah putih, timah hitam, dan tembaga.[82] Selain itu, catatan Odorico da Pordenone, biarawan Katolik Roma dari Italia yang mengunjungi Jawa pada tahun 1321, menyebutkan bahwa istana raja Jawa penuh dengan perhiasan emas, perak, dan permata.[83]

Kemakmuran Majapahit diduga karena dua faktor. Faktor pertama adalah kesuburan lahan di lembah Sungai Brantas dan Bengawan Solo di dataran rendah Jawa Timur utara mendukung pertanian padi. Pada masa jayanya Majapahit membangun berbagai infrastruktur irigasi, sebagian dengan dukungan pemerintah. Faktor kedua adalah pelabuhan-pelabuhan Majapahit di pantai utara Jawa yang berperan penting sebagai ekspor-impor serta transit bagi komoditas rempah-rempah dari timur (Maluku). Pajak yang dikenakan pada komoditas rempah-rempah yang melewati Jawa merupakan sumber pemasukan penting bagi Majapahit.[75]

Nagarakretagama menyebutkan bahwa kemasyhuran penguasa Wilwatikta telah menarik banyak pedagang asing, di antaranya pedagang dari India, Khmer, Siam, dan Tiongkok. Pajak khusus dikenakan pada orang asing terutama yang menetap semi-permanen di Jawa dan melakukan pekerjaan selain perdagangan internasional. Majapahit memiliki pejabat sendiri untuk mengurusi pedagang dari India dan Tiongkok yang menetap di ibu kota kerajaan maupun berbagai tempat lain di wilayah Majapahit di Jawa.[84]

Selama era Majapahit, hampir semua komoditas dari Asia ditemukan di Jawa. Ini dikarenakan perdagangan laut ekstensif yang dilakukan oleh kerajaan Majapahit yang menggunakan berbagai jenis kapal, terutamanya jong, untuk berdagang ke tempat-tempat yang jauh.[85] Ma Huan (penerjemah Cheng Ho) yang mengunjungi Jawa pada 1413, menyatakan bahwa pelabuhan di Jawa adalah memperdagangkan barang dan menawarkan layanan yang lebih banyak dan lebih lengkap daripada pelabuhan lain di Asia Tenggara.[86]

Struktur pemerintahan
Majapahit memiliki struktur pemerintahan dan susunan birokrasi yang teratur pada masa pemerintahan Hayam Wuruk, dan tampaknya struktur dan birokrasi tersebut tidak banyak berubah selama perkembangan sejarahnya.[87] Raja dianggap sebagai penjelmaan dewa di dunia dan ia memegang otoritas politik tertinggi.

Aparat birokrasi
Raja dibantu oleh sejumlah pejabat birokrasi dalam melaksanakan pemerintahan, dengan para putra dan kerabat dekat raja memiliki kedudukan tinggi. Perintah raja biasanya diturunkan kepada pejabat-pejabat di bawahnya, antara lain yaitu:

Rakryan Mahamantri Katrini, biasanya dijabat putra-putra raja
Rakryan Mantri ri Pakira-kiran, dewan menteri yang melaksanakan pemerintahan
Dharmmadhyaksa, para pejabat hukum keagamaan
Dharmma-upapatti, para pejabat keagamaan
Dalam Rakryan Mantri ri Pakira-kiran terdapat seorang pejabat yang terpenting yaitu Rakryan Mapatih atau Patih Hamangkubhumi. Pejabat ini dapat dikatakan sebagai perdana menteri yang bersama-sama raja dapat ikut melaksanakan kebijaksanaan pemerintahan. Selain itu, terdapat pula semacam dewan pertimbangan kerajaan yang anggotanya para sanak saudara raja, yang disebut Bhattara Saptaprabhu.

Pembagian wilayah

Kawasan inti Majapahit dan provinsinya (Mancanagara) di kawasan Jawa Timur dan Jawa Tengah, termasuk pulau Madura dan Bali.
Dalam pembentukannya, kerajaan Majapahit merupakan kelanjutan Singhasari,[17] terdiri atas beberapa kawasan tertentu di bagian timur dan bagian tengah Jawa. Daerah ini diperintah oleh uparaja yang disebut Paduka Bhattara yang bergelar Bhre atau "Bhatara i". Gelar ini adalah gelar tertinggi bangsawan kerajaan. Biasanya posisi ini hanyalah untuk kerabat dekat raja. Tugas mereka adalah untuk mengelola kerajaan mereka, memungut pajak, dan mengirimkan upeti ke pusat, dan mengelola pertahanan di perbatasan daerah yang mereka pimpin.

Hierarki dalam pengklasifikasian wilayah di kerajaan Majapahit dikenal sebagai berikut:

Bhumi: kerajaan, diperintah oleh Raja
Nagara: diperintah oleh rajya (gubernur), atau natha (tuan), atau bhre (pangeran atau bangsawan)
Watek: dikelola oleh wiyasa,
Kuwu: dikelola oleh lurah,
Wanua: dikelola oleh thani,
Kabuyutan: dusun kecil atau tempat sakral.
Saat Majapahit memasuki era kemaharajaan Thalasokrasi saat pemerintahan Gajah Mada, beberapa negara bagian di luar negeri juga termasuk dalam lingkaran pengaruh Majapahit, sebagai hasilnya, konsep teritorial yang lebih besar pun terbentuk:

Negara Agung, atau Negara Utama, inti kerajaan. Area awal Majapahit atau Majapahit Lama selama masa pembentukannya sebelum memasuki era kemaharajaan. Yang termasuk area ini adalah ibu kota kerajaan dan wilayah sekitarnya di mana raja secara efektif menjalankan pemerintahannya. Area ini meliputi setengah bagian timur Jawa, dengan semua provinsinya yang dikelola oleh para Bhre (bangsawan), yang merupakan kerabat dekat raja.
Mancanegara, area yang melingkupi Negara Agung. Area ini secara langsung dipengaruhi oleh kebudayaan Jawa, dan wajib membayar upeti tahunan. Akan tetapi, area-area tersebut biasanya memiliki penguasa atau raja pribumi, yang kemungkinan membentuk persekutuan atau menikah dengan keluarga kerajaan Majapahit. Kerajaan Majapahit menempatkan birokrat dan pegawainya di tempat-tempat ini dan mengatur kegiatan perdagangan luar negeri mereka dan mengumpulkan pajak, namun mereka menikmati otonomi internal yang cukup besar. Wilayah Mancanegara termasuk di dalamnya seluruh daerah Pulau Jawa lainnya, Madura, Bali, dan juga Dharmasraya, Pagaruyung, Lampung dan Palembang di Sumatra.
Nusantara, adalah area yang tidak mencerminkan kebudayaan Jawa, tetapi termasuk ke dalam koloni dan mereka harus membayar upeti tahunan. Mereka menikmati otonomi yang cukup luas dan kebebasan internal, dan Majapahit tidak merasa penting untuk menempatkan birokratnya atau tentara militernya di sini; akan tetapi, tantangan apa pun yang terlihat mengancam ketuanan Majapahit atas wilayah itu akan menuai reaksi keras. Termasuk dalam area ini adalah kerajaan kecil dan koloni di Maluku, Kepulauan Nusa Tenggara, Sulawesi, Kalimantan, dan Semenanjung Malaya.
Ketiga kategori tersebut masuk ke dalam lingkaran pengaruh Kerajaan Majapahit. Akan tetapi Majapahit juga mengenal lingkup keempat yang didefinisikan sebagai hubungan diplomatik luar negeri.

Luas wilayah
Artikel utama: Wilayah Majapahit
Menurut kitab Negarakretagama
Menurut Kakawin Nagarakretagama pupuh XIII–XV, daerah kekuasaan Majapahit meliputi Sumatra, semenanjung Malaya, Kalimantan, Sulawesi, kepulauan Nusa Tenggara, Maluku, Papua, Tumasik (Singapura) dan sebagian kepulauan Filipina.[19] Sumber ini menunjukkan batas terluas sekaligus puncak kejayaan Kemaharajaan Majapahit.

Menurut prasasti Jayanegara II
Prasasti Tuhañaru/Jayanagara II, berasal dari tahun 1245 Saka/1323 Masehi, mencatat aneksasi wilayah di luar Jawa:

... seperti bulan yang membuka kembang tunjung-jantung dari perkampungan segala orang baik-baik; yang membinasakan segala musuh; seperti matahari yang melenyapkan kegelapan pada waktu malam hari, yang digembirakan Wipra dan Satria, yang berbahagia dapat bertegak nama penobatan raja, berbunyi: Iswara Sundarapandyadewa, ...

Menurut H.B. Sarkar, gelar raja Jayanegara ini menandakan bahwa Majapahit memegang kekuasaan tinggi (suzerainty) atas raja Pandia di India Selatan.[88]

Menurut Kidung Sunda
Berdasarkan Kidung Sunda pupuh 1 bait 54b dan 65a, kekuasaan Majapahit meliputi Palembang, Tumasik (Singapura), Sampit, Madura, Bali, Koci (Cochinchina, Vietnam), Wandan (Banda, Maluku Tengah), Tanjungpura (Kalimantan) dan Sawakung (Pulau Sebuku).[89]:20, 23[90]

Menurut Kidung Harsa-Wijaya
Kidung Harsa Wijaya mencatat wilayah Majapahit di luar Jawa antara lain Bali, Tatar, Tumasik, Sampi, Gurun, Wandan, Tanjung-pura, Dompo, Palembang, Makasar, dan Koci.[91]

Menurut Serat Pararaton
Selama masa pemerintahan Hayam Wuruk (1350 s.d. 1389) ada 12 wilayah pusat Majapahit, yang dikelola oleh kerabat dekat raja.

No	Provinsi	Gelar	Penguasa	Hubungan dengan Raja
1	Kahuripan (sekarang Sidoarjo)	Bhre Kahuripan	Tribhuwanatunggadewi	ibu suri
2	Daha (sekarang Kediri)	Bhre Daha	Rajadewi Maharajasa	bibi sekaligus ibu mertua
3	Tumapel (sekarang Malang)	Bhre Tumapel	Kertawardhana	ayah
4	Wengker (sekarang Ponorogo)	Bhre Wengker	Wijayarajasa	paman sekaligus ayah mertua
5	Matahun (sekarang Bojonegoro)	Bhre Matahun	Rajasawardhana	suami dari Putri Lasem, sepupu raja
6	Wirabhumi (sekarang Blambangan)	Bhre Wirabhumi	Bhre Wirabhumi[Catatan 10]1	anak
7	Paguhan	Bhre Paguhan	Singhawardhana	saudara laki-laki ipar
8	Kabalan	Bhre Kabalan	Kusumawardhani[Catatan 11]2	anak perempuan
9	Pawanuan	Bhre Pawanuan	Surawardhani	keponakan perempuan
10	Lasem (sekarang Rembang)	Bhre Lasem	Rajasaduhita Indudewi	sepupu
11	Pajang (sekarang Surakarta)	Bhre Pajang	Rajasaduhita Iswari	saudara perempuan
12	Mataram (sekarang Yogyakarta)	Bhre Mataram	Wikramawardhana[Catatan 11]2	keponakan laki-laki
Catatan:
1 Bhre Wirabhumi sebenarnya adalah gelar: Pangeran Wirabhumi (blambangan), nama aslinya tidak diketahui dan sering disebut sebagai Bhre Wirabhumi dari Pararaton. Dia menikah dengan Nagawardhani, keponakan perempuan raja.
2 Kusumawardhani (putri raja) menikah dengan Wikramawardhana (keponakan laki-laki raja), pasangan ini lalu menjadi pewaris tahta.


Arca dewi Parwati sebagai perwujudan anumerta Tribhuwanattunggadewi, ratu Majapahit ibunda Hayam Wuruk.
Menurut prasasti Waringin Pitu
Sedangkan dalam Prasasti Waringin Pitu (1447 M) disebutkan bahwa pemerintahan Majapahit dibagi menjadi 14 daerah bawahan, yang dipimpin oleh seseorang yang bergelar Bhre.[92] Daerah-daerah bawahan tersebut yaitu:

Kahuripan
Daha
Tumapel
Wengker
Matahun
Wirabumi
Kabalan
Kembang Jenar
Pajang
Jagaraga
Keling
Kelinggapura
Singhapura
Tanjungpura
Menurut kitab Sulalatus Salatin dan buku Suma Oriental
Berdasarkan sumber-sumber luar, seperti Sulalatus Salatin dan buku Suma Oriental ciptaan Tome Pires. Daerah-daerah ini termasuk :

Indragiri di Sumatra dan Siantan (sekarang Pontianak pada pesisir barat Kalimantan), yang menurut Sulalatus Salatin, diberikan sebagai hadiah pernikahan kepada Kesultanan Malaka atas pernihkahan sultan Mansur Syah dari Malaka dengan putri Majapahit. Sultan Mansur Syah memerintah pada tahun 1459–1477, sehingga pada tahun 1447 artinya Indragiri dan Siantan masih dibawah kekuasaan Majapahit.
Jambi dan Palembang, yang hanya mulai lepas dari genggaman Majapahit ketika diambil-alih oleh Kesultanan Demak[27](hlm.154-155) pada saat masa perangnya melawan Majapahit yang diperintah Ranawijaya.
Dan Bali yang merupakan daerah pengungsian terakhir para bangsawan, seniman, pendeta dan penduduk agama Hindu di Jawa ketika Majapahit runtuh oleh Demak.
Menurut Hikayat Banjar
Wilayah Majapahit yang dicatat Hikayat Banjar adalah: Jawa, Bantan (Banten), Palembang, Mangkasar (Makassar), Pahang, Patani, Bali, Pasai, Campa, Maningkabau (Minangkabau),[93] Jambi, Bugis (daerah suku Bugis), Johor, dan Acih (Aceh).[Catatan 12][94]

Hubungan diplomatik
Hubungan diplomatik dengan negara lain dijelaskan dari Kakawin Nagarakretagama pupuh 15, bait 1 sampai 3.[95] Lengkapnya ialah:

Jawa Kuno	Alih bahasa inggris	Alih bahasa Indonesia
nahan / lwir ning deśantara kacaya de śri narapati, tuhun / tang syangkayodyapura kimutang darmmanagari, marutma mwang ring rajapura nguniweh singhanagari, ri campa kambojanyat i yawana mitreka satata	Such is the aspect of the other countries, protected by the Illustrious Prince;
verily, to be sure: Syangkayodhyapura, together with Dharmanagari, Marutma and Rajapura, and Singhanagari too, Campa, Kamboja. Different is Yawana, that is a friend, regular

Begitulah aspek dari negara-negara lain, yang dilindungi oleh Sri Narapati;
sesungguhnya, yang pasti: Syangkayodhyapura, bersama dengan Dharmanagari, Marutma dan Rajapura, dan juga Singhanagari, Campa, Kamboja. Yang berbeda Yawana, yang merupakan teman, sekutu

kunong tekang nusa madura tatan ilwing parapuri, ri denyan tungal / mwang yawadarani rakwaikana danu, samudra(1) nanggung(2) bhumi(3) kta śaka kalanya karengö, teweknyan dadyapantara sasiki tatwanya tan adoh	Concerning now this island of Madura, this is not at all of the same aspect as the foreign kingdoms,
because of the fact that it has been one with the Yawa-country, so it is said, at that time in the past: "The oceans carry a country" (124 = 202 A.D.), such is their Shaka-year, one hears, their moment to become provided with an interstice; (nevertheless) they are one in essence, not far away (from each other).

Mengenai pulau Madura sekarang ini, sama sekali tidak sama dengan kerajaan-kerajaan asing,
karena ia telah menjadi satu dengan negara Yawa, maka dikatakan, pada waktu itu di masa lalu: "Lautan membawa sebuah negara" (124 saka = 202 M), demikianlah tahun Saka mereka, terdengar, saat mereka terpisah; (namun) mereka satu pada hakekatnya, tidak jauh (satu sama lain).

huwus rabdang dwipantara sumiwi ri śri narapati, padasthity awwat / pahudama wijil anken / pratimasa, sake kotsahan / sang prabhu ri sakhahaywanyan iniwö, bhujangga mwang mantrinutus umahalot / patti satata.	Already the other continents are getting ready to show obedience to the Illustrious Prince,
alike orderly they bring in all kinds of products every ordained season. As an instance of the honoured Prabhu's exertion for all the good that is taken care of by him, ecclesiastical officers and mandarins are sent to fetch the produce regularly.

Benua lain sudah bersiap-siap untuk menunjukkan kepatuhan kepada Sri Narapati,
sama-sama teratur mereka membawa segala jenis hasil bumi setiap musim yang ditentukan. Sebagai contoh usaha Sang Prabhu yang terhormat untuk semua kebaikan yang diurusnya, bujangga dan para pegawai dikirim untuk mengambil hasil bumi secara teratur.

Pola kesatuan politik khas sejarah Asia Tenggara purba seperti ini kemudian diidentifikasi oleh sejarahwan modern sebagai "mandala", yaitu kesatuan yang politik ditentukan oleh pusat atau inti kekuasaannya daripada perbatasannya, dan dapat tersusun atas beberapa unit politik bawahan tanpa integrasi administratif lebih lanjut.[96] Daerah-daerah bawahan yang termasuk dalam lingkup mandala Majapahit, yaitu wilayah Mancanegara dan Nusantara, umumnya memiliki pemimpin asli penguasa daerah tersebut yang menikmati kebebasan internal cukup luas. Wilayah-wilayah bawahan ini meskipun sedikit-banyak dipengaruhi Majapahit, tetap menjalankan sistem pemerintahannya sendiri tanpa terintegrasi lebih lanjut oleh kekuasaan pusat di ibu kota Majapahit. Pola kekuasaan mandala ini juga ditemukan dalam kerajaan-kerajaan sebelumnya, seperti Sriwijaya dan Angkor, serta mandala-mandala tetangga Majapahit yang sezaman; Ayutthaya dan Champa.

Wilayah mancanegara atau luar negeri disebut pada Nagarakretagama pupuh 15 bait 1. Wilayah-wilayah itu antara lain Syangka (Siam), Ayodyapura (Ayutthaya), Dharmmanagari (Ligor), Marutma (Martaban atau Mergui), Rajapura (Rajpuri di selatan Siam), Singhanagari (Singhapuri di cabang sungai Menam), Campa, dan Kamboja.[97]:35-36 Hubungan antara Majapahit dengan wilayah-wilayah ini disebut kachaya, yang berarti "terkena cahaya". Ini diartikan sebagai dilindungi atau dinaungi. Istilah "wilayah dilindungi" dalam tatanegara modern disebut sebagai wilayah protektorat.[98]:234-235

Selain itu, pada pupuh 83 bait 4 dan 93 bait 1 disebut tempat-tempat yang menjadi asal para saudagar dan cendekiawan. Wilayah-wilayah itu adalah Jambudwipa (India), Cina, Karnataka (India Selatan), dan Goda (Gauḍa).[97]:35-36 Yang berbeda sendiri adalah Yawana, sebagaimana dikatakan anyat i yawana mitreka satata (yang lain adalah Yawana yang merupakan sekutu tetap).[98]:234 Kern dan Pigeaud menganggap Yawana adalah Annam, tetapi mencatat bahwa Yawana adalah istilah Sanskerta untuk Yunani (Ionian), yang digunakan orang India untuk merujuk pada orang barbar. Kern mencatat orang India menyebut orang Muslim sebagai Yawana. Menurut Pigeaud, agak tidak mungkin Yawana merujuk pada orang Muslim. Ia menganggap Yawana sebagai Annam, karena pada waktu itu raja-raja Annam sangat kuat dan sangat aneh jika meminta perlindungan kepada Jawa.[97]:35 Irawan Djoko Nugroho menolak pendapat ini, karena Nagarakretagama dibuat tahun 1365, dan kekuatan Champa melebihi Annam (yang waktu itu merujuk pada Dai Viet). Majapahit yang mengalahkan Mongol tidak mungkin memiliki sekutu tetap yang lemah. Selain itu, Annam dalam bahasa Jawa kuno memiliki nama sendiri yakni Koci (sekarang disebut Cochinchina untuk membedakannya dari Kochi di India). Koci berasal dari bahasa Cina Jiāozhǐ, dalam bahasa Kanton Kawci, dan disebut Giao Chỉ di Vietnam. Oleh karena itu, Yawana lebih tepat diartikan sebagai Arab.[99][100][101]

Daftar pejabat
Raja-raja Majapahit

Silsilah wangsa Rajasa, keluarga penguasa Singhasari dan Majapahit. Penguasa ditandai dalam gambar ini.[102]
Para penguasa Majapahit adalah penerus dari keluarga kerajaan Singhasari, yang dirintis oleh Sri Ranggah Rajasa, pendiri Wangsa Rajasa pada akhir abad ke-13. Berikut adalah daftar penguasa Majapahit. Perhatikan bahwa terdapat periode kekosongan antara pemerintahan Rajasawardhana (penguasa ke-8) dan Girishawardhana yang mungkin diakibatkan oleh krisis suksesi yang memecahkan keluarga kerajaan Majapahit menjadi dua kelompok.[8]

Nama Raja	Gelar	Tahun
Raden Wijaya	Nararya Sanggramawijaya Sri Maharaja Kertarajasa Jayawardhana	1293–1309
Jayanagara	Sri Maharaja Wiralandagopala Sri Sundarapandya Dewa Adhiswara	1309–1328
Tribhuwana Wijayatunggadewi	Sri Tribhuwanottunggadewi Maharajasa Jayawisnuwardhani	1328–1350
Hayam Wuruk	Maharaja Sri Rajasanagara	1350–1389
Wikramawardhana	Bhra Hyang Wisesa Aji Wikramawardhana	1389–1429
Suhita	Prabu Sri Suhita	1429–1447
Kertawijaya	Sri Maharaja Wijaya Parakramawardhana
1447–1451
Rajasawardhana	Rajasawardhana Sang Sinagara	1451–1453
Girishawardhana	Girishawardhana Dyah Suryawikrama	1456–1466
Suraprabhawa	Sri Adi Suraprabhawa Singhawikramawardhana Giripati Pasutabhupati Ketubhuta	1466–1468
Bhre Kertabhumi		1468–1474
Dyah Ranawijaya	Prabhu Natha Girindrawardhana Dyah Ranawijaya
1474–1498
Daftar perdana menteri
Artikel utama: Mahapatih
1. Patih Nambi (Prasasti Penanggungan, 1296 M)

2. Patih Dyah Halayudha (Prasasti Sidateka, 1323 M)

3. Patih Arya Tadah (Prasasti Berumbung, 1329 M)

4. Patih Gajah Mada (Prasasti Batur dan Bendasari) (1357–1368) tiga tahun lamanya terjadi kekosongan jabatan Patih

5. Patih Gajah Enggon (1371–1398)

6. Patih Gajah Manguri (1398–1410)

7. Patih Gajah Lembana (1410–1413)

8. Patih Tuan Kanaka (1413–.....ada kerancuan dalam penulisan kitab Pararaton dalam hal ini)

9. Patih..... ????

10. Patih Gajah Geger (Prasasti Waringin Pitu, 1447 M)

11. Patih.... ????

12. Patih.... ????

13. Patih Wahan

14. Patih Udara

Keterangan: Nama Patih dari no. 4 Sampai no. 8, tertulis dalam Kitab Pararaton.

Dharmaputra
Artikel utama: Dharmaputra
Anggota Dharmaputra tersebut adalah Ra Kuti, Ra Semi, Ra Tanca, Ra Wedeng, Ra Yuyu, Ra Banyak, dan Ra Pangsa.

Pejabat lain yang terkenal
Nama	Jabatan	Termuat Dalam
Arya Wiraraja	Adipati Madura

Merangkap Jabatan
Pasangguhan

Prasasti Kudadu (1294)
Ranggalawe
(Arya Adikara)	Adipati Tuban

Merangkap Jabatan
Pasangguhan

Prasasti Kudadu (1294)
Lembu Sora	Patih Daha	Prasasti Penanggungan (1296)
Adityawarman
(Mpu Aditya)	Mantri Praudhatara
(Wredda Menteri)

Prasasti Manjusri (1343),
Prasasti Blitar (1330)

Mpu Nala	Tumenggung	Prasasti O.J.O. LXXXIV,
Prasasti Batur, Prasasti Bendasari (O.J.O. LXXXV), Prasasti Sekar

Sumber[103]

Daftar istilah
Nusantara
Artikel utama: Nusantara
Hasta Mandala
Artikel utama: Hasta Mandala
Mitreka Satata
Artikel utama: Mitreka Satata
Bhinneka Tunggal Ika
Artikel utama: Bhinneka Tunggal Ika
Girindrawarddhana
Artikel utama: Girindrawarddhana
Brawijaya
Artikel utama: Brawijaya
Warisan budaya
Arca

Arca pertapa Hindu dari masa Majapahit akhir. Koleksi Museum für Indische Kunst, Berlin-Dahlem, Jerman.

Bidadari Majapahit, arca emas apsara gaya Majapahit menggambarkan zaman kerajaan Majapahit sebagai "zaman keemasan" Nusantara.
Candi
Candi Bajang Ratu
Candi Brahu
Candi Cetho
Candi Jabung
Candi Pari
Candi Penataran
Candi Sukuh
Candi Tikus
Candi Wringin Lawang
Karya sastra
Pada zaman Majapahit ditulis berbagai kakawin (puisi berbahasa Jawa Kuno), seperti Negarakertagama, prosa, seperti Pararaton, dan juga muncul berbagai cerita kembangan (carangan, spin off) dari epos raya India dalam bentuk kidung (seperti Tantu Panggelaran, Garudeya, dan Sudhamala) maupun cerita lisan yang populer hingga masa kini, seperti lingkaran cerita Panji, kisah Sri Tanjung, dan kisah Bhubuksah dan Gagangaking. Berbagai ukiran batu candi dari masa ini banyak mengabadikan fragmen cerita-cerita tersebut.[104]

Prasasti
Prasasti Kudadu
Prasasti Sukamerta
Prasasti Waringin Pitu
Situs
Situs Kumitir
Lain-lain

Meriam Cetbang Majapahit, dari The Metropolitan Museum of Art, yang diperkirakan berasal dari tahun 1470–1478. Perhatikan adanya lambang Surya Majapahit.
Pusat inspirasi
Majapahit telah menjadi sumber inspirasi kejayaan masa lalu bagi bangsa-bangsa Nusantara pada abad-abad berikutnya.

Legitimasi politik
Kesultanan-kesultanan Islam Demak, Pajang, dan Mataram berusaha mendapatkan legitimasi atas kekuasaan mereka melalui hubungan ke Majapahit. Demak menyatakan legitimasi keturunannya melalui Kertabhumi; pendirinya, Raden Patah, menurut babad-babad keraton Demak dinyatakan sebagai anak Kertabhumi dan seorang Putri Cina, yang dikirim ke luar istana sebelum ia melahirkan. Penaklukan Mataram atas Wirasaba tahun 1615 yang dipimpin langsung oleh Sultan Agung sendiri memiliki arti penting karena merupakan lokasi ibu kota Majapahit. Keraton-keraton Jawa Tengah memiliki tradisi dan silsilah yang berusaha membuktikan hubungan para rajanya dengan keluarga kerajaan Majapahit—sering kali dalam bentuk makam leluhur, yang di Jawa merupakan bukti penting—dan legitimasi dianggap meningkat melalui hubungan tersebut. Bali secara khusus mendapat pengaruh besar dari Majapahit, dan masyarakat Bali menganggap diri mereka penerus sejati kebudayaan Majapahit.[76]

Para penggerak nasionalisme Indonesia modern, termasuk mereka yang terlibat Gerakan Kebangkitan Nasional di awal abad ke-20, telah merujuk pada Majapahit, disamping Sriwijaya, sebagai contoh gemilang masa lalu Indonesia. Majapahit kadang dijadikan acuan batas politik negara Republik Indonesia saat ini.[28]:37 Dalam propaganda yang dijalankan tahun 1920-an, Partai Komunis Indonesia menyampaikan visinya tentang masyarakat tanpa kelas sebagai penjelmaan kembali dari Majapahit yang diromantiskan.[105] Sukarno juga mengangkat Majapahit untuk kepentingan persatuan bangsa, sedangkan Orde Baru menggunakannya untuk kepentingan perluasan dan konsolidasi kekuasaan negara.[106] Sebagaimana Majapahit, negara Indonesia modern meliputi wilayah yang luas dan secara politik berpusat di pulau Jawa.[107]

Beberapa simbol dan atribut kenegaraan Indonesia berasal dari elemen-elemen Majapahit. Bendera kebangsaan Indonesia "Sang Merah Putih" atau kadang disebut "Dwiwarna" ("dua warna"), berasal dari warna Panji Kerajaan Majapahit. Demikian pula bendera armada kapal perang TNI Angkatan Laut berupa garis-garis merah dan putih juga berasal dari warna Majapahit. Semboyan nasional Indonesia, "Bhinneka Tunggal Ika", dikutip dari "Kakawin Sutasoma" yang ditulis oleh Mpu Tantular, seorang pujangga Majapahit.

Arsitektur

Sepasang patung penjaga gerbang abad ke-14 dari kuil Majapahit di Jawa Timur (Museum of Asian Art, San Francisco)
Majapahit memiliki pengaruh yang nyata dan berkelanjutan dalam bidang arsitektur di Indonesia. Penggambaran bentuk paviliun (pendopo) berbagai bangunan di ibu kota Majapahit dalam kitab Negarakretagama telah menjadi inspirasi bagi arsitektur berbagai bangunan keraton di Jawa serta Pura dan kompleks perumahan masyarakat di Bali masa kini. Meskipun bata merah sudah digunakan jauh lebih awal, para arsitek Majapahitlah yang menyempurnakan teknik pembuatan struktur bangunan bata ini.

Beberapa elemen arsitektur kompleks bangunan di Jawa dan Bali diketahui berasal dari masa Majapahit. Misalnya gerbang terbelah candi bentar yang kini cenderung dikaitkan dengan arsitektur Bali, sesungguhnya merupakan pengaruh Majapahit, sebagaimana ditemukan pada Candi Wringin Lawang, salah satu candi bentar tertua di Indonesia. Demikian pula dengan gapura paduraksa (kori agung) beratap tinggi, dan pendopo berlandaskan struktur bata. Pengaruh citarasa estetika dan gaya bangunan Majapahit dapat dilihat pada kompleks Keraton Kasepuhan di Cirebon, Masjid Menara Kudus di Jawa Tengah, dan Pura Maospait di Bali. Tata letak kompleks bangunan berupa halaman-halaman berpagar bata yang dihubungkan dengan gerbang dan ditengahnya terdapat pendopo, merupakan warisan arsitektur Majapahit yang dapat ditemukan dalam tata letak beberapa kompleks keraton di Jawa serta kompleks puri (istana) dan pura di Bali.

Kesenian modern
Kebesaran kerajaan ini dan berbagai intrik politik yang terjadi pada masa itu menjadi sumber inspirasi tidak henti-hentinya bagi para seniman masa selanjutnya untuk menuangkan kreasinya, terutama di Indonesia. Berikut adalah daftar beberapa karya seni yang berkaitan dengan masa tersebut.

Puisi lama
Serat Darmagandhul, sebuah kitab yang tidak jelas penulisnya karena menggunakan nama pena Ki Kalamwadi, namun diperkirakan dari masa Kasunanan Surakarta. Kitab ini berkisah tentang hal-hal yang berkaitan dengan perubahan keyakinan orang Majapahit dari agama sinkretis "Hindu" ke Islam dan sejumlah ibadah yang perlu dilakukan sebagai umat Islam.
Komik dan strip komik
Serial "Mahesa Rani" karya Teguh Santosa yang dimuat di Majalah Hai, mengambil latar belakang pada masa keruntuhan Singhasari hingga awal-awal karier Mada (Gajah Mada), adik seperguruan Lubdhaka, seorang rekan Mahesa Rani.
Komik/Cerita bergambar Imperium Majapahit, karya Jan Mintaraga.
Komik Majapahit karya R.A. Kosasih
Strip komik "Panji Koming" karya Dwi Koendoro yang dimuat di surat kabar "Kompas" edisi Minggu, menceritakan kisah sehari-hari seorang warga Majapahit bernama Panji Koming.
Komik "Dharmaputra Winehsuka", karya Alex Irzaqi, kisah Ra Kuti dan Ra Semi dalam latar peristiwa pemerontakan Nambi 1316 M.
Roman/novel sejarah
Sandyakalaning Majapahit (1933), roman sejarah dengan setting masa keruntuhan Majapahit, karya Sanusi Pane.
Pelangi Di langit Singasari (1968–1974), roman sejarah dengan setting zaman kerajaan Kediri dan Singasari, karya S. H. Mintardja.
Bara Di Atas Singgasana, roman sejarah dengan setting zaman kerajaan singasari dan Majapahit, karya S. H. Mintardja
Kemelut Di Majapahit, roman sejarah dengan setting masa kejayaan Majapahit, karya Asmaraman S. Kho Ping Hoo.
Zaman Gemilang (1938/1950/2000), roman sejarah yang menceritakan akhir masa Singasari, masa Majapahit, dan berakhir pada intrik seputar terbunuhnya Jayanegara, karya Matu Mona/Hasbullah Parinduri.
Senopati Pamungkas (1986/2003), cerita silat dengan setting runtuhnya Singhasari dan awal berdirinya Majapahit hingga pemerintahan Jayanagara, karya Arswendo Atmowiloto.
Arus Balik (1995), sebuah epos pasca kejayaaan Nusantara pada awal abad 16, karya Pramoedya Ananta Toer.
Dyah Pitaloka - Senja di Langit Majapahit (2005), roman karya Hermawan Aksan tentang Dyah Pitaloka Citraresmi, putri dari Kerajaan Sunda yang gugur dalam Peristiwa Bubat.
Gajah Mada (2005), sebuah roman sejarah berseri yang mengisahkan kehidupan Gajah Mada dengan ambisinya menguasai Nusantara, karya Langit Kresna Hariadi.
Jung Jawa (2009), sebuah antologi cerita pendek berlatar Nusantara, karya Rendra Fatrisna Kurniawan, diterbitkan Babel Publishing dengan ISBN 978-979-25-3953-0.
Film/sinetron
Tutur Tinular, suatu adaptasi film karya S. Tidjab dari serial sandiwara radio. Kisah ini berlatar belakang Kerajaan Singhasari pada pemerintahan Kertanegara hingga Majapahit pada pemerintahan Jayanagara.
Saur Sepuh, suatu adaptasi film karya Niki Kosasih dari serial sandiwara radio yang populer pada kurun dasawarsa pertengahan 1980-an hingga awal 1990-an. Film ini sebetulnya lebih berfokus pada sejarah Pajajaran namun berkait dengan Majapahit pula.
Walisanga, sinetron Ramadan tahun 2003 yang berlatar Majapahit pada masa Brawijaya V hingga Kesultanan Demak pada zaman Sultan Trenggana.
Puteri Gunung Ledang, sebuah film Malaysia tahun 2004, mengangkat cerita berdasarkan legenda Melayu terkenal, Puteri Gunung Ledang. Film ini menceritakan kisah percintaan Gusti Putri Retno Dumilah, seorang putri Majapahit, dengan Hang Tuah, seorang perwira Kesultanan Malaka.
Permainan video
Civilization V: Brave New World yang terbit pada Juli 2013, terdapat peradaban Indonesia dengan tokoh pemimpinnya Gajah Mada. Meskipun dinamakan peradaban 'Indonesia', namun perdaban ini menggunakan Surya Majapahit sebagai simbolnya. Peradaban ini memiliki bangunan unik yaitu Candi, yang memiliki ikon bergambar Candi bentar di Trowulan, Mojokerto.
Kemudian pada Civilization VI sebuah DLC memiliki salah satu pemimpin Majapahit, Dyah Gitarja sebagai pemimpin peradaban Indonesia dengan simbolnya berupa Surya Majapahit yang lebih sederhana. Unit unik untuk peradaban ini adalah jong, yang menggantikan frigate.
Age of Empires II: The Age of Kings ekspansi keempat Rise of Rajas yang terbit pada Desember 2016, menampilkan misi sebagai Gajah Mada, dari awal pendirian Majapahit mengusir tentara Mongolia dan Kediri (Kerajaan Singhasari), menaklukkan kerajaan-kerajaan lain di kepulauan Nusantara setelah Sumpah Palapa hingga peristiwa Perang Bubat yang mengakhiri karier Gajah Mada sebagai Mahapatih kerajaan Majapahit. Bangunan Candi bentar, Gapura Bajang Ratu serta Candi Kalasan ditampilkan secara visual pada misi Gajah Mada. Gajah Mada juga muncul di Age of Empires II Definitive Edition yang dirilis pada November 2019.
Bendera Majapahit, Getih-Getah Samudra atau Gula Kelapa, ada dalam Age of Empires III Definitive Edition (rilis Oktober 2020) sebagai bendera untuk Indonesia, sebuah negara revolusioner yang hadir bagi peradaban Belanda dan Portugis. Sebuah unit bernama Cetbang Cannon tersedia untuk Indonesia.
Catatan
 Literatur istana yang terpengaruh budaya India menggunakan nama Sanskerta ini, yang berarti sama dengan kata "Majapahit", contohnya pada Nagarakretagama pupuh 1 bait 2 dan Kidung Harsawijaya. Kadang-kadang juga ditulis secara terbalik sebagai Tiktawilwa, contohnya pada Nagarakretagama pupuh 18 bait 4. Meskipun begitu kekaisaran ini tetap dikenal dengan nama Jawanya, seperti yang dicatat dalam hikayat-hikayat dari Aceh, Banjar, Melayu, Palembang, dan lain-lain.
 Menurut sumber Cina, terjadi pada 1376 atau 1377, menurut sumber Jawa sekitar tahun 1361. Lihat Krom, N. J. (1931). Hindoe-Javaansche Geschiedenis (Sejarah Hindu-Jawa) (edisi ke-2). The Hague. Hal. 412.
 Tahunnya ditandai dengan candrasengkala "sirna ilang kertaning bumi" (sirna = 0, ilang = 0, kerta = 4, bumi = 1). Berarti tahun 1400 saka atau 1478 masehi.
 Salah satu jenis peluru sebar—saat ditembak mengeluarkan semburan api, serpihan dan butiran peluru, dan bisa juga panah. Ciri-ciri proyektil ini adalah pelurunya tidak menutupi keseluruhan lubang laras. Needham, Joseph (1986). Science and Civilisation in China, Volume 5: Chemistry and Chemical Technology, Part 7, Military Technology: The Gunpowder Epic. Cambridge: Cambridge University Press. Hal. 9 dan 220.
 Dadap memiliki 2 arti: Dalam bahasa Indonesia, ia merujuk pada perisai bulat yang terbuat dari kulit atau rotan (Departemen Pendidikan Nasional (2008). Kamus Besar Bahasa Indonesia Pusat Bahasa Edisi Keempat. Jakarta: PT Gramedia Pustaka Utama. Dapat diakses secara daring di https://kbbi.kemdikbud.go.id/entri/dadap), sedangkan dalam bahasa Jawa kuno ia merujuk pada pada perisai penangkis panjang dan sempit (lihat Zoetmulder, Petrus Josephus (1982). Old Javanese-English dictionary. The Hague: Martinus Nijhoff. hlm. 345). Dadap di Jawa sepertinya merujuk pada perisai panjang yang cukup berat, mungkin dengan ujung menonjol (Jákl, 2014: 77–78).
 Untuk arti sodok, lihat Gardner, Gerald Brosseau (1936). Keris and Other Malay Weapons Singapore: Progressive Publishing Company. hlm. 85.
 Rampuk kemungkinan berasal dari bahasa Jawa Kuno rampog dan ngrampog, yang artinya “menyerang dalam jumlah besar”. Watang parampogan dalam bahasa Jawa kuno berarti tombak yang digunakan dalam parampogan, yaitu penombakan harimau (rampokan macan). Lihat Zoetmulder, Petrus Josephus (1982). Old Javanese-English dictionary. The Hague: Martinus Nijhoff. h. 1499.
 Kawaca memiliki dua makna. Yang pertama adalah kemeja yang dikenakan oleh para rohaniawan, yang lainnya berarti baju besi. Lihat Nugroho, Irawan Djoko (2011). Hal. 386.
 Pordenone menyebutkan bahwa Raja Jawa memerintah atas "tujuh raja yang bermahkota", mungkin merujuk pada Bhattara Saptaprabhu atau tujuh Bhattara atau Bhre (Adipati / Adipati Wanita), yang merupakan tujuh penatua berpengaruh yang memerintah tujuh nagara atau kerajaan daerah, sesuai dengan provinsi Majapahit di Jawa Timur dan Tengah; yaitu Kahuripan, Daha, Tumapel, Wengker, Lasem, Pajang, dan Mataram.
 Bhre Wirabhumi sebenarnya adalah gelar: Adipati dari Wirabhumi (Blambangan), nama aslinya tidak diketahui. Ia disebut sebagai Bhre Wirabhumi di Pararaton. Dia menikahi Nagawardhani, keponakan raja.
 Kusumawardhani (putri raja) menikah dengan Wikramawardhana (keponakan raja), pasangan ini menjadi ahli waris bersama.
 Kutipan Hikayat Banjar 1047: "Maka raja Majapahit itu bartambah-tambah kabasarannya, banyak raja-raja yang takluk kapadanya itu: sakaliannya orang tanah Jawa dan Bantan, Jambi, Palembang, Mangkasar, Pahang, Patani dan Bali dan Pasai dan Campa, sampai kapada orang tanah Maningkabau tatkala pada zaman itu sama takluk pada raja Majapahit ...", dan Kutipan Hikayat Banjar 3119: "Tunggul Amatung mangkubuminya Patih Gajah Mada itu, sakaliannya orang besar-besar di tanah Jawa itu sama takluk pada raja Tunggul Amatung itu. Bantan, Jambi, Palembang, Bugis, Mangkasar, Johor, Patani, Pahang, Campa, Maningkabau, Acih, Pasai, sakaliannya nagri itu sama takluk pada raja Tunggul Amatung itu."
Kutipan
 D.G.E. Hall (1956). "Problems of Indonesian Historiography". Pacific Affairs. 38 (3/4): 353—359.
 Ricklefs (1991), halaman 19
 Prapantja, Rakawi, trans. by Theodore Gauthier Pigeaud, Java in the 14th Century, A Study in Cultural History: The Negara-Kertagama by Pakawi Parakanca of Majapahit, 1365 AD (The Hague, Martinus Nijhoff, 1962), vol. 4, hlm. 29. 34; G.J. Resink, Indonesia’s History Between the Myths: Essays in Legal History and Historical Theory (The Hague: W. van Hoeve, 1968), hal. 21.
 Taylor, Jean Gelman (2003). Indonesia: Peoples and Histories. New Haven and London: Yale University Press. hlm. 29. ISBN 0-300-10518-5.
 Ricklefs (1991), page 18
 Johns, A.H. (1964). "The Role of Structural Organisation and Myth in Javanese Historiography". The Journal of Asian Studies. 24 (1): 91–99.
 Nagarakretagama Diakui sebagai Memori Dunia, kompas.com
 M.C. Ricklefs, Sejarah Indonesia Modern 1200-2004, Edisi ke-3. Diterjemahkan oleh S. Wahono dkk. Jakarta: Serambi, 2005, hal. 55.
 C. C. Berg. Het rijk van de vijfvoudige Buddha (Verhandelingen der Koninklijke Nederlandse Akademie van Wetenschappen, Afd. Letterkunde, vol. 69, no. 1) Ansterdam: N.V. Noord-Hollandsche Uitgevers Maatschappij, 1962; cited in M.C. Ricklefs, A History of Modern Indonesia Since c. 1300, 2nd ed. Stanford: Stanford University Press, 1993, pages 18 and 311
 http://www.tempo.co/read/news/2010/07/01/061260022/Indonesia-Jepang-Buat-Kapal-Majapahit/ Tempo/
 http://sains.kompas.com/read/2012/12/05/19045066/Majapahit-Jajah-hingga-Semenanjung-Malaya. Kompas/
 http://www.kali-majapahit.com/
 Setiono, Benny. "Kehancuran dan Kebangkitan Martabat/ Jati Diri Etnis Tionghoa Di Indonesia (bagian 1)". Diakses tanggal 16 Juni.
 David Bor - Khubilai khan and Beautiful princesses of Tumapel 2006
 Mulyana 2006, hlm. 122
 Groeneveldt, W.P. Historical Notes on Indonesia and Malaya: Compiled from Chinese Sources. Djakarta: Bhratara, 1960.
 Slamet Muljana. Menuju Puncak Kemegahan (LKIS, 2005)
 Komandoko 2009, hlm. 16
 Poesponegoro, M.D., Notosusanto, N. (editor utama). Sejarah Nasional Indonesia. Edisi ke-4. Jilid II. Jakarta: Balai Pustaka, 1990, hal. 436.
 Munoz, Paul Michel (2006). Early Kingdoms of the Indonesian Archipelago and the Malay Peninsula. Singapore: Editions Didier Millet. hlm. 279. ISBN 9814155675.
 Drs. R. Soekmono, (1973, 5th reprint edition in 1988). Pengantar Sejarah Kebudayaan Indonesia 2, 2nd ed. Yogyakarta: Penerbit Kanisius. hlm. 72.
 Y. Achadiati S, Soeroso M.P., (1988). Sejarah Peradaban Manusia: Zaman Majapahit. Jakarta: PT Gita Karya. hlm. 13.
 Groeneveldt, Willem Pieter (1876). "Notes on the Malay Archipelago and Malacca, Compiled from Chinese Sources". Batavia: W. Bruining.
 (Indonesia) Muljana, Slamet (2005). Runtuhnya kerajaan Hindu-Jawa dan timbulnya negara-negara Islam di Nusantara. PT LKiS Pelangi Aksara. hlm. 63. ISBN 9798451163.ISBN 978-979-8451-16-4
 Ricklefs (2005), hal. 57.
 Poesponegoro & Notosusanto (1990), hal. 448-451.
 Cortesão, Armando (1944). The Suma oriental of Tomé Pires : an account of the East, from the Red Sea to Japan, written in Malacca and India in 1512-1515 ; and, the book of Francisco Rodrigues, rutter of a voyage in the Red Sea, nautical rules, almanack and maps, written and drawn in the East before 1515 volume I. London: The Hakluyt Society. ISBN 9784000085052.  Artikel ini memuat teks dari sumber tersebut, yang berada dalam ranah publik.
 Ricklefs, Merle Calvin (2008). A History of Modern Indonesia Since c. 1200 Fourth Edition (E-Book version). New York: Palgrave Macmillan. ISBN 9780230546851.
 Robert W. Hefner (1983). "Ritual and Cultural Reproduction in Non-Islamic Java". American Ethnologist. 10 (1983): 665––683. doi:10.1525/ae.1983.10.4.02a00030. Diakses tanggal 2008-10-23.
 Poesponegoro, Marwati Djoened; Notosusanto, Nugroho (2019) [2008]. Sejarah Nasional Indonesia Edisi Pemutakhiran Jilid 3: Zaman Pertumbuhan dan Perkembangan Kerajaan Islam di Indonesia. Jakarta: Balai Pustaka.
 Jákl, Jiří (2014). Literary Representations of War and Warfare in Old Javanese Kakawin Poetry (Tesis). The University of Queensland.
 Bankoff, Greg; Swart, Sandra (2007). "1. Breeds of Empire and the 'Invention' of the Horse". Breeds of Empire: The 'Invention' of the Horse in Southern Africa and Maritime Southeast Asia, 1500–1950. 42 (NIAS studies in Asian topics). Copenhagen: NIAS. hlm. 1–20. ISBN 978-8-7769-4014-0.
 Pramono, Djoko (2005). Budaya Bahari. Gramedia Pustaka Utama. ISBN 9789792213768.
 Averoes, Muhammad (2020). Antara Cerita dan Sejarah: Meriam Cetbang Majapahit. Jurnal Sejarah, 3(2), 89 - 100.
 Manguin, Pierre-Yves (1976). "L'Artillerie legere nousantarienne: A propos de six canons conserves dans des collections portugaises". Arts Asiatiques. 32: 233–268.
 Manguin, Pierre-Yves (2012). Lancaran, Ghurab and Ghali. Dalam G. Wade & L. Tana (Ed.), Anthony Reid and the Study of the Southeast Asian Past (hlm. 146–182). Singapore: ISEAS Publishing.
 Ras, Johannes Jacobus, 1968, Hikayat Bandjar. A Study in Malay Historiography. The Hague (Bibliotheca Indonesica, 1)
 Nugroho 2011, hlm. 204-205.
 Nugroho 2011, hlm. 202, 386.
 Nugroho 2011, hlm. 202.
 Nugroho, Irawan Djoko (6 August 2018). "Baju Baja Emas Gajah Mada". Nusantara Review. Diakses tanggal 14 August 2019.
 Berg, Kindung Sundāyana (Kidung Sunda C), Soerakarta, Drukkerij “De Bliksem”, 1928.
 Oktorino, Nino (2020). Hikayat Majapahit - Kebangkitan dan Keruntuhan Kerajaan Terbesar di Nusantara. Jakarta: Elex Media Komputindo. ISBN 978-623-00-1741-4.
 Berg, C. C., 1927, Kidung Sunda. Inleiding, tekst, vertaling en aanteekeningen, BKI LXXXIII : 1-161.
 Mayers (1876). "Chinese explorations of the Indian Ocean during the fifteenth century". The China Review. IV: hlm. 178.
 Crawfurd, John (1856). A Descriptive Dictionary of the Indian Islands and Adjacent Countries. Bradbury and Evans.
 Stanley, Henry Edward John (1866). A Description of the Coasts of East Africa and Malabar in the Beginning of the Sixteenth Century by Duarte Barbosa. The Hakluyt Society.
 Partington, J. R. (1999). A History of Greek Fire and Gunpowder (dalam bahasa Inggris). JHU Press. ISBN 978-0-8018-5954-0.
 Jones, John Winter (1863). The travels of Ludovico di Varthema in Egypt, Syria, Arabia Deserta and Arabia Felix, in Persia, India, and Ethiopia, A.D. 1503 to 1508. Hakluyt Society.
 Wade, G., 2009, “The horse in Southeast Asia prior to 1500 CE: Some vignettes,” in: B. G. Fragner, R. Kauz, R. Ptak and A. Schottenhammer (eds), Pferde in Asien: Geschichte, Handel und Kultur/Horses in Asia: History, Trade and Culture. Vienna, Verlag der Österreichischen Akademie der Wissenschaften: 161-177.
 Teeuw, A. and S. O. Robson (2005). Bhomāntaka. The Death of Bhoma. Leiden: KITLV Press. ISBN 9789067182539.
 Jákl, Jiří. "The Whale in Old Javanese kakawin: timiṅgila, 'elephant fish', and lĕmbwara revisited". Pandanus (dalam bahasa Inggris). 8 (2): 103–118.
 Ptak, Roderich (1999). China’s Seaborne Trade with South and Southeast Asia, 1200-1750. Ashgate. ISBN 9780860787761.
 Jákl, Jiří (2016-05-15). "The Loincloth, Trousers, and Horse-riders in Pre-Islamic Java: Notes on the Old Javanese Term Lañciṅan". Archipel (91): 185–202. doi:10.4000/archipel.312. ISSN 0044-8613.
 de Jong Boers, Bernice (2007). "4. The 'Arab' of the Indonesian Archipelago: The Famed Horse Breeds of Sumbawa". Breeds of Empire: The 'Invention' of the Horse in Southern Africa and Maritime Southeast Asia, 1500–1950. 42 (NIAS studies in Asian topics). Copenhagen: NIAS. hlm. 51–64. ISBN 978-8-7769-4014-0.
 Nugroho 2011, hlm. 178-179.
 Muljana, Raden Benedictus Slamet (1979). Nagarakretagama dan Tafsir Sejarahnya. Bhratara Karya Aksara.
 Pigeaud 1960c, hlm. 19.
 Nugroho (2011). h. 286, mengutip Hikayat Raja-Raja Pasai", 3: 98: "Sa-telah itu, maka di-suroh baginda musta'idkan segala kelengkapan dan segala alat senjata peperangan akan mendatangi negeri Pasai itu, sa-kira-kira empat ratus jong yang besar-besar dan lain daripada itu banyak lagi daripada malangbang dan kelulus.". Juga lihat Hill, A. H. (Juni 1960). "Hikayat Raja-Raja Pasai". Journal of the Malaysian Branch of the Royal Asiatic Society. 33: h. 98 dan 157: Then he directed them to make ready all the equipment and munitions of war needed for an attack on the land of Pasai - about four hundred of the largest junks, and also many barges (malangbang) and galleys.
 Tulisan dari peta Fra Mauro, 10-A13, bahasa Italia aslinya: "Circa hi ani del Signor 1420 una naue ouer çoncho de india discorse per una trauersa per el mar de india a la uia de le isole de hi homeni e de le done de fuora dal cauo de diab e tra le isole uerde e le oscuritade a la uia de ponente e de garbin per 40 çornade, non trouando mai altro che aiere e aqua, e per suo arbitrio iscorse 2000 mia e declinata la fortuna i fece suo retorno in çorni 70 fina al sopradito cauo de diab. E acostandose la naue a le riue per suo bisogno, i marinari uedeno uno ouo de uno oselo nominato chrocho, el qual ouo era de la grandeça de una bota d'anfora." [1]
 Berg, C.C. (1930). Rangga Lawe: Middeljavaansche Historische Roman: Critisch uitgegeven. Batavia: Kon. Bataviaasch Genootschap van Kunsten en Wetenschappen (Bibliotheca Javanica, 1).
 Averoes, Muhammad (2022). "Re-Estimating the Size of Javanese Jong Ship". HISTORIA: Jurnal Pendidik dan Peneliti Sejarah. 5 (1): 57–64.
 Lombard, Denys (2005). Nusa Jawa: Silang Budaya, Bagian 2: Jaringan Asia. Jakarta: Gramedia Pustaka Utama. Terjemahan bahasa Indonesia dari Lombard, Denys (1990). Le carrefour javanais. Essai d'histoire globale (The Javanese Crossroads: Towards a Global History) vol. 2. Paris: Éditions de l'École des Hautes Études en Sciences Sociales.
 Manguin, Pierre-Yves (September 1980). "The Southeast Asian Ship: An Historical Approach". Journal of Southeast Asian Studies. 11 (2): 266–276. doi:10.1017/S002246340000446X. JSTOR 20070359.
 Nugroho (2011). hlm. 271, 399–400, mengutip Sejarah Melayu, 5.4: 47: "Maka betara Majapahitpun menitahkan hulubalangnya berlengkap perahu akan menyerang Singapura itu, seratus buah jung; lain dari itu beberapa melangbing dan kelulus, jongkong, cerucuh, tongkang, tiada terhisabkan lagi banyaknya."
 Nugroho (2011). hlm. 271, 399–400, mengutip Sejarah Melayu, 10.4: 77: "... maka bagindapun segera menyuruh berlengkap tiga ratus buah jung, lain dari pada itu kelulus, pelang, jongkong, tiada terbilang lagi."
 Cortesão, Armando (1944). The Suma oriental of Tomé Pires : an account of the East, from the Red Sea to Japan, written in Malacca and India in 1512-1515 ; and, the book of Francisco Rodrigues, rutter of a voyage in the Red Sea, nautical rules, almanack and maps, written and drawn in the East before 1515 volume II. London: The Hakluyt Society.  Artikel ini memuat teks dari sumber tersebut, yang berada dalam ranah publik.
 Nugroho 2011, hlm. 266-267.
 Nugroho, Irawan Djoko (30 Juli 2018). "Replika Kapal Majapahit, Replika Untuk Menghancurkan Sejarah Bangsa". Nusantara Review. Diakses tanggal 14 Agustus 2020.
 Jones, John Winter (1863). The travels of Ludovico di Varthema in Egypt, Syria, Arabia Deserta and Arabia Felix, in Persia, India, and Ethiopia, A.D. 1503 to 1508. Hakluyt Society.
 Liebner, Horst H. (2005), "Perahu-Perahu Tradisional Nusantara: Suatu Tinjauan Perkapalan dan Pelayaran", dalam Edi, Sedyawati, Eksplorasi Sumberdaya Budaya Maritim, Jakarta: Pusat Riset Wilayah Laut dan Sumber Daya Nonhayati, Badan Riset Kelautan dan Perikanan; Pusat Penelitian Kemasyarakatan dan Budaya, Universitas Indonesia, hlm. 53–124
 Suarez, Thomas (2012). Early Mapping of Southeast Asia: The Epic Story of Seafarers, Adventurers, and Cartographers Who First Mapped the Regions Between China and India. Tuttle Publishing.
 "Teknologi Era Majapahit". Nusantara Review (dalam bahasa Inggris). 2018-10-02. Diakses tanggal 2020-06-11.
 Carta IX, 1 April 1512. Dalam Pato, Raymundo Antonio de Bulhão (1884). Cartas de Affonso de Albuquerque, Seguidas de Documentos que as Elucidam tomo I (pp. 29–65). Lisboa: Typographia da Academia Real das Sciencas. hlm. 64.
 Millet, Didier (August 2003). John Miksic, ed. Indonesian Heritage Series: Ancient History. Singapore 169641: Archipelago Press. hlm. 107. ISBN 981-3018-26-7.
 Schoppert, P., Damais, S. (1997). Di dalam Didier Millet (editor):, ed. Java Style. Paris: Periplus Editions. hlm. 33–34. ISBN 962-593-232-1.
 Yule, Sir Henry (1866). Cathay and the way thither: Being a Collection of Medieval Notices of China vol. 1. London: The Hakluyt Society.
 "Ritual Networks and Royal Power in Majapahit Java, page:100". Persee. 1996. Diakses tanggal 2010-07-14.
 "Uang Kuno Temuan Rohimin Peninggalan Majapahit". Kompas.com. 2008.
 Millet, Didier (Hardcover edition — August 2003). John Miksic, ed. Indonesian Heritage Series: Ancient History. Singapore 169641: Archipelago Press. hlm. 107. ISBN 981-3018-26-7.
 Groeneveldt, Willem Pieter (1896). "Supplementary Jottings to the "Notes on the Malay Archipelago and Malacca Compiled from Chinese Sources" by W. P. Groeneveldt". T'oung Pao. 7: 113–134. doi:10.1163/156853296X00131.
 Poesponegoro & Notosusanto (1990), hal. 434-435.
 Poesponegoro & Notosusanto (1990), hal. 431-432.
 Poesponegoro & Notosusanto (1990), hal. 220.
 Nugroho 2011, hlm. 267-293.
 Nugroho 2011, hlm. 241.
 Poesponegoro & Notosusanto (1990), hal. 451-456.
 Nugroho 2009, hlm. 142.
 Berg, C.C. (1927). Kidung Sunda. Inleiding, tekst, vertaling en aanteekeningen. BKI 83: 1–161.
 Nugroho 2009, hlm. 136-138.
 Nugroho 2011, hlm. 152.
 Nastiti, Titi Surti. Prasasti Majapahit, dalam situs www.Majapahit-Kingdom.com Diarsipkan 2008-12-22 di Wayback Machine. dari Direktorat Jenderal Sejarah dan Purbakala. Jumat, 22 Juni 2007.
 Nugroho 2009, hlm. 140.
 Ras, Johannes Jacobus (1968), Hikajat Bandjar: A Study in Malay Historiography, The Hague: Martinus Nijhoff, hlm. 1047, 3119
 "Materials for the Medieval History of Indonesia".
 Dellios, Rosita (2003-1-1). "Mandala: from sacred origins to sovereign affairs in traditional Southeast Asia" (dalam bahasa Inggris). Bond University Australia. Diakses tanggal 2011-12-11.
 Pigeaud, Theodoor Gautier Thomas (1962). Java in the 14th Century: A Study in Cultural History, Volume IV: Commentaries and Recapitulations (edisi ke-3 (revisi)). The Hague: Martinus Nijhoff. ISBN 978-94-017-7133-7.
 Nugroho, Irawan Djoko (2009). Meluruskan Sejarah Majapahit. Ragam Media.
 Nugroho 2011, hlm. 141-142.
 Reid, Anthony. Southeast Asia in the Age of Commerce. Vol 2: Expansion and Crisis. New Haven: Yale University Press, 1993. p211n.
 Gordon, Alijah (2001). The Propagation of Islam in the Indonesian-Malay Archipelago. Malaysian Sociological Research Institute. hlm. 316. ISBN 9789839986624.
 Bullough, Nigel (1995). P. H., Mujiyono, ed. Historic East Java: Remains in Stone. Jakarta: ADLine Communications. hlm. 116–117.
 Mulyana, Slamet (2006). Tafsir sejarah nagarakretagama (dalam bahasa Indonesia). PT LKiS Pelangi Aksara. hlm. 173 – 209. ISBN 978-979-2552-546.
 Munandar AA. 2004. KARYA SASTRA JAWA KUNO YANG DIABADIKAN PADA RELIEF CANDI-CANDI ABAD KE-13—15 M. MAKARA, SOSIAL HUMANIORA, VOL. 8, NO. 2, AGUSTUS 2004: 54-60.
 Ricklefs, hal. 363
 Friend, Theodore (2003). Indonesian Destinies. Cambridge, Massachusetts and London: Belknap Press, Harvard University Press. hlm. p.19. ISBN 0-674-01137-6.
 "Kerajaan Majapahit". Sejarah Kerajaan. Diakses tanggal 8 August 2021.
Referensi
Hall, D.G.E. (1981). A History of South-East Asia (edisi ke-4th). London: The Macmillan Press Ltd. ISBN 978-1-349-16521-6.
Komandoko, Gamal (2009). Gajah Mada: menangkis ancaman pemberontakan Ra Kuti: kisah ketangguhan seorang patih Majapahit dalam menjaga keutuhan takhta sang raja (dalam bahasa Indonesia). Penerbit Narasi. hlm. 122. ISBN 978-979-164-145-2 Periksa nilai: checksum |isbn= (bantuan).
Muljana, Raden Benedictus Slamet (2005). Al-Fayyadl, Muhammad, ed. Menuju Puncak Kemegahan: Sejarah Kerajaan Majapahit. Yogyakarta: LKiS Pelangi Aksara.
Mulyana, Slamet (2006). Tafsir sejarah nagarakretagama (dalam bahasa Indonesia). PT LKiS Pelangi Aksara. hlm. 122. ISBN 978-979-2552-546.
Nugroho, Irawan Djoko (2009). Meluruskan Sejarah Majapahit. Ragam Media.
Nugroho, Irawan Djoko (2011). Majapahit Peradaban Maritim. Suluh Nuswantara Bakti. ISBN 978-602-9346-00-8.
Pigeaud, Theodoor Gautier Thomas (1960a). Java in the 14th Century: A Study in Cultural History, Volume I: Javanese Texts in Transcription (edisi ke-3rd (revised)). The Hague: Martinus Nijhoff.
Pigeaud, Theodoor Gautier Thomas (1960b). Java in the 14th Century: A Study in Cultural History, Volume II: Notes on the Texts and the Translations (edisi ke-3rd (revised)). The Hague: Martinus Nijhoff. ISBN 978-94-011-8774-9.
Pigeaud, Theodoor Gautier Thomas (1960c). Java in the 14th Century: A Study in Cultural History, Volume III: Translations (edisi ke-3rd (revised)). The Hague: Martinus Nijhoff. ISBN 978-94-011-8772-5.
Pigeaud, Theodoor Gautier Thomas (1962). Java in the 14th Century: A Study in Cultural History, Volume IV: Commentaries and Recapitulations (edisi ke-3rd (revised)). The Hague: Martinus Nijhoff. ISBN 978-94-017-7133-7.
Pigeaud, Theodoor Gautier Thomas (1963). Java in the 14th Century: A Study in Cultural History, Volume V: Glossary, General Index (edisi ke-3rd (revised)). The Hague: Martinus Nijhoff. ISBN 978-94-011-8778-7.
Prapanca, Mpu (2018). Isidora, ed. Kakawin Nagarakertagama: Teks Asli dan Terjemahan. Diterjemahkan oleh Saktiani, Damaika; Widya, Kartika; Aminullah, Zakaria Pamuji; Marginingrum, Novi; Septi, Neda (edisi ke-2nd (revised)). Yogyakarta: Narasi. ISBN 978-979-168-553-5.
Lihat pula
Kakawin Nagarakretagama
Pararaton
Kidung Sunda
Kerajaan Tumapel
Sejarah Nusantara
Gajah Mada
Museum Pusat Informasi Majapahit
Pranala luar

Wikimedia Commons memiliki media mengenai Majapahit.
(Inggris) Memories of Majapahit - memuat sejarah dan keterangan situs-situs peninggalan Majapahit.
(Indonesia) Diskusi tentang Perseteruan Ming dan Majapahit
(Indonesia) Terjemahan Naskah Asli Kitab Negarakretagama Karya Mpu Prapanca Diarsipkan 2013-06-15 di Wayback Machine. - Dari situs www.sejarahnasional.org. Diadaptasi dari Muljana, Slamet (1979). Nagarakretagama dan Tafsir Sejarahnya. Bhratara Karya Aksara.
Didahului oleh:
Kerajaan Tumapel	Kerajaan Hindu-Budha
1293–1527	Diteruskan oleh:
Demak
lbs
Kerajaan di Jawa
lbs
Sejarah imperium-imperium dunia
Lompat ke isi
Buka/tutup bilah samping

WikipediaEnsiklopedia Bebas
Telusuri Wikipedia
Buat akun baru

Perkakas pribadi

Ikuti Wikipedia bahasa Indonesia di F icon.svg Facebook, Twitter-logo.svg Twitter, Instagram simple icon.svg Instagram, dan Telegram logo.svg Telegram
Tutup
Daftar isi sembunyikan
Awal
Asal usul
Silsilah keluarga
Daftar raja-raja
Daftar pustaka
Lihat pula
Wangsa Isyana

Halaman
Pembicaraan
Baca
Perubahan tertunda
Lihat sumber
Lihat riwayat
Dari Wikipedia bahasa Indonesia, ensiklopedia bebas

Artikel ini membutuhkan rujukan tambahan agar kualitasnya dapat dipastikan. Mohon bantu kami mengembangkan artikel ini dengan cara menambahkan rujukan ke sumber tepercaya. Pernyataan tak bersumber bisa saja dipertentangkan dan dihapus.
Cari sumber: "Wangsa Isyana" – berita · surat kabar · buku · cendekiawan · JSTOR (November 2020)
Wangsa Isyana atau Dinasti Ishana adalah sebuah dinasti yang berkuasa dan memerintah di Kerajaan Medang (kaḍatwan mḍaŋ) Periode Jawa Timur pada abad ke-10 sampai masa akhir Kerajaan Kadiri (Paŋjalu) awal abad ke 12.

Wangsa Īśyāna
Daftar Keluarga Raja
Berkuasa	Jawa Timur Indonesia
Wangsa	Isyana
Agama	Hindu Buddha

Adegan keluarga kerajaan Raja dan Ratu dengan segenap abdi pengiringnya di dalam istana Medang.
Bagian dari seri mengenai
Sejarah Indonesia
Nicolaes Visscher Indiae Orientalis.jpg
Prasejarah
Kerajaan Hindu-Buddha
Kerajaan Islam
Kerajaan Kristen
Kolonialisme Eropa
Kemunculan Indonesia
Kemerdekaan
Menurut topik
Garis waktu
Flag of Indonesia.svg Portal Indonesia
lbs
Asal usul
Istilah Isyana berasal dari nama Sri Isyana Wikramadharmottunggadewa, yaitu gelar Mpu Sindok setelah menjadi raja Medang (929–947). Dinasti ini menganut agama Hindu aliran Siwa.

Berdasarkan agama yang dianut, Mpu Sindok diduga merupakan keturunan Sanjaya, pendiri Kerajaan Medang Periode Jawa Tengah. Salah satu pendapat menyebutkan bahwa Mpu Sindok adalah cucu Mpu Daksa yang memerintah sekitar tahun 910–an. Mpu Daksa sendiri memperkenalkan pemakaian Sanjayawarsa (kalender Sanjaya) untuk menunjukkan bahwa dirinya adalah keturunan asli Sanjaya. Dengan demikian, Mpu Daksa dan Mpu Sindok dapat disebut sebagai anggota Wangsa Sanjaya.

Kerajaan Medang di Jawa Tengah hancur akibat letusan Gunung Merapi menurut teori van Bammelen. Mpu Sindok kemudian memindahkan ibu kota Medang dari Mataram menuju Tamwlang. Beberapa tahun kemudian ibu kota dipindahkan lagi ke Watugaluh. Kedua istana baru itu terletak di daerah sekitar wilayah Jombang sekarang.

Mpu Sindok tidak hanya memindahkan istana Medang dari barat ke timur, tetapi ia juga dianggap telah mendirikan dinasti baru bernama Wangsa Isyana.

Namun terdapat pula pendapat yang menolak keberadaan Wangsa Sanjaya dan Wangsa Isyana, antara lain yang diajukan oleh Prof. Poerbatjaraka, Pusponegoro, dan Notosutanto. Menurut versi ini, di dalam Kerajaan Medang hanya ada satu dinasti dan keluarga yang memerintah, yaitu Wangsa Syailendra, keluarga Syailendra yang semula beragama Hindu. Lalu kemudian terpecah dengan munculnya anggota Syailendra lain yang beragama Buddha.

Dengan kata lain, versi ini berpendapat bahwa Mpu Sindok adalah anggota Wangsa Syailendra yang beragama Hindu Siwa, dan kemudian memindahkan istana Kerajaan Medang dari Jawa Tengah ke Jawa Timur.
"""

In [9]:
len(text_2) * 8 # num of bits = 1.2 juta

1208360

In [10]:
message1.input_message(text_2)

'\nLompat ke isi\nBuka/tutup bilah samping\n\nWikipediaEnsiklopedia Bebas\nTelusuri Wikipedia\nBuat akun baru\n\nPerkakas pribadi\n\nIkuti Wikipedia bahasa Indonesia di F icon.svg Facebook, Twitter-logo.svg Twitter, Instagram simple icon.svg Instagram, dan Telegram logo.svg Telegram\nTutup\nDaftar isi sembunyikan\nAwal\nSilsilah Jayakatwang\nPenguasa Gelanggelang\nPemberontakan Jayakatwang\nKekalahan Jayakatwang\nLihat pula\nReferensi\nJayakatwang\n\nHalaman\nPembicaraan\nBaca\nPerubahan tertunda\nSunting\nSunting sumber\nLihat riwayat\nDari Wikipedia bahasa Indonesia, ensiklopedia bebas\nJayakatwang\nPenguasa Gelanggelang\nBerkuasa\t1271 - 1293\nKelahiran\tDaha, Kediri\nJawa Timur\nKematian\t1293\nHujung Galuh, Surabaya, Jawa Timur\nWangsa\tIsyana\nAyah\tSastrajaya (putra Jayashaba putra Kertajaya)\nPasangan\tNararya Turuk Bali\nAnak\tArdharaja\nAgama\tHindu-Buddha\nJayakatwang adalah bupati Gelanggelang (kini termasuk wilayah Madiun) yang pada tahun 1292 memberontak dan meruntuhkan k

In [11]:
message1.keccak()[0] # Approx 1 minutes

'1be3c56878d2304e714315281f81b1b8c6fe690659974fa45d258e55e45692d12ba058db9916ec7c82878027720d30d3'

In [12]:
message1.xoodoo_permutation()

'18c480014d2140867c0c0808082a11aaaa6ad3aa98cdd28575214a9e7eae3a2d5a7d5d4dba18bb19291bc6a4c745a7d5'